In [1]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
from category_encoders import TargetEncoder
from itertools import combinations

pd.set_option('display.max_columns', None)

In [2]:
df = pd.read_csv('train.csv')

df.head()

,id,Gender,Age,Height,Weight,family_history_with_overweight,FAVC,FCVC,NCP,CAEC,SMOKE,CH2O,SCC,FAF,TUE,CALC,MTRANS,NObeyesdad
0,0,Male,24.443011,1.699998,81.669950,yes,yes,2.000000,2.983297,Sometimes,no,2.763573,no,0.000000,0.976473,Sometimes,Public_Transportation,Overweight_Level_II
1,1,Female,18.000000,1.560000,57.000000,yes,yes,2.000000,3.000000,Frequently,no,2.000000,no,1.000000,1.000000,no,Automobile,Normal_Weight
2,2,Female,18.000000,1.711460,50.165754,yes,yes,1.880534,1.411685,Sometimes,no,1.910378,no,0.866045,1.673584,no,Public_Transportation,Insufficient_Weight
3,3,Female,20.952737,1.710730,131.274851,yes,yes,3.000000,3.000000,Sometimes,no,1.674061,no,1.467863,0.780199,Sometimes,Public_Transportation,Obesity_Type_III
4,4,Male,31.641081,1.914186,93.798055,yes,yes,2.679664,1.971472,Sometimes,no,1.979848,no,1.967973,0.931721,Sometimes,Public_Transportation,Overweight_Level_II


In [3]:
TARGET = 'NObeyesdad'
cat_feats = ['Gender', 'CALC', 'CAEC']
bool_feats = ['family_history_with_overweight', 'FAVC', 'SMOKE', 'SCC']

In [4]:
num_to_be_rounded = ['Age']
cat_to_be_rounded = ['FCVC', 'NCP', 'CH2O', 'FAF', 'TUE']

In [5]:
# Make new dataframes for all the different tasks
# To avoid messing with the original

df_cat_rounded = df.copy()
df_num_rounded = df.copy()
df_2decimal = df.copy()
df_encoded = df.copy()

In [6]:
# Define the mapping function
def mapping_categorical(value):
    if value == 'yes':
        return 1
    elif value == 'Male':
        return 1
    elif value == 'Female':
        return 0
    elif value == 'Sometimes':
        return 1
    elif value == 'Frequently':
        return 2
    elif value == 'Always':
        return 3
    elif value == 'no':
        return 0
    else:
        return value

In [7]:
# For features that should be integars based off the original research paper
# Round using numpy's default function
# Round using numpy's ceil and floor functions

for feat in cat_to_be_rounded:
    df_cat_rounded[f'{feat}_round'] = df[feat].apply(np.round).apply(int)
    df_cat_rounded[f'{feat}_ceil_round'] = df[feat].apply(np.ceil).apply(int)
    df_cat_rounded[f'{feat}_floor_round'] = df[feat].apply(np.floor).apply(int)


for feat in num_to_be_rounded:
    df_num_rounded[f'{feat}_round'] = df[feat].apply(np.round).apply(int)
    df_num_rounded[f'{feat}_ceil_round'] = df[feat].apply(np.ceil).apply(int)
    df_num_rounded[f'{feat}_floor_round'] = df[feat].apply(np.floor).apply(int)


# Round Height and Weigh to 2 decimals
for feat in ['Height', 'Weight']:
    df_2decimal[f'{feat}_2dec'] = df[feat].apply(lambda x: round(x, 2))

# Label Encoding
# Apply the mapping function to the bool and string cat features
for feat in bool_feats:
    df_encoded[f'{feat}_map'] = df[feat].apply(mapping_categorical)

for feat in cat_feats:
    df_encoded[f'{feat}_map'] = df[feat].apply(mapping_categorical)

# One Hot Encode MTRANS
df_onehot = pd.get_dummies(df['MTRANS']).astype(int)
df_encoded.drop('MTRANS', axis=1, inplace=True)

In [8]:
# LabelEncode and map the Target feature
obesity_mapping = {'Insufficient_Weight' : 0,
                   'Normal_Weight': 1, 
                   'Overweight_Level_I' : 2,
                   'Overweight_Level_II' : 3,
                   'Obesity_Type_I' : 4,
                   'Obesity_Type_II' : 5,
                   'Obesity_Type_III' : 6,}

df[TARGET] = df[TARGET].map(obesity_mapping)

In [9]:
# Get a list of the rounded features
rounded_cat_feats = list(set(df_cat_rounded.columns) - set(df.columns))
rounded_num_feats = list(set(df_num_rounded.columns) - set(df.columns))
decimal_feats = list(set(df_2decimal.columns) - set(df.columns))
onehot_feats = list(set(df_onehot.columns) - set(df.columns))
mapped_feats = list(set(df_encoded.columns) - set(df.columns))
# rounded_cat_feats, rounded_num_feats, decimal_feats, onehot_feats

In [10]:
df_final = pd.concat([df_cat_rounded[rounded_cat_feats], df_num_rounded[rounded_num_feats], df_2decimal[decimal_feats], df_onehot, df_encoded[mapped_feats], df[TARGET]], axis=1)
df_final.sample(3)

,FCVC_ceil_round,CH2O_ceil_round,TUE_ceil_round,FAF_round,FAF_floor_round,NCP_ceil_round,FCVC_round,CH2O_round,NCP_floor_round,TUE_round,CH2O_floor_round,TUE_floor_round,NCP_round,FCVC_floor_round,FAF_ceil_round,Age_ceil_round,Age_floor_round,Age_round,Weight_2dec,Height_2dec,Automobile,Bike,Motorbike,Public_Transportation,Walking,SMOKE_map,family_history_with_overweight_map,SCC_map,CAEC_map,CALC_map,FAVC_map,Gender_map,NObeyesdad
15601,3,2,1,0,0,3,3,2,3,1,2,1,3,3,0,22,22,22,75.00,1.75,0,0,0,1,0,0,1,0,2,1,1,1,2
1519,3,2,2,0,0,3,3,2,3,1,2,1,3,2,0,27,26,27,112.29,1.76,0,0,0,1,0,0,1,0,1,1,1,1,5
512,3,2,1,1,1,4,3,2,3,1,2,1,4,2,1,20,19,20,49.71,1.69,0,0,0,1,0,0,0,1,1,1,0,0,0


### Target Encoding Categorical and Boolean Features

In [11]:
# TargetEncode the cat_feats
to_be_encoded = df[['Gender', 'CALC', 'CAEC', 'MTRANS', 'family_history_with_overweight', 'FAVC', 'SMOKE', 'SCC']]

te = TargetEncoder()
target_encoded = te.fit_transform(to_be_encoded, df_final[TARGET])
target_encoded

,Gender,CALC,CAEC,MTRANS,family_history_with_overweight,FAVC,SMOKE,SCC
0,3.119098,3.559936,3.658737,3.323306,3.764664,3.395322,3.256325,3.329979
1,3.398484,2.448770,0.883495,3.201188,3.764664,3.395322,3.256325,3.329979
2,3.398484,2.448770,3.658737,3.323306,3.764664,3.395322,3.256325,3.329979
3,3.398484,3.559936,3.658737,3.323306,3.764664,3.395322,3.256325,3.329979
4,3.119098,3.559936,3.658737,3.323306,3.764664,3.395322,3.256325,3.329979
...,...,...,...,...,...,...,...,...
20753,3.119098,3.559936,3.658737,3.323306,3.764664,3.395322,3.256325,3.329979
20754,3.119098,3.559936,0.883495,3.323306,0.963141,3.395322,3.256325,3.329979
20755,3.119098,2.448770,3.658737,3.323306,3.764664,3.395322,3.256325,3.329979
20756,3.119098,2.448770,3.658737,3.201188,3.764664,3.395322,3.256325,3.329979


In [12]:
# Join the target_encoded features back into df_final

df_final_2 = pd.concat([df_final, target_encoded], axis=1)
df_final_2.head()

,FCVC_ceil_round,CH2O_ceil_round,TUE_ceil_round,FAF_round,FAF_floor_round,NCP_ceil_round,FCVC_round,CH2O_round,NCP_floor_round,TUE_round,CH2O_floor_round,TUE_floor_round,NCP_round,FCVC_floor_round,FAF_ceil_round,Age_ceil_round,Age_floor_round,Age_round,Weight_2dec,Height_2dec,Automobile,Bike,Motorbike,Public_Transportation,Walking,SMOKE_map,family_history_with_overweight_map,SCC_map,CAEC_map,CALC_map,FAVC_map,Gender_map,NObeyesdad,Gender,CALC,CAEC,MTRANS,family_history_with_overweight,FAVC,SMOKE,SCC
0,2,3,1,0,0,3,2,3,2,1,2,0,3,2,0,25,24,24,81.67,1.70,0,0,0,1,0,0,1,0,1,1,1,1,3,3.119098,3.559936,3.658737,3.323306,3.764664,3.395322,3.256325,3.329979
1,2,2,1,1,1,3,2,2,3,1,2,1,3,2,1,18,18,18,57.00,1.56,1,0,0,0,0,0,1,0,2,0,1,0,1,3.398484,2.448770,0.883495,3.201188,3.764664,3.395322,3.256325,3.329979
2,2,2,2,1,0,2,2,2,1,2,1,1,1,1,1,18,18,18,50.17,1.71,0,0,0,1,0,0,1,0,1,0,1,0,0,3.398484,2.448770,3.658737,3.323306,3.764664,3.395322,3.256325,3.329979
3,3,2,1,1,1,3,3,2,3,1,1,0,3,3,2,21,20,21,131.27,1.71,0,0,0,1,0,0,1,0,1,1,1,0,6,3.398484,3.559936,3.658737,3.323306,3.764664,3.395322,3.256325,3.329979
4,3,2,1,2,1,2,3,2,1,1,1,0,2,2,2,32,31,32,93.80,1.91,0,0,0,1,0,0,1,0,1,1,1,1,3,3.119098,3.559936,3.658737,3.323306,3.764664,3.395322,3.256325,3.329979


### Extensive Feature Engineering

In [13]:
X = df_final_2.drop(TARGET, axis=1)
y = df_final_2[TARGET]

In [14]:
def gender_height_feature(row):
    if row['Gender'] == 1 and row['Height'] > 1.8:
        return 2
    elif row['Gender'] == 1 and row['Height'] < 1.64:
        return 0
    elif row['Gender'] == 0 and row['Height'] > 1.66:
        return 2
    elif row['Gender'] == 0 and row['Height'] < 1.54:
        return 0
    else:
        return 1

In [16]:
epsilon = 1e-10

def feat_engineering(df, numerical_features, binary_cols):
    df_new = df.copy()

    # Transformation of numerical features
    for feature in numerical_features:
        # Log transformation (add small value to avoid log(0))
        df_new[f"{feature}_log"] = np.log(df[feature] + 1e-10)

        # Square root transformation
        df_new[f"{feature}_sqrt"] = np.sqrt(df[feature])

        # Polynomial transformations
        df_new[f"{feature}_degree2"] = df[feature]**2
        df_new[f"{feature}_degree3"] = df[feature]**3

    # Pairwise numerical feature operations
    for i in range(len(numerical_features)):
        for j in range(i, len(numerical_features)):
            if i != j:
                feat1 = numerical_features[i]
                feat2 = numerical_features[j]

                # Multiplication
                df_new[f"{feat1}_x_{feat2}"] = df[feat1] * df[feat2]

                # Addition
                df_new[f"{feat1}_plus_{feat2}"] = df[feat1] + df[feat2]

                # Subtraction (both ways)
                df_new[f"{feat1}_minus_{feat2}"] = df[feat1] - df[feat2]
                df_new[f"{feat2}_minus_{feat1}"] = df[feat2] - df[feat1]

                # Ratio (both ways)
                df_new[f"{feat1}_div_{feat2}"] = df[feat1] / (df[feat2] + epsilon)
                df_new[f"{feat2}_div_{feat1}"] = df[feat2] / (df[feat1] + epsilon)

                # Mean
                df_new[f"Mean_{feat1}_{feat2}"] = df[[feat1, feat2]].mean(axis=1)

                # Standard Deviation
                df_new[f"Std_{feat1}_{feat2}"] = df[[feat1, feat2]].std(axis=1)

                # Percentage Sum (both ways)
                df_new[f"percent_sum_{feat1}_{feat2}"] = (df[feat1] / (df[feat1] + df[feat2] + epsilon)) * 100
                df_new[f"percent_sum_{feat2}_{feat1}"] = (df[feat2] / (df[feat1] + df[feat2] + epsilon)) * 100
                
                # Compare value of features
                df_new[f"{feat1}_gt_{feat2}"] = (df[feat1] > df[feat2]).astype(int)

    # Additional feature engineering for 3 numerical features
    for i in range(len(numerical_features)):
        for j in range(i+1, len(numerical_features)):
            for k in range(j+1, len(numerical_features)):
                feat1, feat2, feat3 = numerical_features[i], numerical_features[j], numerical_features[k]

                # Addition of two features and then subtraction a third
                df_new[f"{feat1}_plus_{feat2}_minus_{feat3}"] = df[feat1] + df[feat2] - df[feat3]
                df_new[f"{feat1}_plus_{feat3}_minus_{feat2}"] = df[feat1] + df[feat3] - df[feat2]
                df_new[f"{feat2}_plus_{feat3}_minus_{feat1}"] = df[feat2] + df[feat3] - df[feat1]

                # Addition of all three features
                df_new[f"{feat1}_plus_{feat2}_plus_{feat3}"] = df[feat1] + df[feat2] + df[feat3]

                # Addition of two features to then be subtracted from the third
                df_new[f"{feat1}_plus_{feat2}_sub_by_{feat3}"] = df[feat3] - (df[feat1] + df[feat2])
                df_new[f"{feat1}_plus_{feat3}_sub_by_{feat2}"] = df[feat2] - (df[feat1] + df[feat3])
                df_new[f"{feat2}_plus_{feat3}_sub_by_{feat1}"] = df[feat1] - (df[feat2] + df[feat3])

                # Multiplication of all three features
                df_new[f"{feat1}_x_{feat2}_x_{feat3}"] = df[feat1] * df[feat2] * df[feat3]

                # One feature divided by the sum of the other two features (three combinations)
                df_new[f"{feat1}_div_sum_of_{feat2}_and_{feat3}"] = df[feat1] / (df[feat2] + df[feat3] + epsilon)
                df_new[f"{feat2}_div_sum_of_{feat1}_and_{feat3}"] = df[feat2] / (df[feat1] + df[feat3] + epsilon)
                df_new[f"{feat3}_div_sum_of_{feat1}_and_{feat2}"] = df[feat3] / (df[feat1] + df[feat2] + epsilon)

                # One feature divided by the multiplication of the other two features (three combinations)
                df_new[f"{feat1}_div_prod_of_{feat2}_and_{feat3}"] = df[feat1] / (df[feat2] * df[feat3] + epsilon)
                df_new[f"{feat2}_div_prod_of_{feat1}_and_{feat3}"] = df[feat2] / (df[feat1] * df[feat3] + epsilon)
                df_new[f"{feat3}_div_prod_of_{feat1}_and_{feat2}"] = df[feat3] / (df[feat1] * df[feat2] + epsilon)

                # The mean of all three features
                df_new[f"Mean_of_{feat1}_{feat2}_{feat3}"] = df[[feat1, feat2, feat3]].mean(axis=1)

                # The standard deviation of all three features
                df_new[f"Std_of_{feat1}_{feat2}_{feat3}"] = df[[feat1, feat2, feat3]].std(axis=1)

    for col1, col2 in combinations(binary_cols, 2):
        df_new[f'{col1}+{col2}'] = df[col1] + df[col2]     

    # Domain Knowledge
    df_new['BMI_2dec'] = df['Weight_2dec'] / (df['Height_2dec'] ** 2)

    df_new['gender_height'] = df.apply(gender_height_feature, axis=1)

    df_new['BMI_category'] = pd.cut(df_new['BMI_2dec'], bins=[0, 18.5, 25, 27.5, 30, 35, 40, float('inf')], labels=[0, 1, 2, 3, 4, 5, 6])

    return df_new

In [29]:
numerical_feats = X.select_dtypes('float').columns
categorical_feats = X.select_dtypes('int').columns

In [30]:
new_train = feat_engineering(X, numerical_feats, categorical_feats)

In [31]:
new_train.shape

(20758, 2933)

In [32]:
spearman_high_corr_feats = list(set(['NCP_round', 'Age_floor_round', 'Age_ceil_round', 'Public_Transportation', 'Gender', 'MTRANS', 'family_history_with_overweight', 'FAVC', 'SMOKE', 'SCC', 'Height_2dec_log', 'Height_2dec_sqrt', 'Height_2dec_degree2', 'Height_2dec_degree3', 'Weight_2dec_log', 'Weight_2dec_sqrt', 'Weight_2dec_degree2', 'Weight_2dec_degree3', 'Gender_log', 'Gender_sqrt', 'Gender_degree2', 'Gender_degree3', 'CALC_log', 'CALC_sqrt', 'CALC_degree2', 'CALC_degree3', 'CAEC_log', 'CAEC_sqrt', 'CAEC_degree2', 'CAEC_degree3', 'MTRANS_log', 'MTRANS_sqrt', 'MTRANS_degree2', 'MTRANS_degree3', 'family_history_with_overweight_log', 'family_history_with_overweight_sqrt', 'family_history_with_overweight_degree2', 'family_history_with_overweight_degree3', 'FAVC_log', 'FAVC_sqrt', 'FAVC_degree2', 'FAVC_degree3', 'SMOKE_log', 'SMOKE_sqrt', 'SMOKE_degree2', 'SMOKE_degree3', 'SCC_log', 'SCC_sqrt', 'SCC_degree2', 'SCC_degree3'
,'NCP_round', 'Age_floor_round', 'Age_ceil_round', 'Public_Transportation', 'Gender', 'MTRANS', 'family_history_with_overweight', 'FAVC', 'SMOKE', 'SCC', 'Height_2dec_x_Weight_2dec', 'Height_2dec_plus_Weight_2dec', 'Height_2dec_minus_Weight_2dec', 'Weight_2dec_minus_Height_2dec', 'Height_2dec_div_Weight_2dec', 'Weight_2dec_div_Height_2dec', 'Mean_Height_2dec_Weight_2dec', 'Std_Height_2dec_Weight_2dec', 'percent_sum_Height_2dec_Weight_2dec', 'percent_sum_Weight_2dec_Height_2dec', 'Height_2dec_minus_Gender', 'Gender_minus_Height_2dec', 'Height_2dec_div_Gender', 'Gender_div_Height_2dec', 'Mean_Height_2dec_Gender', 'Std_Height_2dec_Gender', 'percent_sum_Height_2dec_Gender', 'percent_sum_Gender_Height_2dec', 'Height_2dec_plus_CALC', 'CALC_minus_Height_2dec', 'Height_2dec_div_CALC', 'CALC_div_Height_2dec', 'Mean_Height_2dec_CALC', 'Std_Height_2dec_CALC', 'percent_sum_Height_2dec_CALC', 'percent_sum_CALC_Height_2dec', 'Height_2dec_plus_CAEC', 'CAEC_minus_Height_2dec', 'Height_2dec_div_CAEC', 'CAEC_div_Height_2dec', 'Mean_Height_2dec_CAEC', 'Std_Height_2dec_CAEC', 'percent_sum_Height_2dec_CAEC', 'percent_sum_CAEC_Height_2dec', 'Height_2dec_gt_CAEC', 'Height_2dec_plus_MTRANS', 'MTRANS_minus_Height_2dec', 'Height_2dec_div_MTRANS', 'MTRANS_div_Height_2dec', 'Mean_Height_2dec_MTRANS', 'Std_Height_2dec_MTRANS', 'percent_sum_Height_2dec_MTRANS', 'percent_sum_MTRANS_Height_2dec', 'Height_2dec_gt_MTRANS', 'Height_2dec_plus_family_history_with_overweight', 'family_history_with_overweight_minus_Height_2dec', 'Height_2dec_div_family_history_with_overweight', 'family_history_with_overweight_div_Height_2dec', 'Mean_Height_2dec_family_history_with_overweight', 'Std_Height_2dec_family_history_with_overweight', 'percent_sum_Height_2dec_family_history_with_overweight', 'percent_sum_family_history_with_overweight_Height_2dec', 'Height_2dec_gt_family_history_with_overweight', 'Height_2dec_plus_FAVC', 'FAVC_minus_Height_2dec', 'Height_2dec_div_FAVC', 'FAVC_div_Height_2dec', 'Mean_Height_2dec_FAVC', 'Std_Height_2dec_FAVC', 'percent_sum_Height_2dec_FAVC', 'percent_sum_FAVC_Height_2dec', 'Height_2dec_x_SMOKE', 'Height_2dec_plus_SMOKE', 'Height_2dec_minus_SMOKE', 'SMOKE_minus_Height_2dec', 'Height_2dec_div_SMOKE', 'SMOKE_div_Height_2dec', 'Mean_Height_2dec_SMOKE', 'Std_Height_2dec_SMOKE', 'percent_sum_Height_2dec_SMOKE', 'percent_sum_SMOKE_Height_2dec', 'Height_2dec_x_SCC', 'Height_2dec_plus_SCC', 'Height_2dec_minus_SCC', 'SCC_minus_Height_2dec', 'Height_2dec_div_SCC', 'SCC_div_Height_2dec', 'Mean_Height_2dec_SCC', 'Std_Height_2dec_SCC', 'percent_sum_Height_2dec_SCC', 'percent_sum_SCC_Height_2dec', 'Height_2dec_gt_SCC', 'Weight_2dec_x_Gender', 'Weight_2dec_plus_Gender', 'Weight_2dec_minus_Gender', 'Gender_minus_Weight_2dec', 'Weight_2dec_div_Gender', 'Gender_div_Weight_2dec', 'Mean_Weight_2dec_Gender', 'Std_Weight_2dec_Gender', 'percent_sum_Weight_2dec_Gender', 'percent_sum_Gender_Weight_2dec', 'Weight_2dec_x_CALC', 'Weight_2dec_plus_CALC', 'Weight_2dec_minus_CALC', 'CALC_minus_Weight_2dec', 'CALC_div_Weight_2dec', 'Mean_Weight_2dec_CALC', 'Std_Weight_2dec_CALC', 'percent_sum_Weight_2dec_CALC', 'percent_sum_CALC_Weight_2dec', 'Weight_2dec_x_CAEC', 'Weight_2dec_plus_CAEC', 'Weight_2dec_minus_CAEC', 'CAEC_minus_Weight_2dec', 'CAEC_div_Weight_2dec', 'Mean_Weight_2dec_CAEC', 'Std_Weight_2dec_CAEC', 'percent_sum_Weight_2dec_CAEC', 'percent_sum_CAEC_Weight_2dec', 'Weight_2dec_x_MTRANS', 'Weight_2dec_plus_MTRANS', 'Weight_2dec_minus_MTRANS', 'MTRANS_minus_Weight_2dec', 'Weight_2dec_div_MTRANS', 'MTRANS_div_Weight_2dec', 'Mean_Weight_2dec_MTRANS', 'Std_Weight_2dec_MTRANS', 'percent_sum_Weight_2dec_MTRANS', 'percent_sum_MTRANS_Weight_2dec', 'Weight_2dec_x_family_history_with_overweight', 'Weight_2dec_plus_family_history_with_overweight', 'Weight_2dec_minus_family_history_with_overweight', 'family_history_with_overweight_minus_Weight_2dec', 'family_history_with_overweight_div_Weight_2dec', 'Mean_Weight_2dec_family_history_with_overweight', 'Std_Weight_2dec_family_history_with_overweight', 'percent_sum_Weight_2dec_family_history_with_overweight', 'percent_sum_family_history_with_overweight_Weight_2dec', 'Weight_2dec_x_FAVC', 'Weight_2dec_plus_FAVC', 'Weight_2dec_minus_FAVC', 'FAVC_minus_Weight_2dec', 'FAVC_div_Weight_2dec', 'Mean_Weight_2dec_FAVC', 'Std_Weight_2dec_FAVC', 'percent_sum_Weight_2dec_FAVC', 'percent_sum_FAVC_Weight_2dec', 'Weight_2dec_x_SMOKE', 'Weight_2dec_plus_SMOKE', 'Weight_2dec_minus_SMOKE', 'SMOKE_minus_Weight_2dec', 'Weight_2dec_div_SMOKE', 'SMOKE_div_Weight_2dec', 'Mean_Weight_2dec_SMOKE', 'Std_Weight_2dec_SMOKE', 'percent_sum_Weight_2dec_SMOKE', 'percent_sum_SMOKE_Weight_2dec', 'Weight_2dec_x_SCC', 'Weight_2dec_plus_SCC', 'Weight_2dec_minus_SCC', 'SCC_minus_Weight_2dec', 'SCC_div_Weight_2dec', 'Mean_Weight_2dec_SCC', 'Std_Weight_2dec_SCC', 'percent_sum_Weight_2dec_SCC', 'percent_sum_SCC_Weight_2dec', 'Gender_plus_CALC', 'CALC_minus_Gender', 'Gender_div_CALC', 'CALC_div_Gender', 'Mean_Gender_CALC', 'Std_Gender_CALC', 'percent_sum_Gender_CALC', 'percent_sum_CALC_Gender', 'Gender_gt_CALC', 'Gender_plus_CAEC', 'CAEC_minus_Gender', 'Gender_div_CAEC', 'CAEC_div_Gender', 'Mean_Gender_CAEC', 'Std_Gender_CAEC', 'percent_sum_Gender_CAEC', 'percent_sum_CAEC_Gender', 'Gender_gt_CAEC', 'Gender_x_MTRANS', 'Gender_plus_MTRANS', 'MTRANS_minus_Gender', 'Gender_div_MTRANS', 'MTRANS_div_Gender', 'Mean_Gender_MTRANS', 'percent_sum_Gender_MTRANS', 'percent_sum_MTRANS_Gender', 'Gender_gt_MTRANS', 'Gender_plus_family_history_with_overweight', 'family_history_with_overweight_minus_Gender', 'Gender_div_family_history_with_overweight', 'family_history_with_overweight_div_Gender', 'Mean_Gender_family_history_with_overweight', 'Std_Gender_family_history_with_overweight', 'percent_sum_Gender_family_history_with_overweight', 'percent_sum_family_history_with_overweight_Gender', 'Gender_gt_family_history_with_overweight', 'Gender_plus_FAVC', 'FAVC_minus_Gender', 'Gender_div_FAVC', 'FAVC_div_Gender', 'Mean_Gender_FAVC', 'Std_Gender_FAVC', 'percent_sum_Gender_FAVC', 'percent_sum_FAVC_Gender', 'Gender_gt_FAVC', 'Gender_x_SMOKE', 'Gender_plus_SMOKE', 'Gender_minus_SMOKE', 'SMOKE_minus_Gender', 'Gender_div_SMOKE', 'SMOKE_div_Gender', 'Mean_Gender_SMOKE', 'Std_Gender_SMOKE', 'percent_sum_Gender_SMOKE', 'percent_sum_SMOKE_Gender', 'Gender_gt_SMOKE', 'Gender_plus_SCC', 'Gender_minus_SCC', 'SCC_minus_Gender', 'Gender_div_SCC', 'SCC_div_Gender', 'Mean_Gender_SCC', 'Std_Gender_SCC', 'percent_sum_Gender_SCC', 'percent_sum_SCC_Gender', 'Gender_gt_SCC', 'CALC_plus_CAEC', 'CAEC_minus_CALC', 'CALC_div_CAEC', 'CAEC_div_CALC', 'Mean_CALC_CAEC', 'Std_CALC_CAEC', 'percent_sum_CALC_CAEC', 'percent_sum_CAEC_CALC', 'CALC_gt_CAEC', 'CALC_plus_MTRANS', 'MTRANS_minus_CALC', 'CALC_div_MTRANS', 'MTRANS_div_CALC', 'Mean_CALC_MTRANS', 'Std_CALC_MTRANS', 'percent_sum_CALC_MTRANS', 'percent_sum_MTRANS_CALC', 'CALC_gt_MTRANS', 'CALC_plus_family_history_with_overweight', 'family_history_with_overweight_minus_CALC', 'CALC_div_family_history_with_overweight', 'family_history_with_overweight_div_CALC', 'Mean_CALC_family_history_with_overweight', 'Std_CALC_family_history_with_overweight', 'percent_sum_CALC_family_history_with_overweight', 'percent_sum_family_history_with_overweight_CALC', 'CALC_gt_family_history_with_overweight', 'CALC_plus_FAVC', 'FAVC_minus_CALC', 'CALC_div_FAVC', 'FAVC_div_CALC', 'Mean_CALC_FAVC', 'Std_CALC_FAVC', 'percent_sum_CALC_FAVC', 'percent_sum_FAVC_CALC', 'CALC_gt_FAVC', 'CALC_x_SMOKE', 'CALC_plus_SMOKE', 'CALC_minus_SMOKE', 'SMOKE_minus_CALC', 'CALC_div_SMOKE', 'SMOKE_div_CALC', 'Mean_CALC_SMOKE', 'Std_CALC_SMOKE', 'percent_sum_CALC_SMOKE', 'percent_sum_SMOKE_CALC', 'CALC_gt_SMOKE', 'CALC_x_SCC', 'CALC_plus_SCC', 'CALC_minus_SCC', 'SCC_minus_CALC', 'CALC_div_SCC', 'SCC_div_CALC', 'Mean_CALC_SCC', 'Std_CALC_SCC', 'percent_sum_CALC_SCC', 'percent_sum_SCC_CALC', 'CALC_gt_SCC', 'CAEC_plus_MTRANS', 'MTRANS_minus_CAEC', 'CAEC_div_MTRANS', 'MTRANS_div_CAEC', 'Mean_CAEC_MTRANS', 'Std_CAEC_MTRANS', 'percent_sum_CAEC_MTRANS', 'percent_sum_MTRANS_CAEC', 'CAEC_gt_MTRANS', 'CAEC_plus_family_history_with_overweight', 'family_history_with_overweight_minus_CAEC', 'family_history_with_overweight_div_CAEC', 'Mean_CAEC_family_history_with_overweight', 'percent_sum_CAEC_family_history_with_overweight', 'percent_sum_family_history_with_overweight_CAEC', 'CAEC_plus_FAVC', 'FAVC_minus_CAEC', 'CAEC_div_FAVC', 'FAVC_div_CAEC', 'Mean_CAEC_FAVC', 'Std_CAEC_FAVC', 'percent_sum_CAEC_FAVC', 'percent_sum_FAVC_CAEC', 'CAEC_gt_FAVC', 'CAEC_x_SMOKE', 'CAEC_plus_SMOKE', 'CAEC_minus_SMOKE', 'SMOKE_minus_CAEC', 'CAEC_div_SMOKE', 'SMOKE_div_CAEC', 'Mean_CAEC_SMOKE', 'Std_CAEC_SMOKE', 'percent_sum_CAEC_SMOKE', 'percent_sum_SMOKE_CAEC', 'CAEC_gt_SMOKE', 'CAEC_x_SCC', 'CAEC_plus_SCC', 'CAEC_minus_SCC', 'SCC_minus_CAEC', 'CAEC_div_SCC', 'SCC_div_CAEC', 'Mean_CAEC_SCC', 'percent_sum_CAEC_SCC', 'percent_sum_SCC_CAEC', 'CAEC_gt_SCC', 'MTRANS_plus_family_history_with_overweight', 'family_history_with_overweight_minus_MTRANS', 'MTRANS_div_family_history_with_overweight', 'family_history_with_overweight_div_MTRANS', 'Mean_MTRANS_family_history_with_overweight', 'Std_MTRANS_family_history_with_overweight', 'percent_sum_MTRANS_family_history_with_overweight', 'percent_sum_family_history_with_overweight_MTRANS', 'MTRANS_gt_family_history_with_overweight', 'MTRANS_plus_FAVC', 'FAVC_minus_MTRANS', 'MTRANS_div_FAVC', 'FAVC_div_MTRANS', 'Mean_MTRANS_FAVC', 'Std_MTRANS_FAVC', 'percent_sum_MTRANS_FAVC', 'percent_sum_FAVC_MTRANS', 'MTRANS_gt_FAVC', 'MTRANS_x_SMOKE', 'MTRANS_plus_SMOKE', 'MTRANS_minus_SMOKE', 'SMOKE_minus_MTRANS', 'MTRANS_div_SMOKE', 'SMOKE_div_MTRANS', 'Mean_MTRANS_SMOKE', 'percent_sum_MTRANS_SMOKE', 'percent_sum_SMOKE_MTRANS', 'MTRANS_gt_SMOKE', 'MTRANS_plus_SCC', 'SCC_minus_MTRANS', 'MTRANS_div_SCC', 'SCC_div_MTRANS', 'Mean_MTRANS_SCC', 'Std_MTRANS_SCC', 'percent_sum_MTRANS_SCC', 'percent_sum_SCC_MTRANS', 'MTRANS_gt_SCC', 'family_history_with_overweight_x_FAVC', 'family_history_with_overweight_plus_FAVC', 'family_history_with_overweight_minus_FAVC', 'FAVC_minus_family_history_with_overweight', 'family_history_with_overweight_div_FAVC', 'FAVC_div_family_history_with_overweight', 'Mean_family_history_with_overweight_FAVC', 'Std_family_history_with_overweight_FAVC', 'percent_sum_family_history_with_overweight_FAVC', 'percent_sum_FAVC_family_history_with_overweight', 'family_history_with_overweight_gt_FAVC', 'family_history_with_overweight_x_SMOKE', 'family_history_with_overweight_plus_SMOKE', 'family_history_with_overweight_minus_SMOKE', 'SMOKE_minus_family_history_with_overweight', 'family_history_with_overweight_div_SMOKE', 'SMOKE_div_family_history_with_overweight', 'Mean_family_history_with_overweight_SMOKE', 'Std_family_history_with_overweight_SMOKE', 'percent_sum_family_history_with_overweight_SMOKE', 'percent_sum_SMOKE_family_history_with_overweight', 'family_history_with_overweight_gt_SMOKE', 'family_history_with_overweight_x_SCC', 'family_history_with_overweight_plus_SCC', 'family_history_with_overweight_minus_SCC', 'SCC_minus_family_history_with_overweight', 'family_history_with_overweight_div_SCC', 'SCC_div_family_history_with_overweight', 'Mean_family_history_with_overweight_SCC', 'percent_sum_family_history_with_overweight_SCC', 'percent_sum_SCC_family_history_with_overweight', 'family_history_with_overweight_gt_SCC', 'FAVC_x_SMOKE', 'FAVC_plus_SMOKE', 'FAVC_minus_SMOKE', 'SMOKE_minus_FAVC', 'FAVC_div_SMOKE', 'SMOKE_div_FAVC', 'Mean_FAVC_SMOKE', 'Std_FAVC_SMOKE', 'percent_sum_FAVC_SMOKE', 'percent_sum_SMOKE_FAVC', 'FAVC_gt_SMOKE', 'FAVC_plus_SCC', 'SCC_minus_FAVC', 'FAVC_div_SCC', 'SCC_div_FAVC', 'Mean_FAVC_SCC', 'Std_FAVC_SCC', 'percent_sum_FAVC_SCC', 'percent_sum_SCC_FAVC', 'FAVC_gt_SCC', 'SMOKE_plus_SCC', 'SCC_minus_SMOKE', 'SMOKE_div_SCC', 'SCC_div_SMOKE', 'Mean_SMOKE_SCC', 'Std_SMOKE_SCC', 'percent_sum_SMOKE_SCC', 'percent_sum_SCC_SMOKE', 'SMOKE_gt_SCC'
,'NCP_round', 'Age_round', 'Age_ceil_round', 'Public_Transportation', 'Gender', 'MTRANS', 'family_history_with_overweight', 'FAVC', 'SMOKE', 'SCC', 'Weight_2dec_plus_Height_2dec_minus_Gender', 'Weight_2dec_plus_Gender_minus_Height_2dec', 'Height_2dec_plus_Gender_minus_Weight_2dec', 'Weight_2dec_plus_Height_2dec_plus_Gender', 'Weight_2dec_plus_Height_2dec_minus_CALC', 'Weight_2dec_plus_CALC_minus_Height_2dec', 'Height_2dec_plus_CALC_minus_Weight_2dec', 'Weight_2dec_plus_Height_2dec_plus_CALC', 'Weight_2dec_plus_Height_2dec_minus_CAEC', 'Weight_2dec_plus_CAEC_minus_Height_2dec', 'Height_2dec_plus_CAEC_minus_Weight_2dec', 'Weight_2dec_plus_Height_2dec_plus_CAEC', 'Weight_2dec_plus_Height_2dec_minus_MTRANS', 'Weight_2dec_plus_MTRANS_minus_Height_2dec', 'Height_2dec_plus_MTRANS_minus_Weight_2dec', 'Weight_2dec_plus_Height_2dec_plus_MTRANS', 'Weight_2dec_plus_Height_2dec_minus_family_history_with_overweight', 'Weight_2dec_plus_family_history_with_overweight_minus_Height_2dec', 'Height_2dec_plus_family_history_with_overweight_minus_Weight_2dec', 'Weight_2dec_plus_Height_2dec_plus_family_history_with_overweight', 'Weight_2dec_plus_Height_2dec_minus_FAVC', 'Weight_2dec_plus_FAVC_minus_Height_2dec', 'Height_2dec_plus_FAVC_minus_Weight_2dec', 'Weight_2dec_plus_Height_2dec_plus_FAVC', 'Weight_2dec_plus_Height_2dec_minus_SMOKE', 'Weight_2dec_plus_SMOKE_minus_Height_2dec', 'Height_2dec_plus_SMOKE_minus_Weight_2dec', 'Weight_2dec_plus_Height_2dec_plus_SMOKE', 'Weight_2dec_plus_Height_2dec_minus_SCC', 'Weight_2dec_plus_SCC_minus_Height_2dec', 'Height_2dec_plus_SCC_minus_Weight_2dec', 'Weight_2dec_plus_Height_2dec_plus_SCC', 'Weight_2dec_plus_Gender_minus_CALC', 'Weight_2dec_plus_CALC_minus_Gender', 'Gender_plus_CALC_minus_Weight_2dec', 'Weight_2dec_plus_Gender_plus_CALC', 'Weight_2dec_plus_Gender_minus_CAEC', 'Weight_2dec_plus_CAEC_minus_Gender', 'Gender_plus_CAEC_minus_Weight_2dec', 'Weight_2dec_plus_Gender_plus_CAEC', 'Weight_2dec_plus_Gender_minus_MTRANS', 'Weight_2dec_plus_MTRANS_minus_Gender', 'Gender_plus_MTRANS_minus_Weight_2dec', 'Weight_2dec_plus_Gender_plus_MTRANS', 'Weight_2dec_plus_Gender_minus_family_history_with_overweight', 'Weight_2dec_plus_family_history_with_overweight_minus_Gender', 'Gender_plus_family_history_with_overweight_minus_Weight_2dec', 'Weight_2dec_plus_Gender_plus_family_history_with_overweight', 'Weight_2dec_plus_Gender_minus_FAVC', 'Weight_2dec_plus_FAVC_minus_Gender', 'Gender_plus_FAVC_minus_Weight_2dec', 'Weight_2dec_plus_Gender_plus_FAVC', 'Weight_2dec_plus_Gender_minus_SMOKE', 'Weight_2dec_plus_SMOKE_minus_Gender', 'Gender_plus_SMOKE_minus_Weight_2dec', 'Weight_2dec_plus_Gender_plus_SMOKE', 'Weight_2dec_plus_Gender_minus_SCC', 'Weight_2dec_plus_SCC_minus_Gender', 'Gender_plus_SCC_minus_Weight_2dec', 'Weight_2dec_plus_Gender_plus_SCC', 'Weight_2dec_plus_CALC_minus_CAEC', 'Weight_2dec_plus_CAEC_minus_CALC', 'CALC_plus_CAEC_minus_Weight_2dec', 'Weight_2dec_plus_CALC_plus_CAEC', 'Weight_2dec_plus_CALC_minus_MTRANS', 'Weight_2dec_plus_MTRANS_minus_CALC', 'CALC_plus_MTRANS_minus_Weight_2dec', 'Weight_2dec_plus_CALC_plus_MTRANS', 'Weight_2dec_plus_CALC_minus_family_history_with_overweight', 'Weight_2dec_plus_family_history_with_overweight_minus_CALC', 'CALC_plus_family_history_with_overweight_minus_Weight_2dec', 'Weight_2dec_plus_CALC_plus_family_history_with_overweight', 'Weight_2dec_plus_CALC_minus_FAVC', 'Weight_2dec_plus_FAVC_minus_CALC', 'CALC_plus_FAVC_minus_Weight_2dec', 'Weight_2dec_plus_CALC_plus_FAVC', 'Weight_2dec_plus_CALC_minus_SMOKE', 'Weight_2dec_plus_SMOKE_minus_CALC', 'CALC_plus_SMOKE_minus_Weight_2dec', 'Weight_2dec_plus_CALC_plus_SMOKE', 'Weight_2dec_plus_CALC_minus_SCC', 'Weight_2dec_plus_SCC_minus_CALC', 'CALC_plus_SCC_minus_Weight_2dec', 'Weight_2dec_plus_CALC_plus_SCC', 'Weight_2dec_plus_CAEC_minus_MTRANS', 'Weight_2dec_plus_MTRANS_minus_CAEC', 'CAEC_plus_MTRANS_minus_Weight_2dec', 'Weight_2dec_plus_CAEC_plus_MTRANS', 'Weight_2dec_plus_CAEC_minus_family_history_with_overweight', 'Weight_2dec_plus_family_history_with_overweight_minus_CAEC', 'CAEC_plus_family_history_with_overweight_minus_Weight_2dec', 'Weight_2dec_plus_CAEC_plus_family_history_with_overweight', 'Weight_2dec_plus_CAEC_minus_FAVC', 'Weight_2dec_plus_FAVC_minus_CAEC', 'CAEC_plus_FAVC_minus_Weight_2dec', 'Weight_2dec_plus_CAEC_plus_FAVC', 'Weight_2dec_plus_CAEC_minus_SMOKE', 'Weight_2dec_plus_SMOKE_minus_CAEC', 'CAEC_plus_SMOKE_minus_Weight_2dec', 'Weight_2dec_plus_CAEC_plus_SMOKE', 'Weight_2dec_plus_CAEC_minus_SCC', 'Weight_2dec_plus_SCC_minus_CAEC', 'CAEC_plus_SCC_minus_Weight_2dec', 'Weight_2dec_plus_CAEC_plus_SCC', 'Weight_2dec_plus_MTRANS_minus_family_history_with_overweight', 'Weight_2dec_plus_family_history_with_overweight_minus_MTRANS', 'MTRANS_plus_family_history_with_overweight_minus_Weight_2dec', 'Weight_2dec_plus_MTRANS_plus_family_history_with_overweight', 'Weight_2dec_plus_MTRANS_minus_FAVC', 'Weight_2dec_plus_FAVC_minus_MTRANS', 'MTRANS_plus_FAVC_minus_Weight_2dec', 'Weight_2dec_plus_MTRANS_plus_FAVC', 'Weight_2dec_plus_MTRANS_minus_SMOKE', 'Weight_2dec_plus_SMOKE_minus_MTRANS', 'MTRANS_plus_SMOKE_minus_Weight_2dec', 'Weight_2dec_plus_MTRANS_plus_SMOKE', 'Weight_2dec_plus_MTRANS_minus_SCC', 'Weight_2dec_plus_SCC_minus_MTRANS', 'MTRANS_plus_SCC_minus_Weight_2dec', 'Weight_2dec_plus_MTRANS_plus_SCC', 'Weight_2dec_plus_family_history_with_overweight_minus_FAVC', 'Weight_2dec_plus_FAVC_minus_family_history_with_overweight', 'family_history_with_overweight_plus_FAVC_minus_Weight_2dec', 'Weight_2dec_plus_family_history_with_overweight_plus_FAVC', 'Weight_2dec_plus_family_history_with_overweight_minus_SMOKE', 'Weight_2dec_plus_SMOKE_minus_family_history_with_overweight', 'family_history_with_overweight_plus_SMOKE_minus_Weight_2dec', 'Weight_2dec_plus_family_history_with_overweight_plus_SMOKE', 'Weight_2dec_plus_family_history_with_overweight_minus_SCC', 'Weight_2dec_plus_SCC_minus_family_history_with_overweight', 'family_history_with_overweight_plus_SCC_minus_Weight_2dec', 'Weight_2dec_plus_family_history_with_overweight_plus_SCC', 'Weight_2dec_plus_FAVC_minus_SMOKE', 'Weight_2dec_plus_SMOKE_minus_FAVC', 'FAVC_plus_SMOKE_minus_Weight_2dec', 'Weight_2dec_plus_FAVC_plus_SMOKE', 'Weight_2dec_plus_FAVC_minus_SCC', 'Weight_2dec_plus_SCC_minus_FAVC', 'FAVC_plus_SCC_minus_Weight_2dec', 'Weight_2dec_plus_FAVC_plus_SCC', 'Weight_2dec_plus_SMOKE_minus_SCC', 'Weight_2dec_plus_SCC_minus_SMOKE', 'SMOKE_plus_SCC_minus_Weight_2dec', 'Weight_2dec_plus_SMOKE_plus_SCC', 'Height_2dec_plus_SMOKE_minus_Gender', 'Gender_plus_SMOKE_minus_Height_2dec', 'Height_2dec_plus_Gender_plus_SMOKE', 'Height_2dec_plus_Gender_minus_SCC', 'Height_2dec_plus_SCC_minus_Gender', 'Height_2dec_plus_Gender_plus_SCC', 'CALC_plus_MTRANS_minus_Height_2dec', 'Height_2dec_plus_CALC_minus_SMOKE', 'Height_2dec_plus_SMOKE_minus_CALC', 'CALC_plus_SMOKE_minus_Height_2dec', 'Height_2dec_plus_CALC_plus_SMOKE', 'Height_2dec_plus_CALC_minus_SCC', 'Height_2dec_plus_SCC_minus_CALC', 'CALC_plus_SCC_minus_Height_2dec', 'Height_2dec_plus_CALC_plus_SCC', 'Height_2dec_plus_CAEC_plus_FAVC', 'Height_2dec_plus_CAEC_minus_SMOKE', 'Height_2dec_plus_SMOKE_minus_CAEC', 'CAEC_plus_SMOKE_minus_Height_2dec', 'Height_2dec_plus_CAEC_plus_SMOKE', 'Height_2dec_plus_CAEC_minus_SCC', 'Height_2dec_plus_SCC_minus_CAEC', 'CAEC_plus_SCC_minus_Height_2dec', 'Height_2dec_plus_CAEC_plus_SCC', 'Height_2dec_plus_family_history_with_overweight_minus_MTRANS', 'MTRANS_plus_family_history_with_overweight_minus_Height_2dec', 'Height_2dec_plus_SMOKE_minus_MTRANS', 'MTRANS_plus_SMOKE_minus_Height_2dec', 'Height_2dec_plus_MTRANS_plus_SMOKE', 'Height_2dec_plus_SCC_minus_MTRANS', 'Height_2dec_plus_MTRANS_plus_SCC', 'Height_2dec_plus_FAVC_minus_family_history_with_overweight', 'Height_2dec_plus_family_history_with_overweight_plus_FAVC', 'Height_2dec_plus_family_history_with_overweight_minus_SMOKE', 'Height_2dec_plus_SMOKE_minus_family_history_with_overweight', 'family_history_with_overweight_plus_SMOKE_minus_Height_2dec', 'Height_2dec_plus_family_history_with_overweight_plus_SMOKE', 'Height_2dec_plus_family_history_with_overweight_minus_SCC', 'Height_2dec_plus_SCC_minus_family_history_with_overweight', 'family_history_with_overweight_plus_SCC_minus_Height_2dec', 'Height_2dec_plus_family_history_with_overweight_plus_SCC', 'Height_2dec_plus_FAVC_minus_SMOKE', 'Height_2dec_plus_SMOKE_minus_FAVC', 'FAVC_plus_SMOKE_minus_Height_2dec', 'Height_2dec_plus_FAVC_plus_SMOKE', 'Height_2dec_plus_SCC_minus_FAVC', 'FAVC_plus_SCC_minus_Height_2dec', 'Height_2dec_plus_FAVC_plus_SCC', 'Height_2dec_plus_SMOKE_minus_SCC', 'Height_2dec_plus_SCC_minus_SMOKE', 'SMOKE_plus_SCC_minus_Height_2dec', 'Height_2dec_plus_SMOKE_plus_SCC', 'Gender_plus_CALC_minus_CAEC', 'Gender_plus_CAEC_minus_CALC', 'CALC_plus_CAEC_minus_Gender', 'Gender_plus_CALC_plus_CAEC', 'Gender_plus_CALC_minus_family_history_with_overweight', 'Gender_plus_family_history_with_overweight_minus_CALC', 'CALC_plus_family_history_with_overweight_minus_Gender', 'Gender_plus_CALC_plus_family_history_with_overweight', 'Gender_plus_CALC_minus_SMOKE', 'Gender_plus_SMOKE_minus_CALC', 'CALC_plus_SMOKE_minus_Gender', 'Gender_plus_CALC_plus_SMOKE', 'Gender_plus_CALC_minus_SCC', 'Gender_plus_SCC_minus_CALC', 'CALC_plus_SCC_minus_Gender', 'Gender_plus_CALC_plus_SCC', 'Gender_plus_CAEC_minus_SMOKE', 'Gender_plus_SMOKE_minus_CAEC', 'CAEC_plus_SMOKE_minus_Gender', 'Gender_plus_CAEC_plus_SMOKE', 'Gender_plus_CAEC_minus_SCC', 'Gender_plus_SCC_minus_CAEC', 'CAEC_plus_SCC_minus_Gender', 'Gender_plus_CAEC_plus_SCC', 'Gender_plus_MTRANS_minus_SMOKE', 'Gender_plus_SMOKE_minus_MTRANS', 'MTRANS_plus_SMOKE_minus_Gender', 'Gender_plus_MTRANS_plus_SMOKE', 'Gender_plus_MTRANS_minus_SCC', 'Gender_plus_SCC_minus_MTRANS', 'MTRANS_plus_SCC_minus_Gender', 'Gender_plus_MTRANS_plus_SCC', 'Gender_plus_family_history_with_overweight_minus_SMOKE', 'Gender_plus_SMOKE_minus_family_history_with_overweight', 'family_history_with_overweight_plus_SMOKE_minus_Gender', 'Gender_plus_family_history_with_overweight_plus_SMOKE', 'Gender_plus_family_history_with_overweight_minus_SCC', 'Gender_plus_SCC_minus_family_history_with_overweight', 'family_history_with_overweight_plus_SCC_minus_Gender', 'Gender_plus_family_history_with_overweight_plus_SCC', 'Gender_plus_FAVC_minus_SMOKE', 'Gender_plus_SMOKE_minus_FAVC', 'FAVC_plus_SMOKE_minus_Gender', 'Gender_plus_FAVC_plus_SMOKE', 'Gender_plus_FAVC_minus_SCC', 'FAVC_plus_SCC_minus_Gender', 'Gender_plus_FAVC_plus_SCC', 'Gender_plus_SMOKE_minus_SCC', 'Gender_plus_SCC_minus_SMOKE', 'SMOKE_plus_SCC_minus_Gender', 'Gender_plus_SMOKE_plus_SCC', 'CALC_plus_CAEC_minus_SMOKE', 'CALC_plus_SMOKE_minus_CAEC', 'CAEC_plus_SMOKE_minus_CALC', 'CALC_plus_CAEC_plus_SMOKE', 'CALC_plus_CAEC_minus_SCC', 'CALC_plus_SCC_minus_CAEC', 'CAEC_plus_SCC_minus_CALC', 'CALC_plus_CAEC_plus_SCC', 'MTRANS_plus_SMOKE_minus_CALC', 'CALC_plus_MTRANS_plus_SMOKE', 'CALC_plus_MTRANS_minus_SCC', 'CALC_plus_SCC_minus_MTRANS', 'MTRANS_plus_SCC_minus_CALC', 'CALC_plus_MTRANS_plus_SCC', 'CALC_plus_family_history_with_overweight_minus_SMOKE', 'CALC_plus_SMOKE_minus_family_history_with_overweight', 'family_history_with_overweight_plus_SMOKE_minus_CALC', 'CALC_plus_family_history_with_overweight_plus_SMOKE', 'CALC_plus_family_history_with_overweight_minus_SCC', 'CALC_plus_SCC_minus_family_history_with_overweight', 'family_history_with_overweight_plus_SCC_minus_CALC', 'CALC_plus_family_history_with_overweight_plus_SCC', 'FAVC_plus_SMOKE_minus_CALC', 'CALC_plus_FAVC_plus_SMOKE', 'FAVC_plus_SCC_minus_CALC', 'CALC_plus_FAVC_plus_SCC', 'SMOKE_plus_SCC_minus_CALC', 'CALC_plus_SMOKE_plus_SCC', 'CAEC_plus_MTRANS_minus_SMOKE', 'MTRANS_plus_SMOKE_minus_CAEC', 'CAEC_plus_MTRANS_plus_SMOKE', 'CAEC_plus_MTRANS_minus_SCC', 'CAEC_plus_SCC_minus_MTRANS', 'MTRANS_plus_SCC_minus_CAEC', 'CAEC_plus_MTRANS_plus_SCC', 'CAEC_plus_family_history_with_overweight_minus_SMOKE', 'CAEC_plus_SMOKE_minus_family_history_with_overweight', 'family_history_with_overweight_plus_SMOKE_minus_CAEC', 'CAEC_plus_family_history_with_overweight_plus_SMOKE', 'CAEC_plus_family_history_with_overweight_minus_SCC', 'CAEC_plus_SCC_minus_family_history_with_overweight', 'family_history_with_overweight_plus_SCC_minus_CAEC', 'CAEC_plus_family_history_with_overweight_plus_SCC', 'FAVC_plus_SMOKE_minus_CAEC', 'CAEC_plus_FAVC_plus_SMOKE', 'CAEC_plus_FAVC_minus_SCC', 'CAEC_plus_SCC_minus_FAVC', 'FAVC_plus_SCC_minus_CAEC', 'CAEC_plus_FAVC_plus_SCC', 'CAEC_plus_SMOKE_minus_SCC', 'CAEC_plus_SCC_minus_SMOKE', 'SMOKE_plus_SCC_minus_CAEC', 'CAEC_plus_SMOKE_plus_SCC', 'MTRANS_plus_family_history_with_overweight_minus_SMOKE', 'MTRANS_plus_SMOKE_minus_family_history_with_overweight', 'family_history_with_overweight_plus_SMOKE_minus_MTRANS', 'MTRANS_plus_family_history_with_overweight_plus_SMOKE', 'MTRANS_plus_family_history_with_overweight_minus_SCC', 'MTRANS_plus_SCC_minus_family_history_with_overweight', 'family_history_with_overweight_plus_SCC_minus_MTRANS', 'MTRANS_plus_family_history_with_overweight_plus_SCC', 'FAVC_plus_SMOKE_minus_MTRANS', 'MTRANS_plus_FAVC_plus_SMOKE', 'FAVC_plus_SCC_minus_MTRANS', 'MTRANS_plus_FAVC_plus_SCC', 'FAVC_plus_SMOKE_minus_family_history_with_overweight', 'family_history_with_overweight_plus_FAVC_plus_SMOKE', 'family_history_with_overweight_plus_FAVC_minus_SCC', 'family_history_with_overweight_plus_SCC_minus_FAVC', 'FAVC_plus_SCC_minus_family_history_with_overweight', 'family_history_with_overweight_plus_FAVC_plus_SCC', 'family_history_with_overweight_plus_SMOKE_minus_SCC', 'family_history_with_overweight_plus_SCC_minus_SMOKE', 'SMOKE_plus_SCC_minus_family_history_with_overweight', 'family_history_with_overweight_plus_SMOKE_plus_SCC'
,'NCP_round', 'Age_round', 'Age_ceil_round', 'Public_Transportation', 'Gender', 'MTRANS', 'family_history_with_overweight', 'FAVC', 'SMOKE', 'SCC', 'Weight_2dec_plus_Height_2dec_sub_by_Gender', 'Weight_2dec_plus_Gender_sub_by_Height_2dec', 'Height_2dec_plus_Gender_sub_by_Weight_2dec', 'Weight_2dec_plus_Height_2dec_sub_by_CALC', 'Weight_2dec_plus_CALC_sub_by_Height_2dec', 'Height_2dec_plus_CALC_sub_by_Weight_2dec', 'Weight_2dec_plus_Height_2dec_sub_by_CAEC', 'Weight_2dec_plus_CAEC_sub_by_Height_2dec', 'Height_2dec_plus_CAEC_sub_by_Weight_2dec', 'Weight_2dec_plus_Height_2dec_sub_by_MTRANS', 'Weight_2dec_plus_MTRANS_sub_by_Height_2dec', 'Height_2dec_plus_MTRANS_sub_by_Weight_2dec', 'Weight_2dec_plus_Height_2dec_sub_by_family_history_with_overweight', 'Weight_2dec_plus_family_history_with_overweight_sub_by_Height_2dec', 'Height_2dec_plus_family_history_with_overweight_sub_by_Weight_2dec', 'Weight_2dec_plus_Height_2dec_sub_by_FAVC', 'Weight_2dec_plus_FAVC_sub_by_Height_2dec', 'Height_2dec_plus_FAVC_sub_by_Weight_2dec', 'Weight_2dec_plus_Height_2dec_sub_by_SMOKE', 'Weight_2dec_plus_SMOKE_sub_by_Height_2dec', 'Height_2dec_plus_SMOKE_sub_by_Weight_2dec', 'Weight_2dec_plus_Height_2dec_sub_by_SCC', 'Weight_2dec_plus_SCC_sub_by_Height_2dec', 'Height_2dec_plus_SCC_sub_by_Weight_2dec', 'Weight_2dec_plus_Gender_sub_by_CALC', 'Weight_2dec_plus_CALC_sub_by_Gender', 'Gender_plus_CALC_sub_by_Weight_2dec', 'Weight_2dec_plus_Gender_sub_by_CAEC', 'Weight_2dec_plus_CAEC_sub_by_Gender', 'Gender_plus_CAEC_sub_by_Weight_2dec', 'Weight_2dec_plus_Gender_sub_by_MTRANS', 'Weight_2dec_plus_MTRANS_sub_by_Gender', 'Gender_plus_MTRANS_sub_by_Weight_2dec', 'Weight_2dec_plus_Gender_sub_by_family_history_with_overweight', 'Weight_2dec_plus_family_history_with_overweight_sub_by_Gender', 'Gender_plus_family_history_with_overweight_sub_by_Weight_2dec', 'Weight_2dec_plus_Gender_sub_by_FAVC', 'Weight_2dec_plus_FAVC_sub_by_Gender', 'Gender_plus_FAVC_sub_by_Weight_2dec', 'Weight_2dec_plus_Gender_sub_by_SMOKE', 'Weight_2dec_plus_SMOKE_sub_by_Gender', 'Gender_plus_SMOKE_sub_by_Weight_2dec', 'Weight_2dec_plus_Gender_sub_by_SCC', 'Weight_2dec_plus_SCC_sub_by_Gender', 'Gender_plus_SCC_sub_by_Weight_2dec', 'Weight_2dec_plus_CALC_sub_by_CAEC', 'Weight_2dec_plus_CAEC_sub_by_CALC', 'CALC_plus_CAEC_sub_by_Weight_2dec', 'Weight_2dec_plus_CALC_sub_by_MTRANS', 'Weight_2dec_plus_MTRANS_sub_by_CALC', 'CALC_plus_MTRANS_sub_by_Weight_2dec', 'Weight_2dec_plus_CALC_sub_by_family_history_with_overweight', 'Weight_2dec_plus_family_history_with_overweight_sub_by_CALC', 'CALC_plus_family_history_with_overweight_sub_by_Weight_2dec', 'Weight_2dec_plus_CALC_sub_by_FAVC', 'Weight_2dec_plus_FAVC_sub_by_CALC', 'CALC_plus_FAVC_sub_by_Weight_2dec', 'Weight_2dec_plus_CALC_sub_by_SMOKE', 'Weight_2dec_plus_SMOKE_sub_by_CALC', 'CALC_plus_SMOKE_sub_by_Weight_2dec', 'Weight_2dec_plus_CALC_sub_by_SCC', 'Weight_2dec_plus_SCC_sub_by_CALC', 'CALC_plus_SCC_sub_by_Weight_2dec', 'Weight_2dec_plus_CAEC_sub_by_MTRANS', 'Weight_2dec_plus_MTRANS_sub_by_CAEC', 'CAEC_plus_MTRANS_sub_by_Weight_2dec', 'Weight_2dec_plus_CAEC_sub_by_family_history_with_overweight', 'Weight_2dec_plus_family_history_with_overweight_sub_by_CAEC', 'CAEC_plus_family_history_with_overweight_sub_by_Weight_2dec', 'Weight_2dec_plus_CAEC_sub_by_FAVC', 'Weight_2dec_plus_FAVC_sub_by_CAEC', 'CAEC_plus_FAVC_sub_by_Weight_2dec', 'Weight_2dec_plus_CAEC_sub_by_SMOKE', 'Weight_2dec_plus_SMOKE_sub_by_CAEC', 'CAEC_plus_SMOKE_sub_by_Weight_2dec', 'Weight_2dec_plus_CAEC_sub_by_SCC', 'Weight_2dec_plus_SCC_sub_by_CAEC', 'CAEC_plus_SCC_sub_by_Weight_2dec', 'Weight_2dec_plus_MTRANS_sub_by_family_history_with_overweight', 'Weight_2dec_plus_family_history_with_overweight_sub_by_MTRANS', 'MTRANS_plus_family_history_with_overweight_sub_by_Weight_2dec', 'Weight_2dec_plus_MTRANS_sub_by_FAVC', 'Weight_2dec_plus_FAVC_sub_by_MTRANS', 'MTRANS_plus_FAVC_sub_by_Weight_2dec', 'Weight_2dec_plus_MTRANS_sub_by_SMOKE', 'Weight_2dec_plus_SMOKE_sub_by_MTRANS', 'MTRANS_plus_SMOKE_sub_by_Weight_2dec', 'Weight_2dec_plus_MTRANS_sub_by_SCC', 'Weight_2dec_plus_SCC_sub_by_MTRANS', 'MTRANS_plus_SCC_sub_by_Weight_2dec', 'Weight_2dec_plus_family_history_with_overweight_sub_by_FAVC', 'Weight_2dec_plus_FAVC_sub_by_family_history_with_overweight', 'family_history_with_overweight_plus_FAVC_sub_by_Weight_2dec', 'Weight_2dec_plus_family_history_with_overweight_sub_by_SMOKE', 'Weight_2dec_plus_SMOKE_sub_by_family_history_with_overweight', 'family_history_with_overweight_plus_SMOKE_sub_by_Weight_2dec', 'Weight_2dec_plus_family_history_with_overweight_sub_by_SCC', 'Weight_2dec_plus_SCC_sub_by_family_history_with_overweight', 'family_history_with_overweight_plus_SCC_sub_by_Weight_2dec', 'Weight_2dec_plus_FAVC_sub_by_SMOKE', 'Weight_2dec_plus_SMOKE_sub_by_FAVC', 'FAVC_plus_SMOKE_sub_by_Weight_2dec', 'Weight_2dec_plus_FAVC_sub_by_SCC', 'Weight_2dec_plus_SCC_sub_by_FAVC', 'FAVC_plus_SCC_sub_by_Weight_2dec', 'Weight_2dec_plus_SMOKE_sub_by_SCC', 'Weight_2dec_plus_SCC_sub_by_SMOKE', 'SMOKE_plus_SCC_sub_by_Weight_2dec', 'Height_2dec_plus_SMOKE_sub_by_Gender', 'Gender_plus_SMOKE_sub_by_Height_2dec', 'Height_2dec_plus_Gender_sub_by_SCC', 'Height_2dec_plus_SCC_sub_by_Gender', 'CALC_plus_MTRANS_sub_by_Height_2dec', 'Height_2dec_plus_CALC_sub_by_SMOKE', 'Height_2dec_plus_SMOKE_sub_by_CALC', 'CALC_plus_SMOKE_sub_by_Height_2dec', 'Height_2dec_plus_CALC_sub_by_SCC', 'Height_2dec_plus_SCC_sub_by_CALC', 'CALC_plus_SCC_sub_by_Height_2dec', 'Height_2dec_plus_CAEC_sub_by_SMOKE', 'Height_2dec_plus_SMOKE_sub_by_CAEC', 'CAEC_plus_SMOKE_sub_by_Height_2dec', 'Height_2dec_plus_CAEC_sub_by_SCC', 'Height_2dec_plus_SCC_sub_by_CAEC', 'CAEC_plus_SCC_sub_by_Height_2dec', 'Height_2dec_plus_family_history_with_overweight_sub_by_MTRANS', 'MTRANS_plus_family_history_with_overweight_sub_by_Height_2dec', 'Height_2dec_plus_SMOKE_sub_by_MTRANS', 'MTRANS_plus_SMOKE_sub_by_Height_2dec', 'Height_2dec_plus_SCC_sub_by_MTRANS', 'Height_2dec_plus_FAVC_sub_by_family_history_with_overweight', 'Height_2dec_plus_family_history_with_overweight_sub_by_SMOKE', 'Height_2dec_plus_SMOKE_sub_by_family_history_with_overweight', 'family_history_with_overweight_plus_SMOKE_sub_by_Height_2dec', 'Height_2dec_plus_family_history_with_overweight_sub_by_SCC', 'Height_2dec_plus_SCC_sub_by_family_history_with_overweight', 'family_history_with_overweight_plus_SCC_sub_by_Height_2dec', 'Height_2dec_plus_FAVC_sub_by_SMOKE', 'Height_2dec_plus_SMOKE_sub_by_FAVC', 'FAVC_plus_SMOKE_sub_by_Height_2dec', 'Height_2dec_plus_SCC_sub_by_FAVC', 'FAVC_plus_SCC_sub_by_Height_2dec', 'Height_2dec_plus_SMOKE_sub_by_SCC', 'Height_2dec_plus_SCC_sub_by_SMOKE', 'SMOKE_plus_SCC_sub_by_Height_2dec', 'Gender_plus_CALC_sub_by_CAEC', 'Gender_plus_CAEC_sub_by_CALC', 'Gender_plus_CALC_sub_by_family_history_with_overweight', 'Gender_plus_family_history_with_overweight_sub_by_CALC', 'Gender_plus_CALC_sub_by_SMOKE', 'Gender_plus_SMOKE_sub_by_CALC', 'CALC_plus_SMOKE_sub_by_Gender', 'Gender_plus_CALC_sub_by_SCC', 'Gender_plus_SCC_sub_by_CALC', 'CALC_plus_SCC_sub_by_Gender', 'Gender_plus_CAEC_sub_by_SMOKE', 'Gender_plus_SMOKE_sub_by_CAEC', 'CAEC_plus_SMOKE_sub_by_Gender', 'Gender_plus_CAEC_sub_by_SCC', 'Gender_plus_SCC_sub_by_CAEC', 'CAEC_plus_SCC_sub_by_Gender', 'Gender_plus_MTRANS_sub_by_SMOKE', 'Gender_plus_SMOKE_sub_by_MTRANS', 'MTRANS_plus_SMOKE_sub_by_Gender', 'Gender_plus_MTRANS_sub_by_SCC', 'Gender_plus_SCC_sub_by_MTRANS', 'MTRANS_plus_SCC_sub_by_Gender', 'Gender_plus_family_history_with_overweight_sub_by_SMOKE', 'Gender_plus_SMOKE_sub_by_family_history_with_overweight', 'family_history_with_overweight_plus_SMOKE_sub_by_Gender', 'Gender_plus_family_history_with_overweight_sub_by_SCC', 'Gender_plus_SCC_sub_by_family_history_with_overweight', 'family_history_with_overweight_plus_SCC_sub_by_Gender', 'Gender_plus_SMOKE_sub_by_FAVC', 'FAVC_plus_SMOKE_sub_by_Gender', 'Gender_plus_FAVC_sub_by_SCC', 'FAVC_plus_SCC_sub_by_Gender', 'Gender_plus_SMOKE_sub_by_SCC', 'SMOKE_plus_SCC_sub_by_Gender', 'CALC_plus_SMOKE_sub_by_CAEC', 'CAEC_plus_SMOKE_sub_by_CALC', 'CALC_plus_CAEC_sub_by_SCC', 'CALC_plus_SCC_sub_by_CAEC', 'CAEC_plus_SCC_sub_by_CALC', 'MTRANS_plus_SMOKE_sub_by_CALC', 'CALC_plus_MTRANS_sub_by_SCC', 'CALC_plus_SCC_sub_by_MTRANS', 'MTRANS_plus_SCC_sub_by_CALC', 'CALC_plus_family_history_with_overweight_sub_by_SMOKE', 'CALC_plus_SMOKE_sub_by_family_history_with_overweight', 'family_history_with_overweight_plus_SMOKE_sub_by_CALC', 'CALC_plus_family_history_with_overweight_sub_by_SCC', 'CALC_plus_SCC_sub_by_family_history_with_overweight', 'family_history_with_overweight_plus_SCC_sub_by_CALC', 'FAVC_plus_SMOKE_sub_by_CALC', 'FAVC_plus_SCC_sub_by_CALC', 'SMOKE_plus_SCC_sub_by_CALC', 'MTRANS_plus_SMOKE_sub_by_CAEC', 'CAEC_plus_MTRANS_sub_by_SCC', 'CAEC_plus_SCC_sub_by_MTRANS', 'MTRANS_plus_SCC_sub_by_CAEC', 'CAEC_plus_family_history_with_overweight_sub_by_SMOKE', 'CAEC_plus_SMOKE_sub_by_family_history_with_overweight', 'family_history_with_overweight_plus_SMOKE_sub_by_CAEC', 'CAEC_plus_family_history_with_overweight_sub_by_SCC', 'CAEC_plus_SCC_sub_by_family_history_with_overweight', 'family_history_with_overweight_plus_SCC_sub_by_CAEC', 'FAVC_plus_SMOKE_sub_by_CAEC', 'CAEC_plus_FAVC_sub_by_SCC', 'CAEC_plus_SCC_sub_by_FAVC', 'FAVC_plus_SCC_sub_by_CAEC', 'CAEC_plus_SMOKE_sub_by_SCC', 'CAEC_plus_SCC_sub_by_SMOKE', 'SMOKE_plus_SCC_sub_by_CAEC', 'MTRANS_plus_family_history_with_overweight_sub_by_SMOKE', 'MTRANS_plus_SMOKE_sub_by_family_history_with_overweight', 'family_history_with_overweight_plus_SMOKE_sub_by_MTRANS', 'MTRANS_plus_family_history_with_overweight_sub_by_SCC', 'MTRANS_plus_SCC_sub_by_family_history_with_overweight', 'family_history_with_overweight_plus_SCC_sub_by_MTRANS', 'FAVC_plus_SMOKE_sub_by_MTRANS', 'FAVC_plus_SCC_sub_by_MTRANS', 'FAVC_plus_SMOKE_sub_by_family_history_with_overweight', 'family_history_with_overweight_plus_FAVC_sub_by_SCC', 'family_history_with_overweight_plus_SCC_sub_by_FAVC', 'FAVC_plus_SCC_sub_by_family_history_with_overweight', 'family_history_with_overweight_plus_SMOKE_sub_by_SCC', 'family_history_with_overweight_plus_SCC_sub_by_SMOKE', 'SMOKE_plus_SCC_sub_by_family_history_with_overweight'
,'NCP_round', 'Age_round', 'Age_ceil_round', 'Public_Transportation', 'Gender', 'MTRANS', 'family_history_with_overweight', 'FAVC', 'SMOKE', 'SCC', 'Weight_2dec_x_Height_2dec_x_Gender', 'Weight_2dec_x_Height_2dec_x_CALC', 'Weight_2dec_x_Height_2dec_x_CAEC', 'Weight_2dec_x_Height_2dec_x_MTRANS', 'Weight_2dec_x_Height_2dec_x_family_history_with_overweight', 'Weight_2dec_x_Height_2dec_x_FAVC', 'Weight_2dec_x_Height_2dec_x_SMOKE', 'Weight_2dec_x_Height_2dec_x_SCC', 'Weight_2dec_x_Gender_x_CALC', 'Weight_2dec_x_Gender_x_CAEC', 'Weight_2dec_x_Gender_x_MTRANS', 'Weight_2dec_x_Gender_x_family_history_with_overweight', 'Weight_2dec_x_Gender_x_FAVC', 'Weight_2dec_x_Gender_x_SMOKE', 'Weight_2dec_x_Gender_x_SCC', 'Weight_2dec_x_CALC_x_CAEC', 'Weight_2dec_x_CALC_x_MTRANS', 'Weight_2dec_x_CALC_x_family_history_with_overweight', 'Weight_2dec_x_CALC_x_FAVC', 'Weight_2dec_x_CALC_x_SMOKE', 'Weight_2dec_x_CALC_x_SCC', 'Weight_2dec_x_CAEC_x_MTRANS', 'Weight_2dec_x_CAEC_x_family_history_with_overweight', 'Weight_2dec_x_CAEC_x_FAVC', 'Weight_2dec_x_CAEC_x_SMOKE', 'Weight_2dec_x_CAEC_x_SCC', 'Weight_2dec_x_MTRANS_x_family_history_with_overweight', 'Weight_2dec_x_MTRANS_x_FAVC', 'Weight_2dec_x_MTRANS_x_SMOKE', 'Weight_2dec_x_MTRANS_x_SCC', 'Weight_2dec_x_family_history_with_overweight_x_FAVC', 'Weight_2dec_x_family_history_with_overweight_x_SMOKE', 'Weight_2dec_x_family_history_with_overweight_x_SCC', 'Weight_2dec_x_FAVC_x_SMOKE', 'Weight_2dec_x_FAVC_x_SCC', 'Weight_2dec_x_SMOKE_x_SCC', 'Height_2dec_x_Gender_x_SCC', 'Height_2dec_x_CALC_x_SMOKE', 'Height_2dec_x_CALC_x_SCC', 'Height_2dec_x_CAEC_x_FAVC', 'Height_2dec_x_CAEC_x_SMOKE', 'Height_2dec_x_CAEC_x_SCC', 'Height_2dec_x_MTRANS_x_SMOKE', 'Height_2dec_x_MTRANS_x_SCC', 'Height_2dec_x_family_history_with_overweight_x_FAVC', 'Height_2dec_x_family_history_with_overweight_x_SMOKE', 'Height_2dec_x_family_history_with_overweight_x_SCC', 'Height_2dec_x_FAVC_x_SMOKE', 'Height_2dec_x_FAVC_x_SCC', 'Height_2dec_x_SMOKE_x_SCC', 'Gender_x_CALC_x_SMOKE', 'Gender_x_CALC_x_SCC', 'Gender_x_CAEC_x_SMOKE', 'Gender_x_CAEC_x_SCC', 'Gender_x_MTRANS_x_SCC', 'Gender_x_family_history_with_overweight_x_SMOKE', 'Gender_x_family_history_with_overweight_x_SCC', 'Gender_x_FAVC_x_SMOKE', 'Gender_x_FAVC_x_SCC', 'Gender_x_SMOKE_x_SCC', 'CALC_x_CAEC_x_SMOKE', 'CALC_x_CAEC_x_SCC', 'CALC_x_MTRANS_x_SCC', 'CALC_x_family_history_with_overweight_x_SMOKE', 'CALC_x_family_history_with_overweight_x_SCC', 'CALC_x_FAVC_x_SCC', 'CAEC_x_MTRANS_x_SMOKE', 'CAEC_x_MTRANS_x_SCC', 'CAEC_x_family_history_with_overweight_x_SMOKE', 'CAEC_x_family_history_with_overweight_x_SCC', 'CAEC_x_FAVC_x_SCC', 'CAEC_x_SMOKE_x_SCC', 'MTRANS_x_family_history_with_overweight_x_SMOKE', 'MTRANS_x_family_history_with_overweight_x_SCC', 'family_history_with_overweight_x_FAVC_x_SCC', 'family_history_with_overweight_x_SMOKE_x_SCC'
,'NCP_round', 'Age_round', 'Age_ceil_round', 'Public_Transportation', 'Gender', 'MTRANS', 'family_history_with_overweight', 'FAVC', 'SMOKE', 'SCC', 'Weight_2dec_div_sum_of_Height_2dec_and_Gender', 'Height_2dec_div_sum_of_Weight_2dec_and_Gender', 'Gender_div_sum_of_Weight_2dec_and_Height_2dec', 'Weight_2dec_div_sum_of_Height_2dec_and_CALC', 'Height_2dec_div_sum_of_Weight_2dec_and_CALC', 'CALC_div_sum_of_Weight_2dec_and_Height_2dec', 'Height_2dec_div_sum_of_Weight_2dec_and_CAEC', 'CAEC_div_sum_of_Weight_2dec_and_Height_2dec', 'Weight_2dec_div_sum_of_Height_2dec_and_MTRANS', 'Height_2dec_div_sum_of_Weight_2dec_and_MTRANS', 'MTRANS_div_sum_of_Weight_2dec_and_Height_2dec', 'Height_2dec_div_sum_of_Weight_2dec_and_family_history_with_overweight', 'family_history_with_overweight_div_sum_of_Weight_2dec_and_Height_2dec', 'Weight_2dec_div_sum_of_Height_2dec_and_FAVC', 'Height_2dec_div_sum_of_Weight_2dec_and_FAVC', 'FAVC_div_sum_of_Weight_2dec_and_Height_2dec', 'Weight_2dec_div_sum_of_Height_2dec_and_SMOKE', 'Height_2dec_div_sum_of_Weight_2dec_and_SMOKE', 'SMOKE_div_sum_of_Weight_2dec_and_Height_2dec', 'Weight_2dec_div_sum_of_Height_2dec_and_SCC', 'Height_2dec_div_sum_of_Weight_2dec_and_SCC', 'SCC_div_sum_of_Weight_2dec_and_Height_2dec', 'Weight_2dec_div_sum_of_Gender_and_CALC', 'Gender_div_sum_of_Weight_2dec_and_CALC', 'CALC_div_sum_of_Weight_2dec_and_Gender', 'Weight_2dec_div_sum_of_Gender_and_CAEC', 'Gender_div_sum_of_Weight_2dec_and_CAEC', 'CAEC_div_sum_of_Weight_2dec_and_Gender', 'Weight_2dec_div_sum_of_Gender_and_MTRANS', 'Gender_div_sum_of_Weight_2dec_and_MTRANS', 'MTRANS_div_sum_of_Weight_2dec_and_Gender', 'Weight_2dec_div_sum_of_Gender_and_family_history_with_overweight', 'Gender_div_sum_of_Weight_2dec_and_family_history_with_overweight', 'family_history_with_overweight_div_sum_of_Weight_2dec_and_Gender', 'Weight_2dec_div_sum_of_Gender_and_FAVC', 'Gender_div_sum_of_Weight_2dec_and_FAVC', 'FAVC_div_sum_of_Weight_2dec_and_Gender', 'Weight_2dec_div_sum_of_Gender_and_SMOKE', 'Gender_div_sum_of_Weight_2dec_and_SMOKE', 'SMOKE_div_sum_of_Weight_2dec_and_Gender', 'Weight_2dec_div_sum_of_Gender_and_SCC', 'Gender_div_sum_of_Weight_2dec_and_SCC', 'SCC_div_sum_of_Weight_2dec_and_Gender', 'Weight_2dec_div_sum_of_CALC_and_CAEC', 'CALC_div_sum_of_Weight_2dec_and_CAEC', 'CAEC_div_sum_of_Weight_2dec_and_CALC', 'Weight_2dec_div_sum_of_CALC_and_MTRANS', 'CALC_div_sum_of_Weight_2dec_and_MTRANS', 'MTRANS_div_sum_of_Weight_2dec_and_CALC', 'Weight_2dec_div_sum_of_CALC_and_family_history_with_overweight', 'CALC_div_sum_of_Weight_2dec_and_family_history_with_overweight', 'family_history_with_overweight_div_sum_of_Weight_2dec_and_CALC', 'Weight_2dec_div_sum_of_CALC_and_FAVC', 'CALC_div_sum_of_Weight_2dec_and_FAVC', 'FAVC_div_sum_of_Weight_2dec_and_CALC', 'Weight_2dec_div_sum_of_CALC_and_SMOKE', 'CALC_div_sum_of_Weight_2dec_and_SMOKE', 'SMOKE_div_sum_of_Weight_2dec_and_CALC', 'Weight_2dec_div_sum_of_CALC_and_SCC', 'CALC_div_sum_of_Weight_2dec_and_SCC', 'SCC_div_sum_of_Weight_2dec_and_CALC', 'Weight_2dec_div_sum_of_CAEC_and_MTRANS', 'CAEC_div_sum_of_Weight_2dec_and_MTRANS', 'MTRANS_div_sum_of_Weight_2dec_and_CAEC', 'CAEC_div_sum_of_Weight_2dec_and_family_history_with_overweight', 'family_history_with_overweight_div_sum_of_Weight_2dec_and_CAEC', 'Weight_2dec_div_sum_of_CAEC_and_FAVC', 'CAEC_div_sum_of_Weight_2dec_and_FAVC', 'FAVC_div_sum_of_Weight_2dec_and_CAEC', 'Weight_2dec_div_sum_of_CAEC_and_SMOKE', 'CAEC_div_sum_of_Weight_2dec_and_SMOKE', 'SMOKE_div_sum_of_Weight_2dec_and_CAEC', 'Weight_2dec_div_sum_of_CAEC_and_SCC', 'CAEC_div_sum_of_Weight_2dec_and_SCC', 'SCC_div_sum_of_Weight_2dec_and_CAEC', 'Weight_2dec_div_sum_of_MTRANS_and_family_history_with_overweight', 'MTRANS_div_sum_of_Weight_2dec_and_family_history_with_overweight', 'family_history_with_overweight_div_sum_of_Weight_2dec_and_MTRANS', 'Weight_2dec_div_sum_of_MTRANS_and_FAVC', 'MTRANS_div_sum_of_Weight_2dec_and_FAVC', 'FAVC_div_sum_of_Weight_2dec_and_MTRANS', 'Weight_2dec_div_sum_of_MTRANS_and_SMOKE', 'MTRANS_div_sum_of_Weight_2dec_and_SMOKE', 'SMOKE_div_sum_of_Weight_2dec_and_MTRANS', 'Weight_2dec_div_sum_of_MTRANS_and_SCC', 'MTRANS_div_sum_of_Weight_2dec_and_SCC', 'SCC_div_sum_of_Weight_2dec_and_MTRANS', 'Weight_2dec_div_sum_of_family_history_with_overweight_and_FAVC', 'family_history_with_overweight_div_sum_of_Weight_2dec_and_FAVC', 'FAVC_div_sum_of_Weight_2dec_and_family_history_with_overweight', 'Weight_2dec_div_sum_of_family_history_with_overweight_and_SMOKE', 'family_history_with_overweight_div_sum_of_Weight_2dec_and_SMOKE', 'SMOKE_div_sum_of_Weight_2dec_and_family_history_with_overweight', 'Weight_2dec_div_sum_of_family_history_with_overweight_and_SCC', 'family_history_with_overweight_div_sum_of_Weight_2dec_and_SCC', 'SCC_div_sum_of_Weight_2dec_and_family_history_with_overweight', 'Weight_2dec_div_sum_of_FAVC_and_SMOKE', 'FAVC_div_sum_of_Weight_2dec_and_SMOKE', 'SMOKE_div_sum_of_Weight_2dec_and_FAVC', 'Weight_2dec_div_sum_of_FAVC_and_SCC', 'FAVC_div_sum_of_Weight_2dec_and_SCC', 'SCC_div_sum_of_Weight_2dec_and_FAVC', 'Weight_2dec_div_sum_of_SMOKE_and_SCC', 'SMOKE_div_sum_of_Weight_2dec_and_SCC', 'SCC_div_sum_of_Weight_2dec_and_SMOKE', 'Height_2dec_div_sum_of_Gender_and_MTRANS', 'Height_2dec_div_sum_of_Gender_and_SMOKE', 'Gender_div_sum_of_Height_2dec_and_SMOKE', 'Gender_div_sum_of_Height_2dec_and_SCC', 'SCC_div_sum_of_Height_2dec_and_Gender', 'Height_2dec_div_sum_of_CALC_and_MTRANS', 'Height_2dec_div_sum_of_CALC_and_SMOKE', 'CALC_div_sum_of_Height_2dec_and_SMOKE', 'SMOKE_div_sum_of_Height_2dec_and_CALC', 'Height_2dec_div_sum_of_CALC_and_SCC', 'CALC_div_sum_of_Height_2dec_and_SCC', 'SCC_div_sum_of_Height_2dec_and_CALC', 'Height_2dec_div_sum_of_CAEC_and_MTRANS', 'CAEC_div_sum_of_Height_2dec_and_FAVC', 'Height_2dec_div_sum_of_CAEC_and_SMOKE', 'CAEC_div_sum_of_Height_2dec_and_SMOKE', 'SMOKE_div_sum_of_Height_2dec_and_CAEC', 'Height_2dec_div_sum_of_CAEC_and_SCC', 'CAEC_div_sum_of_Height_2dec_and_SCC', 'Height_2dec_div_sum_of_MTRANS_and_family_history_with_overweight', 'Height_2dec_div_sum_of_MTRANS_and_FAVC', 'Height_2dec_div_sum_of_MTRANS_and_SMOKE', 'Height_2dec_div_sum_of_MTRANS_and_SCC', 'MTRANS_div_sum_of_Height_2dec_and_SCC', 'Height_2dec_div_sum_of_family_history_with_overweight_and_FAVC', 'family_history_with_overweight_div_sum_of_Height_2dec_and_FAVC', 'Height_2dec_div_sum_of_family_history_with_overweight_and_SMOKE', 'family_history_with_overweight_div_sum_of_Height_2dec_and_SMOKE', 'SMOKE_div_sum_of_Height_2dec_and_family_history_with_overweight', 'Height_2dec_div_sum_of_family_history_with_overweight_and_SCC', 'family_history_with_overweight_div_sum_of_Height_2dec_and_SCC', 'Height_2dec_div_sum_of_FAVC_and_SMOKE', 'FAVC_div_sum_of_Height_2dec_and_SMOKE', 'SMOKE_div_sum_of_Height_2dec_and_FAVC', 'Height_2dec_div_sum_of_FAVC_and_SCC', 'FAVC_div_sum_of_Height_2dec_and_SCC', 'Height_2dec_div_sum_of_SMOKE_and_SCC', 'SMOKE_div_sum_of_Height_2dec_and_SCC', 'SCC_div_sum_of_Height_2dec_and_SMOKE', 'CAEC_div_sum_of_Gender_and_CALC', 'CALC_div_sum_of_Gender_and_MTRANS', 'CALC_div_sum_of_Gender_and_family_history_with_overweight', 'family_history_with_overweight_div_sum_of_Gender_and_CALC', 'CALC_div_sum_of_Gender_and_FAVC', 'Gender_div_sum_of_CALC_and_SMOKE', 'CALC_div_sum_of_Gender_and_SMOKE', 'SMOKE_div_sum_of_Gender_and_CALC', 'Gender_div_sum_of_CALC_and_SCC', 'CALC_div_sum_of_Gender_and_SCC', 'SCC_div_sum_of_Gender_and_CALC', 'Gender_div_sum_of_CAEC_and_SMOKE', 'CAEC_div_sum_of_Gender_and_SMOKE', 'Gender_div_sum_of_CAEC_and_SCC', 'CAEC_div_sum_of_Gender_and_SCC', 'Gender_div_sum_of_MTRANS_and_SMOKE', 'MTRANS_div_sum_of_Gender_and_SMOKE', 'Gender_div_sum_of_MTRANS_and_SCC', 'MTRANS_div_sum_of_Gender_and_SCC', 'SCC_div_sum_of_Gender_and_MTRANS', 'Gender_div_sum_of_family_history_with_overweight_and_SMOKE', 'family_history_with_overweight_div_sum_of_Gender_and_SMOKE', 'Gender_div_sum_of_family_history_with_overweight_and_SCC', 'family_history_with_overweight_div_sum_of_Gender_and_SCC', 'Gender_div_sum_of_FAVC_and_SMOKE', 'FAVC_div_sum_of_Gender_and_SMOKE', 'Gender_div_sum_of_FAVC_and_SCC', 'FAVC_div_sum_of_Gender_and_SCC', 'SCC_div_sum_of_Gender_and_FAVC', 'Gender_div_sum_of_SMOKE_and_SCC', 'SCC_div_sum_of_Gender_and_SMOKE', 'CALC_div_sum_of_CAEC_and_SMOKE', 'CAEC_div_sum_of_CALC_and_SMOKE', 'CALC_div_sum_of_CAEC_and_SCC', 'CAEC_div_sum_of_CALC_and_SCC', 'CALC_div_sum_of_MTRANS_and_SMOKE', 'MTRANS_div_sum_of_CALC_and_SMOKE', 'CALC_div_sum_of_MTRANS_and_SCC', 'MTRANS_div_sum_of_CALC_and_SCC', 'SCC_div_sum_of_CALC_and_MTRANS', 'CALC_div_sum_of_family_history_with_overweight_and_FAVC', 'CALC_div_sum_of_family_history_with_overweight_and_SMOKE', 'family_history_with_overweight_div_sum_of_CALC_and_SMOKE', 'CALC_div_sum_of_family_history_with_overweight_and_SCC', 'family_history_with_overweight_div_sum_of_CALC_and_SCC', 'CALC_div_sum_of_FAVC_and_SMOKE', 'CALC_div_sum_of_FAVC_and_SCC', 'FAVC_div_sum_of_CALC_and_SCC', 'CALC_div_sum_of_SMOKE_and_SCC', 'SCC_div_sum_of_CALC_and_SMOKE', 'CAEC_div_sum_of_MTRANS_and_SMOKE', 'MTRANS_div_sum_of_CAEC_and_SMOKE', 'CAEC_div_sum_of_MTRANS_and_SCC', 'MTRANS_div_sum_of_CAEC_and_SCC', 'CAEC_div_sum_of_family_history_with_overweight_and_SMOKE', 'family_history_with_overweight_div_sum_of_CAEC_and_SMOKE', 'CAEC_div_sum_of_family_history_with_overweight_and_SCC', 'family_history_with_overweight_div_sum_of_CAEC_and_SCC', 'CAEC_div_sum_of_FAVC_and_SCC', 'FAVC_div_sum_of_CAEC_and_SCC', 'CAEC_div_sum_of_SMOKE_and_SCC', 'SMOKE_div_sum_of_CAEC_and_SCC', 'MTRANS_div_sum_of_family_history_with_overweight_and_SMOKE', 'family_history_with_overweight_div_sum_of_MTRANS_and_SMOKE', 'MTRANS_div_sum_of_family_history_with_overweight_and_SCC', 'family_history_with_overweight_div_sum_of_MTRANS_and_SCC', 'MTRANS_div_sum_of_FAVC_and_SCC', 'FAVC_div_sum_of_MTRANS_and_SCC', 'family_history_with_overweight_div_sum_of_FAVC_and_SCC', 'FAVC_div_sum_of_family_history_with_overweight_and_SCC', 'family_history_with_overweight_div_sum_of_SMOKE_and_SCC', 'SMOKE_div_sum_of_family_history_with_overweight_and_SCC'
,'NCP_round', 'Age_round', 'Age_ceil_round', 'Public_Transportation', 'Gender', 'MTRANS', 'family_history_with_overweight', 'FAVC', 'SMOKE', 'SCC', 'Weight_2dec_div_prod_of_Height_2dec_and_Gender', 'Height_2dec_div_prod_of_Weight_2dec_and_Gender', 'Gender_div_prod_of_Weight_2dec_and_Height_2dec', 'Height_2dec_div_prod_of_Weight_2dec_and_CALC', 'CALC_div_prod_of_Weight_2dec_and_Height_2dec', 'Height_2dec_div_prod_of_Weight_2dec_and_CAEC', 'CAEC_div_prod_of_Weight_2dec_and_Height_2dec', 'Weight_2dec_div_prod_of_Height_2dec_and_MTRANS', 'Height_2dec_div_prod_of_Weight_2dec_and_MTRANS', 'MTRANS_div_prod_of_Weight_2dec_and_Height_2dec', 'Height_2dec_div_prod_of_Weight_2dec_and_family_history_with_overweight', 'family_history_with_overweight_div_prod_of_Weight_2dec_and_Height_2dec', 'Height_2dec_div_prod_of_Weight_2dec_and_FAVC', 'FAVC_div_prod_of_Weight_2dec_and_Height_2dec', 'Weight_2dec_div_prod_of_Height_2dec_and_SMOKE', 'Height_2dec_div_prod_of_Weight_2dec_and_SMOKE', 'SMOKE_div_prod_of_Weight_2dec_and_Height_2dec', 'Height_2dec_div_prod_of_Weight_2dec_and_SCC', 'SCC_div_prod_of_Weight_2dec_and_Height_2dec', 'Weight_2dec_div_prod_of_Gender_and_CALC', 'Gender_div_prod_of_Weight_2dec_and_CALC', 'CALC_div_prod_of_Weight_2dec_and_Gender', 'Weight_2dec_div_prod_of_Gender_and_CAEC', 'Gender_div_prod_of_Weight_2dec_and_CAEC', 'CAEC_div_prod_of_Weight_2dec_and_Gender', 'Weight_2dec_div_prod_of_Gender_and_MTRANS', 'Gender_div_prod_of_Weight_2dec_and_MTRANS', 'MTRANS_div_prod_of_Weight_2dec_and_Gender', 'Weight_2dec_div_prod_of_Gender_and_family_history_with_overweight', 'Gender_div_prod_of_Weight_2dec_and_family_history_with_overweight', 'family_history_with_overweight_div_prod_of_Weight_2dec_and_Gender', 'Weight_2dec_div_prod_of_Gender_and_FAVC', 'Gender_div_prod_of_Weight_2dec_and_FAVC', 'FAVC_div_prod_of_Weight_2dec_and_Gender', 'Weight_2dec_div_prod_of_Gender_and_SMOKE', 'Gender_div_prod_of_Weight_2dec_and_SMOKE', 'SMOKE_div_prod_of_Weight_2dec_and_Gender', 'Weight_2dec_div_prod_of_Gender_and_SCC', 'Gender_div_prod_of_Weight_2dec_and_SCC', 'SCC_div_prod_of_Weight_2dec_and_Gender', 'CALC_div_prod_of_Weight_2dec_and_CAEC', 'CAEC_div_prod_of_Weight_2dec_and_CALC', 'Weight_2dec_div_prod_of_CALC_and_MTRANS', 'CALC_div_prod_of_Weight_2dec_and_MTRANS', 'MTRANS_div_prod_of_Weight_2dec_and_CALC', 'CALC_div_prod_of_Weight_2dec_and_family_history_with_overweight', 'family_history_with_overweight_div_prod_of_Weight_2dec_and_CALC', 'CALC_div_prod_of_Weight_2dec_and_FAVC', 'FAVC_div_prod_of_Weight_2dec_and_CALC', 'Weight_2dec_div_prod_of_CALC_and_SMOKE', 'CALC_div_prod_of_Weight_2dec_and_SMOKE', 'SMOKE_div_prod_of_Weight_2dec_and_CALC', 'CALC_div_prod_of_Weight_2dec_and_SCC', 'SCC_div_prod_of_Weight_2dec_and_CALC', 'Weight_2dec_div_prod_of_CAEC_and_MTRANS', 'CAEC_div_prod_of_Weight_2dec_and_MTRANS', 'MTRANS_div_prod_of_Weight_2dec_and_CAEC', 'CAEC_div_prod_of_Weight_2dec_and_FAVC', 'FAVC_div_prod_of_Weight_2dec_and_CAEC', 'Weight_2dec_div_prod_of_CAEC_and_SMOKE', 'CAEC_div_prod_of_Weight_2dec_and_SMOKE', 'SMOKE_div_prod_of_Weight_2dec_and_CAEC', 'Weight_2dec_div_prod_of_CAEC_and_SCC', 'CAEC_div_prod_of_Weight_2dec_and_SCC', 'SCC_div_prod_of_Weight_2dec_and_CAEC', 'Weight_2dec_div_prod_of_MTRANS_and_family_history_with_overweight', 'MTRANS_div_prod_of_Weight_2dec_and_family_history_with_overweight', 'family_history_with_overweight_div_prod_of_Weight_2dec_and_MTRANS', 'Weight_2dec_div_prod_of_MTRANS_and_FAVC', 'MTRANS_div_prod_of_Weight_2dec_and_FAVC', 'FAVC_div_prod_of_Weight_2dec_and_MTRANS', 'Weight_2dec_div_prod_of_MTRANS_and_SMOKE', 'MTRANS_div_prod_of_Weight_2dec_and_SMOKE', 'SMOKE_div_prod_of_Weight_2dec_and_MTRANS', 'Weight_2dec_div_prod_of_MTRANS_and_SCC', 'MTRANS_div_prod_of_Weight_2dec_and_SCC', 'SCC_div_prod_of_Weight_2dec_and_MTRANS', 'Weight_2dec_div_prod_of_family_history_with_overweight_and_FAVC', 'family_history_with_overweight_div_prod_of_Weight_2dec_and_FAVC', 'FAVC_div_prod_of_Weight_2dec_and_family_history_with_overweight', 'Weight_2dec_div_prod_of_family_history_with_overweight_and_SMOKE', 'family_history_with_overweight_div_prod_of_Weight_2dec_and_SMOKE', 'SMOKE_div_prod_of_Weight_2dec_and_family_history_with_overweight', 'Weight_2dec_div_prod_of_family_history_with_overweight_and_SCC', 'family_history_with_overweight_div_prod_of_Weight_2dec_and_SCC', 'SCC_div_prod_of_Weight_2dec_and_family_history_with_overweight', 'Weight_2dec_div_prod_of_FAVC_and_SMOKE', 'FAVC_div_prod_of_Weight_2dec_and_SMOKE', 'SMOKE_div_prod_of_Weight_2dec_and_FAVC', 'Weight_2dec_div_prod_of_FAVC_and_SCC', 'FAVC_div_prod_of_Weight_2dec_and_SCC', 'SCC_div_prod_of_Weight_2dec_and_FAVC', 'Weight_2dec_div_prod_of_SMOKE_and_SCC', 'SMOKE_div_prod_of_Weight_2dec_and_SCC', 'SCC_div_prod_of_Weight_2dec_and_SMOKE', 'Height_2dec_div_prod_of_Gender_and_SMOKE', 'Gender_div_prod_of_Height_2dec_and_SMOKE', 'Gender_div_prod_of_Height_2dec_and_SCC', 'SCC_div_prod_of_Height_2dec_and_Gender', 'Height_2dec_div_prod_of_CALC_and_MTRANS', 'MTRANS_div_prod_of_Height_2dec_and_CALC', 'Height_2dec_div_prod_of_CALC_and_SMOKE', 'CALC_div_prod_of_Height_2dec_and_SMOKE', 'SMOKE_div_prod_of_Height_2dec_and_CALC', 'Height_2dec_div_prod_of_CALC_and_SCC', 'CALC_div_prod_of_Height_2dec_and_SCC', 'SCC_div_prod_of_Height_2dec_and_CALC', 'Height_2dec_div_prod_of_CAEC_and_MTRANS', 'MTRANS_div_prod_of_Height_2dec_and_CAEC', 'CAEC_div_prod_of_Height_2dec_and_FAVC', 'Height_2dec_div_prod_of_CAEC_and_SMOKE', 'CAEC_div_prod_of_Height_2dec_and_SMOKE', 'SMOKE_div_prod_of_Height_2dec_and_CAEC', 'Height_2dec_div_prod_of_CAEC_and_SCC', 'CAEC_div_prod_of_Height_2dec_and_SCC', 'SCC_div_prod_of_Height_2dec_and_CAEC', 'Height_2dec_div_prod_of_MTRANS_and_family_history_with_overweight', 'MTRANS_div_prod_of_Height_2dec_and_family_history_with_overweight', 'family_history_with_overweight_div_prod_of_Height_2dec_and_MTRANS', 'Height_2dec_div_prod_of_MTRANS_and_FAVC', 'Height_2dec_div_prod_of_MTRANS_and_SMOKE', 'MTRANS_div_prod_of_Height_2dec_and_SMOKE', 'SMOKE_div_prod_of_Height_2dec_and_MTRANS', 'MTRANS_div_prod_of_Height_2dec_and_SCC', 'family_history_with_overweight_div_prod_of_Height_2dec_and_FAVC', 'Height_2dec_div_prod_of_family_history_with_overweight_and_SMOKE', 'family_history_with_overweight_div_prod_of_Height_2dec_and_SMOKE', 'SMOKE_div_prod_of_Height_2dec_and_family_history_with_overweight', 'Height_2dec_div_prod_of_family_history_with_overweight_and_SCC', 'family_history_with_overweight_div_prod_of_Height_2dec_and_SCC', 'SCC_div_prod_of_Height_2dec_and_family_history_with_overweight', 'Height_2dec_div_prod_of_FAVC_and_SMOKE', 'FAVC_div_prod_of_Height_2dec_and_SMOKE', 'SMOKE_div_prod_of_Height_2dec_and_FAVC', 'Height_2dec_div_prod_of_FAVC_and_SCC', 'FAVC_div_prod_of_Height_2dec_and_SCC', 'Height_2dec_div_prod_of_SMOKE_and_SCC', 'SMOKE_div_prod_of_Height_2dec_and_SCC', 'SCC_div_prod_of_Height_2dec_and_SMOKE', 'CALC_div_prod_of_Gender_and_CAEC', 'CAEC_div_prod_of_Gender_and_CALC', 'CALC_div_prod_of_Gender_and_family_history_with_overweight', 'family_history_with_overweight_div_prod_of_Gender_and_CALC', 'Gender_div_prod_of_CALC_and_SMOKE', 'CALC_div_prod_of_Gender_and_SMOKE', 'SMOKE_div_prod_of_Gender_and_CALC', 'Gender_div_prod_of_CALC_and_SCC', 'CALC_div_prod_of_Gender_and_SCC', 'SCC_div_prod_of_Gender_and_CALC', 'Gender_div_prod_of_CAEC_and_SMOKE', 'CAEC_div_prod_of_Gender_and_SMOKE', 'SMOKE_div_prod_of_Gender_and_CAEC', 'Gender_div_prod_of_CAEC_and_SCC', 'CAEC_div_prod_of_Gender_and_SCC', 'SCC_div_prod_of_Gender_and_CAEC', 'MTRANS_div_prod_of_Gender_and_SMOKE', 'Gender_div_prod_of_MTRANS_and_SCC', 'MTRANS_div_prod_of_Gender_and_SCC', 'SCC_div_prod_of_Gender_and_MTRANS', 'Gender_div_prod_of_family_history_with_overweight_and_SMOKE', 'family_history_with_overweight_div_prod_of_Gender_and_SMOKE', 'SMOKE_div_prod_of_Gender_and_family_history_with_overweight', 'Gender_div_prod_of_family_history_with_overweight_and_SCC', 'family_history_with_overweight_div_prod_of_Gender_and_SCC', 'SCC_div_prod_of_Gender_and_family_history_with_overweight', 'Gender_div_prod_of_FAVC_and_SMOKE', 'FAVC_div_prod_of_Gender_and_SMOKE', 'Gender_div_prod_of_FAVC_and_SCC', 'SCC_div_prod_of_Gender_and_FAVC', 'Gender_div_prod_of_SMOKE_and_SCC', 'SCC_div_prod_of_Gender_and_SMOKE', 'CALC_div_prod_of_CAEC_and_SMOKE', 'CAEC_div_prod_of_CALC_and_SMOKE', 'CALC_div_prod_of_CAEC_and_SCC', 'CAEC_div_prod_of_CALC_and_SCC', 'SCC_div_prod_of_CALC_and_CAEC', 'MTRANS_div_prod_of_CALC_and_SMOKE', 'CALC_div_prod_of_MTRANS_and_SCC', 'MTRANS_div_prod_of_CALC_and_SCC', 'SCC_div_prod_of_CALC_and_MTRANS', 'CALC_div_prod_of_family_history_with_overweight_and_SMOKE', 'family_history_with_overweight_div_prod_of_CALC_and_SMOKE', 'SMOKE_div_prod_of_CALC_and_family_history_with_overweight', 'CALC_div_prod_of_family_history_with_overweight_and_SCC', 'family_history_with_overweight_div_prod_of_CALC_and_SCC', 'SCC_div_prod_of_CALC_and_family_history_with_overweight', 'FAVC_div_prod_of_CALC_and_SMOKE', 'CALC_div_prod_of_FAVC_and_SCC', 'SCC_div_prod_of_CALC_and_SMOKE', 'MTRANS_div_prod_of_CAEC_and_SMOKE', 'SMOKE_div_prod_of_CAEC_and_MTRANS', 'CAEC_div_prod_of_MTRANS_and_SCC', 'MTRANS_div_prod_of_CAEC_and_SCC', 'SCC_div_prod_of_CAEC_and_MTRANS', 'CAEC_div_prod_of_family_history_with_overweight_and_SMOKE', 'family_history_with_overweight_div_prod_of_CAEC_and_SMOKE', 'SMOKE_div_prod_of_CAEC_and_family_history_with_overweight', 'CAEC_div_prod_of_family_history_with_overweight_and_SCC', 'family_history_with_overweight_div_prod_of_CAEC_and_SCC', 'SCC_div_prod_of_CAEC_and_family_history_with_overweight', 'FAVC_div_prod_of_CAEC_and_SMOKE', 'CAEC_div_prod_of_FAVC_and_SCC', 'FAVC_div_prod_of_CAEC_and_SCC', 'SCC_div_prod_of_CAEC_and_FAVC', 'CAEC_div_prod_of_SMOKE_and_SCC', 'SMOKE_div_prod_of_CAEC_and_SCC', 'SCC_div_prod_of_CAEC_and_SMOKE', 'MTRANS_div_prod_of_family_history_with_overweight_and_SMOKE', 'family_history_with_overweight_div_prod_of_MTRANS_and_SMOKE', 'SMOKE_div_prod_of_MTRANS_and_family_history_with_overweight', 'MTRANS_div_prod_of_family_history_with_overweight_and_SCC', 'family_history_with_overweight_div_prod_of_MTRANS_and_SCC', 'SCC_div_prod_of_MTRANS_and_family_history_with_overweight', 'FAVC_div_prod_of_MTRANS_and_SMOKE', 'MTRANS_div_prod_of_FAVC_and_SCC', 'FAVC_div_prod_of_family_history_with_overweight_and_SMOKE', 'family_history_with_overweight_div_prod_of_FAVC_and_SCC', 'FAVC_div_prod_of_family_history_with_overweight_and_SCC', 'SCC_div_prod_of_family_history_with_overweight_and_FAVC', 'family_history_with_overweight_div_prod_of_SMOKE_and_SCC', 'SMOKE_div_prod_of_family_history_with_overweight_and_SCC', 'SCC_div_prod_of_family_history_with_overweight_and_SMOKE'
,'NCP_round', 'Age_round', 'Age_ceil_round', 'Public_Transportation', 'Gender', 'MTRANS', 'family_history_with_overweight', 'FAVC', 'SMOKE', 'SCC', 'Mean_of_Weight_2dec_Height_2dec_Gender', 'Std_of_Weight_2dec_Height_2dec_Gender', 'Mean_of_Weight_2dec_Height_2dec_CALC', 'Std_of_Weight_2dec_Height_2dec_CALC', 'Mean_of_Weight_2dec_Height_2dec_CAEC', 'Std_of_Weight_2dec_Height_2dec_CAEC', 'Mean_of_Weight_2dec_Height_2dec_MTRANS', 'Std_of_Weight_2dec_Height_2dec_MTRANS', 'Mean_of_Weight_2dec_Height_2dec_family_history_with_overweight', 'Std_of_Weight_2dec_Height_2dec_family_history_with_overweight', 'Mean_of_Weight_2dec_Height_2dec_FAVC', 'Std_of_Weight_2dec_Height_2dec_FAVC', 'Mean_of_Weight_2dec_Height_2dec_SMOKE', 'Std_of_Weight_2dec_Height_2dec_SMOKE', 'Mean_of_Weight_2dec_Height_2dec_SCC', 'Std_of_Weight_2dec_Height_2dec_SCC', 'Mean_of_Weight_2dec_Gender_CALC', 'Std_of_Weight_2dec_Gender_CALC', 'Mean_of_Weight_2dec_Gender_CAEC', 'Std_of_Weight_2dec_Gender_CAEC', 'Mean_of_Weight_2dec_Gender_MTRANS', 'Std_of_Weight_2dec_Gender_MTRANS', 'Mean_of_Weight_2dec_Gender_family_history_with_overweight', 'Std_of_Weight_2dec_Gender_family_history_with_overweight', 'Mean_of_Weight_2dec_Gender_FAVC', 'Std_of_Weight_2dec_Gender_FAVC', 'Mean_of_Weight_2dec_Gender_SMOKE', 'Std_of_Weight_2dec_Gender_SMOKE', 'Mean_of_Weight_2dec_Gender_SCC', 'Std_of_Weight_2dec_Gender_SCC', 'Mean_of_Weight_2dec_CALC_CAEC', 'Std_of_Weight_2dec_CALC_CAEC', 'Mean_of_Weight_2dec_CALC_MTRANS', 'Std_of_Weight_2dec_CALC_MTRANS', 'Mean_of_Weight_2dec_CALC_family_history_with_overweight', 'Std_of_Weight_2dec_CALC_family_history_with_overweight', 'Mean_of_Weight_2dec_CALC_FAVC', 'Std_of_Weight_2dec_CALC_FAVC', 'Mean_of_Weight_2dec_CALC_SMOKE', 'Std_of_Weight_2dec_CALC_SMOKE', 'Mean_of_Weight_2dec_CALC_SCC', 'Std_of_Weight_2dec_CALC_SCC', 'Mean_of_Weight_2dec_CAEC_MTRANS', 'Std_of_Weight_2dec_CAEC_MTRANS', 'Mean_of_Weight_2dec_CAEC_family_history_with_overweight', 'Std_of_Weight_2dec_CAEC_family_history_with_overweight', 'Mean_of_Weight_2dec_CAEC_FAVC', 'Std_of_Weight_2dec_CAEC_FAVC', 'Mean_of_Weight_2dec_CAEC_SMOKE', 'Std_of_Weight_2dec_CAEC_SMOKE', 'Mean_of_Weight_2dec_CAEC_SCC', 'Std_of_Weight_2dec_CAEC_SCC', 'Mean_of_Weight_2dec_MTRANS_family_history_with_overweight', 'Std_of_Weight_2dec_MTRANS_family_history_with_overweight', 'Mean_of_Weight_2dec_MTRANS_FAVC', 'Std_of_Weight_2dec_MTRANS_FAVC', 'Mean_of_Weight_2dec_MTRANS_SMOKE', 'Std_of_Weight_2dec_MTRANS_SMOKE', 'Mean_of_Weight_2dec_MTRANS_SCC', 'Std_of_Weight_2dec_MTRANS_SCC', 'Mean_of_Weight_2dec_family_history_with_overweight_FAVC', 'Std_of_Weight_2dec_family_history_with_overweight_FAVC', 'Mean_of_Weight_2dec_family_history_with_overweight_SMOKE', 'Std_of_Weight_2dec_family_history_with_overweight_SMOKE', 'Mean_of_Weight_2dec_family_history_with_overweight_SCC', 'Std_of_Weight_2dec_family_history_with_overweight_SCC', 'Mean_of_Weight_2dec_FAVC_SMOKE', 'Std_of_Weight_2dec_FAVC_SMOKE', 'Mean_of_Weight_2dec_FAVC_SCC', 'Std_of_Weight_2dec_FAVC_SCC', 'Mean_of_Weight_2dec_SMOKE_SCC', 'Std_of_Weight_2dec_SMOKE_SCC', 'Std_of_Height_2dec_Gender_MTRANS', 'Std_of_Height_2dec_Gender_family_history_with_overweight', 'Std_of_Height_2dec_Gender_FAVC', 'Std_of_Height_2dec_Gender_SMOKE', 'Mean_of_Height_2dec_Gender_SCC', 'Std_of_Height_2dec_Gender_SCC', 'Std_of_Height_2dec_CALC_CAEC', 'Std_of_Height_2dec_CALC_MTRANS', 'Std_of_Height_2dec_CALC_family_history_with_overweight', 'Std_of_Height_2dec_CALC_FAVC', 'Mean_of_Height_2dec_CALC_SMOKE', 'Std_of_Height_2dec_CALC_SMOKE', 'Mean_of_Height_2dec_CALC_SCC', 'Std_of_Height_2dec_CALC_SCC', 'Std_of_Height_2dec_CAEC_MTRANS', 'Mean_of_Height_2dec_CAEC_SMOKE', 'Std_of_Height_2dec_CAEC_SMOKE', 'Mean_of_Height_2dec_CAEC_SCC', 'Std_of_Height_2dec_CAEC_SCC', 'Std_of_Height_2dec_MTRANS_SMOKE', 'Mean_of_Height_2dec_MTRANS_SCC', 'Std_of_Height_2dec_MTRANS_SCC', 'Mean_of_Height_2dec_family_history_with_overweight_SMOKE', 'Std_of_Height_2dec_family_history_with_overweight_SMOKE', 'Mean_of_Height_2dec_family_history_with_overweight_SCC', 'Std_of_Height_2dec_FAVC_SMOKE', 'Mean_of_Height_2dec_FAVC_SCC', 'Std_of_Height_2dec_FAVC_SCC', 'Mean_of_Height_2dec_SMOKE_SCC', 'Std_of_Height_2dec_SMOKE_SCC', 'Std_of_Gender_CALC_CAEC', 'Std_of_Gender_CALC_MTRANS', 'Std_of_Gender_CALC_family_history_with_overweight', 'Std_of_Gender_CALC_FAVC', 'Mean_of_Gender_CALC_SMOKE', 'Std_of_Gender_CALC_SMOKE', 'Mean_of_Gender_CALC_SCC', 'Std_of_Gender_CALC_SCC', 'Std_of_Gender_CAEC_MTRANS', 'Std_of_Gender_CAEC_family_history_with_overweight', 'Std_of_Gender_CAEC_FAVC', 'Mean_of_Gender_CAEC_SMOKE', 'Std_of_Gender_CAEC_SMOKE', 'Mean_of_Gender_CAEC_SCC', 'Std_of_Gender_CAEC_SCC', 'Std_of_Gender_MTRANS_family_history_with_overweight', 'Mean_of_Gender_MTRANS_SCC', 'Std_of_Gender_family_history_with_overweight_FAVC', 'Mean_of_Gender_family_history_with_overweight_SMOKE', 'Std_of_Gender_family_history_with_overweight_SMOKE', 'Mean_of_Gender_family_history_with_overweight_SCC', 'Std_of_Gender_family_history_with_overweight_SCC', 'Mean_of_Gender_FAVC_SMOKE', 'Std_of_Gender_FAVC_SMOKE', 'Mean_of_Gender_FAVC_SCC', 'Std_of_Gender_FAVC_SCC', 'Mean_of_Gender_SMOKE_SCC', 'Std_of_Gender_SMOKE_SCC', 'Std_of_CALC_CAEC_MTRANS', 'Std_of_CALC_CAEC_family_history_with_overweight', 'Std_of_CALC_CAEC_FAVC', 'Mean_of_CALC_CAEC_SMOKE', 'Std_of_CALC_CAEC_SMOKE', 'Mean_of_CALC_CAEC_SCC', 'Std_of_CALC_CAEC_SCC', 'Std_of_CALC_MTRANS_family_history_with_overweight', 'Std_of_CALC_MTRANS_FAVC', 'Std_of_CALC_MTRANS_SMOKE', 'Mean_of_CALC_MTRANS_SCC', 'Std_of_CALC_MTRANS_SCC', 'Std_of_CALC_family_history_with_overweight_FAVC', 'Mean_of_CALC_family_history_with_overweight_SMOKE', 'Std_of_CALC_family_history_with_overweight_SMOKE', 'Mean_of_CALC_family_history_with_overweight_SCC', 'Std_of_CALC_family_history_with_overweight_SCC', 'Std_of_CALC_FAVC_SMOKE', 'Mean_of_CALC_FAVC_SCC', 'Std_of_CALC_FAVC_SCC', 'Std_of_CALC_SMOKE_SCC', 'Std_of_CAEC_MTRANS_family_history_with_overweight', 'Std_of_CAEC_MTRANS_FAVC', 'Mean_of_CAEC_MTRANS_SMOKE', 'Std_of_CAEC_MTRANS_SMOKE', 'Mean_of_CAEC_MTRANS_SCC', 'Std_of_CAEC_MTRANS_SCC', 'Std_of_CAEC_family_history_with_overweight_FAVC', 'Mean_of_CAEC_family_history_with_overweight_SMOKE', 'Std_of_CAEC_family_history_with_overweight_SMOKE', 'Mean_of_CAEC_family_history_with_overweight_SCC', 'Std_of_CAEC_family_history_with_overweight_SCC', 'Std_of_CAEC_FAVC_SMOKE', 'Mean_of_CAEC_FAVC_SCC', 'Std_of_CAEC_FAVC_SCC', 'Mean_of_CAEC_SMOKE_SCC', 'Std_of_CAEC_SMOKE_SCC', 'Std_of_MTRANS_family_history_with_overweight_FAVC', 'Mean_of_MTRANS_family_history_with_overweight_SMOKE', 'Std_of_MTRANS_family_history_with_overweight_SMOKE', 'Mean_of_MTRANS_family_history_with_overweight_SCC', 'Std_of_MTRANS_family_history_with_overweight_SCC', 'Std_of_MTRANS_FAVC_SMOKE', 'Mean_of_MTRANS_FAVC_SCC', 'Std_of_MTRANS_FAVC_SCC', 'Std_of_family_history_with_overweight_FAVC_SMOKE', 'Mean_of_family_history_with_overweight_FAVC_SCC', 'Std_of_family_history_with_overweight_FAVC_SCC', 'Mean_of_family_history_with_overweight_SMOKE_SCC', 'Std_of_family_history_with_overweight_SMOKE_SCC'
,'NCP_round', 'Age_round', 'Age_ceil_round', 'Public_Transportation', 'Gender', 'MTRANS', 'family_history_with_overweight', 'FAVC', 'SMOKE', 'SCC', 'FCVC_round+FCVC_ceil_round', 'FCVC_round+CH2O_ceil_round', 'FCVC_round+NCP_floor_round', 'FCVC_round+NCP_round', 'FCVC_round+FAF_round', 'FCVC_round+FCVC_floor_round', 'FCVC_round+Age_floor_round', 'FCVC_round+Age_round', 'FCVC_round+Age_ceil_round', 'FCVC_round+Bike', 'FCVC_round+Motorbike', 'FCVC_round+Walking', 'FCVC_round+SMOKE_map', 'FCVC_round+SCC_map', 'FCVC_round+FAVC_map', 'FCVC_ceil_round+CH2O_round', 'FCVC_ceil_round+TUE_round', 'FCVC_ceil_round+NCP_ceil_round', 'FCVC_ceil_round+FAF_ceil_round', 'FCVC_ceil_round+CH2O_ceil_round', 'FCVC_ceil_round+NCP_floor_round', 'FCVC_ceil_round+NCP_round', 'FCVC_ceil_round+FAF_floor_round', 'FCVC_ceil_round+TUE_ceil_round', 'FCVC_ceil_round+FAF_round', 'FCVC_ceil_round+CH2O_floor_round', 'FCVC_ceil_round+FCVC_floor_round', 'FCVC_ceil_round+Age_floor_round', 'FCVC_ceil_round+Age_round', 'FCVC_ceil_round+Age_ceil_round', 'FCVC_ceil_round+Bike', 'FCVC_ceil_round+Motorbike', 'FCVC_ceil_round+Public_Transportation', 'FCVC_ceil_round+Walking', 'FCVC_ceil_round+SMOKE_map', 'FCVC_ceil_round+SCC_map', 'FCVC_ceil_round+CALC_map', 'FCVC_ceil_round+FAVC_map', 'CH2O_round+CH2O_ceil_round', 'CH2O_round+NCP_floor_round', 'CH2O_round+NCP_round', 'CH2O_round+TUE_ceil_round', 'CH2O_round+FAF_round', 'CH2O_round+CH2O_floor_round', 'CH2O_round+FCVC_floor_round', 'CH2O_round+Age_floor_round', 'CH2O_round+Age_round', 'CH2O_round+Age_ceil_round', 'CH2O_round+Bike', 'CH2O_round+Motorbike', 'CH2O_round+Walking', 'CH2O_round+SMOKE_map', 'CH2O_round+SCC_map', 'CH2O_round+family_history_with_overweight_map', 'CH2O_round+FAVC_map', 'TUE_round+NCP_floor_round', 'TUE_round+NCP_round', 'TUE_round+TUE_ceil_round', 'TUE_round+TUE_floor_round', 'TUE_round+FAF_round', 'TUE_round+FCVC_floor_round', 'TUE_round+Age_floor_round', 'TUE_round+Age_round', 'TUE_round+Age_ceil_round', 'TUE_round+Bike', 'TUE_round+Motorbike', 'TUE_round+Walking', 'TUE_round+SMOKE_map', 'TUE_round+SCC_map', 'TUE_round+FAVC_map', 'NCP_ceil_round+NCP_floor_round', 'NCP_ceil_round+NCP_round', 'NCP_ceil_round+FAF_round', 'NCP_ceil_round+Age_floor_round', 'NCP_ceil_round+Age_round', 'NCP_ceil_round+Age_ceil_round', 'NCP_ceil_round+Bike', 'NCP_ceil_round+Motorbike', 'NCP_ceil_round+Walking', 'NCP_ceil_round+SMOKE_map', 'NCP_ceil_round+SCC_map', 'FAF_ceil_round+CH2O_ceil_round', 'FAF_ceil_round+NCP_floor_round', 'FAF_ceil_round+NCP_round', 'FAF_ceil_round+FAF_floor_round', 'FAF_ceil_round+TUE_ceil_round', 'FAF_ceil_round+TUE_floor_round', 'FAF_ceil_round+FAF_round', 'FAF_ceil_round+CH2O_floor_round', 'FAF_ceil_round+FCVC_floor_round', 'FAF_ceil_round+Age_floor_round', 'FAF_ceil_round+Age_round', 'FAF_ceil_round+Age_ceil_round', 'FAF_ceil_round+Automobile', 'FAF_ceil_round+Bike', 'FAF_ceil_round+Motorbike', 'FAF_ceil_round+Public_Transportation', 'FAF_ceil_round+Walking', 'FAF_ceil_round+SMOKE_map', 'FAF_ceil_round+SCC_map', 'FAF_ceil_round+family_history_with_overweight_map', 'FAF_ceil_round+FAVC_map', 'FAF_ceil_round+CAEC_map', 'CH2O_ceil_round+NCP_floor_round', 'CH2O_ceil_round+NCP_round', 'CH2O_ceil_round+FAF_floor_round', 'CH2O_ceil_round+TUE_ceil_round', 'CH2O_ceil_round+FAF_round', 'CH2O_ceil_round+CH2O_floor_round', 'CH2O_ceil_round+FCVC_floor_round', 'CH2O_ceil_round+Age_floor_round', 'CH2O_ceil_round+Age_round', 'CH2O_ceil_round+Age_ceil_round', 'CH2O_ceil_round+Bike', 'CH2O_ceil_round+Motorbike', 'CH2O_ceil_round+Walking', 'CH2O_ceil_round+SMOKE_map', 'CH2O_ceil_round+SCC_map', 'CH2O_ceil_round+family_history_with_overweight_map', 'CH2O_ceil_round+FAVC_map', 'NCP_floor_round+NCP_round', 'NCP_floor_round+FAF_floor_round', 'NCP_floor_round+TUE_ceil_round', 'NCP_floor_round+TUE_floor_round', 'NCP_floor_round+FAF_round', 'NCP_floor_round+CH2O_floor_round', 'NCP_floor_round+FCVC_floor_round', 'NCP_floor_round+Age_floor_round', 'NCP_floor_round+Age_round', 'NCP_floor_round+Age_ceil_round', 'NCP_floor_round+Bike', 'NCP_floor_round+Motorbike', 'NCP_floor_round+Walking', 'NCP_floor_round+SMOKE_map', 'NCP_floor_round+SCC_map', 'NCP_floor_round+CALC_map', 'NCP_floor_round+FAVC_map', 'NCP_round+FAF_floor_round', 'NCP_round+TUE_ceil_round', 'NCP_round+TUE_floor_round', 'NCP_round+FAF_round', 'NCP_round+CH2O_floor_round', 'NCP_round+FCVC_floor_round', 'NCP_round+Age_floor_round', 'NCP_round+Age_round', 'NCP_round+Age_ceil_round', 'NCP_round+Automobile', 'NCP_round+Bike', 'NCP_round+Motorbike', 'NCP_round+Public_Transportation', 'NCP_round+Walking', 'NCP_round+SMOKE_map', 'NCP_round+SCC_map', 'NCP_round+Gender_map', 'NCP_round+CALC_map', 'NCP_round+family_history_with_overweight_map', 'NCP_round+FAVC_map', 'NCP_round+CAEC_map', 'FAF_floor_round+TUE_ceil_round', 'FAF_floor_round+TUE_floor_round', 'FAF_floor_round+FAF_round', 'FAF_floor_round+FCVC_floor_round', 'FAF_floor_round+Age_floor_round', 'FAF_floor_round+Age_round', 'FAF_floor_round+Age_ceil_round', 'FAF_floor_round+Bike', 'FAF_floor_round+Motorbike', 'FAF_floor_round+Public_Transportation', 'FAF_floor_round+Walking', 'FAF_floor_round+SMOKE_map', 'FAF_floor_round+SCC_map', 'FAF_floor_round+FAVC_map', 'FAF_floor_round+CAEC_map', 'TUE_ceil_round+TUE_floor_round', 'TUE_ceil_round+FAF_round', 'TUE_ceil_round+FCVC_floor_round', 'TUE_ceil_round+Age_floor_round', 'TUE_ceil_round+Age_round', 'TUE_ceil_round+Age_ceil_round', 'TUE_ceil_round+Bike', 'TUE_ceil_round+Motorbike', 'TUE_ceil_round+Public_Transportation', 'TUE_ceil_round+Walking', 'TUE_ceil_round+SMOKE_map', 'TUE_ceil_round+SCC_map', 'TUE_ceil_round+FAVC_map', 'TUE_floor_round+FAF_round', 'TUE_floor_round+Age_floor_round', 'TUE_floor_round+Age_round', 'TUE_floor_round+Age_ceil_round', 'TUE_floor_round+Bike', 'TUE_floor_round+Motorbike', 'TUE_floor_round+Walking', 'TUE_floor_round+SMOKE_map', 'TUE_floor_round+SCC_map', 'TUE_floor_round+FAVC_map', 'FAF_round+CH2O_floor_round', 'FAF_round+FCVC_floor_round', 'FAF_round+Age_floor_round', 'FAF_round+Age_round', 'FAF_round+Age_ceil_round', 'FAF_round+Automobile', 'FAF_round+Bike', 'FAF_round+Motorbike', 'FAF_round+Public_Transportation', 'FAF_round+Walking', 'FAF_round+SMOKE_map', 'FAF_round+SCC_map', 'FAF_round+Gender_map', 'FAF_round+CALC_map', 'FAF_round+family_history_with_overweight_map', 'FAF_round+FAVC_map', 'FAF_round+CAEC_map', 'CH2O_floor_round+Age_floor_round', 'CH2O_floor_round+Age_round', 'CH2O_floor_round+Age_ceil_round', 'CH2O_floor_round+Bike', 'CH2O_floor_round+Motorbike', 'CH2O_floor_round+Walking', 'CH2O_floor_round+SMOKE_map', 'CH2O_floor_round+SCC_map', 'CH2O_floor_round+FAVC_map', 'FCVC_floor_round+Age_floor_round', 'FCVC_floor_round+Age_round', 'FCVC_floor_round+Age_ceil_round', 'FCVC_floor_round+Bike', 'FCVC_floor_round+Motorbike', 'FCVC_floor_round+Walking', 'FCVC_floor_round+SMOKE_map', 'FCVC_floor_round+SCC_map', 'FCVC_floor_round+FAVC_map', 'Age_floor_round+Age_round', 'Age_floor_round+Age_ceil_round', 'Age_floor_round+Automobile', 'Age_floor_round+Bike', 'Age_floor_round+Motorbike', 'Age_floor_round+Public_Transportation', 'Age_floor_round+Walking', 'Age_floor_round+SMOKE_map', 'Age_floor_round+SCC_map', 'Age_floor_round+Gender_map', 'Age_floor_round+CALC_map', 'Age_floor_round+family_history_with_overweight_map', 'Age_floor_round+FAVC_map', 'Age_floor_round+CAEC_map', 'Age_round+Age_ceil_round', 'Age_round+Automobile', 'Age_round+Bike', 'Age_round+Motorbike', 'Age_round+Public_Transportation', 'Age_round+Walking', 'Age_round+SMOKE_map', 'Age_round+SCC_map', 'Age_round+Gender_map', 'Age_round+CALC_map', 'Age_round+family_history_with_overweight_map', 'Age_round+FAVC_map', 'Age_round+CAEC_map', 'Age_ceil_round+Automobile', 'Age_ceil_round+Bike', 'Age_ceil_round+Motorbike', 'Age_ceil_round+Public_Transportation', 'Age_ceil_round+Walking', 'Age_ceil_round+SMOKE_map', 'Age_ceil_round+SCC_map', 'Age_ceil_round+Gender_map', 'Age_ceil_round+CALC_map', 'Age_ceil_round+family_history_with_overweight_map', 'Age_ceil_round+FAVC_map', 'Age_ceil_round+CAEC_map', 'Automobile+Bike', 'Automobile+Motorbike', 'Automobile+Public_Transportation', 'Automobile+Walking', 'Automobile+SMOKE_map', 'Automobile+SCC_map', 'Bike+Public_Transportation', 'Bike+Walking', 'Bike+SMOKE_map', 'Bike+SCC_map', 'Bike+Gender_map', 'Bike+CALC_map', 'Bike+family_history_with_overweight_map', 'Bike+FAVC_map', 'Bike+CAEC_map', 'Motorbike+Public_Transportation', 'Motorbike+Walking', 'Motorbike+SMOKE_map', 'Motorbike+SCC_map', 'Motorbike+Gender_map', 'Motorbike+CALC_map', 'Motorbike+family_history_with_overweight_map', 'Motorbike+FAVC_map', 'Motorbike+CAEC_map', 'Public_Transportation+Walking', 'Public_Transportation+SMOKE_map', 'Public_Transportation+SCC_map', 'Walking+Gender_map', 'Walking+CALC_map', 'Walking+family_history_with_overweight_map', 'Walking+CAEC_map', 'SMOKE_map+Gender_map', 'SMOKE_map+CALC_map', 'SMOKE_map+family_history_with_overweight_map', 'SMOKE_map+FAVC_map', 'SMOKE_map+CAEC_map', 'SCC_map+Gender_map', 'SCC_map+CALC_map', 'SCC_map+family_history_with_overweight_map', 'SCC_map+CAEC_map', 'Gender_map+FAVC_map'
,'NCP_round', 'Age_round', 'Age_ceil_round', 'Public_Transportation', 'Gender', 'MTRANS', 'family_history_with_overweight', 'FAVC', 'SMOKE', 'SCC', 'BMI_2dec', 'BMI_category']))
len(spearman_high_corr_feats)

2033

In [33]:
pearson_high_corr_feats = list(set(['NCP_round', 'NCP_floor_round', 'Age_floor_round', 'Age_ceil_round', 'Public_Transportation', 'Gender', 'MTRANS', 'family_history_with_overweight', 'FAVC', 'SMOKE', 'SCC', 'Height_2dec_log', 'Height_2dec_sqrt', 'Height_2dec_degree2', 'Height_2dec_degree3', 'Weight_2dec_log', 'Weight_2dec_sqrt', 'Weight_2dec_degree2', 'Weight_2dec_degree3', 'Gender_log', 'Gender_sqrt', 'Gender_degree2', 'Gender_degree3', 'CALC_log', 'CALC_sqrt', 'CALC_degree2', 'CALC_degree3', 'CAEC_log', 'CAEC_sqrt', 'CAEC_degree2', 'CAEC_degree3', 'MTRANS_log', 'MTRANS_sqrt', 'MTRANS_degree2', 'MTRANS_degree3', 'family_history_with_overweight_log', 'family_history_with_overweight_sqrt', 'family_history_with_overweight_degree2', 'family_history_with_overweight_degree3', 'FAVC_log', 'FAVC_sqrt', 'FAVC_degree2', 'FAVC_degree3', 'SMOKE_log', 'SMOKE_sqrt', 'SMOKE_degree2', 'SMOKE_degree3', 'SCC_log', 'SCC_sqrt', 'SCC_degree2', 'SCC_degree3'
,'NCP_round', 'NCP_floor_round', 'Age_floor_round', 'Age_ceil_round', 'Public_Transportation', 'Gender', 'MTRANS', 'family_history_with_overweight', 'FAVC', 'SMOKE', 'SCC', 'Height_2dec_x_Weight_2dec', 'Height_2dec_plus_Weight_2dec', 'Height_2dec_minus_Weight_2dec', 'Weight_2dec_minus_Height_2dec', 'Height_2dec_div_Weight_2dec', 'Weight_2dec_div_Height_2dec', 'Mean_Height_2dec_Weight_2dec', 'Std_Height_2dec_Weight_2dec', 'percent_sum_Height_2dec_Weight_2dec', 'percent_sum_Weight_2dec_Height_2dec', 'Height_2dec_minus_Gender', 'Gender_minus_Height_2dec', 'Height_2dec_div_Gender', 'Gender_div_Height_2dec', 'Mean_Height_2dec_Gender', 'Std_Height_2dec_Gender', 'percent_sum_Height_2dec_Gender', 'percent_sum_Gender_Height_2dec', 'Height_2dec_x_CALC', 'Height_2dec_plus_CALC', 'Height_2dec_minus_CALC', 'CALC_minus_Height_2dec', 'Height_2dec_div_CALC', 'CALC_div_Height_2dec', 'Mean_Height_2dec_CALC', 'Std_Height_2dec_CALC', 'percent_sum_Height_2dec_CALC', 'percent_sum_CALC_Height_2dec', 'Height_2dec_x_CAEC', 'Height_2dec_plus_CAEC', 'Height_2dec_minus_CAEC', 'CAEC_minus_Height_2dec', 'Height_2dec_div_CAEC', 'CAEC_div_Height_2dec', 'Mean_Height_2dec_CAEC', 'Std_Height_2dec_CAEC', 'percent_sum_Height_2dec_CAEC', 'percent_sum_CAEC_Height_2dec', 'Height_2dec_gt_CAEC', 'Height_2dec_plus_MTRANS', 'Height_2dec_minus_MTRANS', 'MTRANS_minus_Height_2dec', 'Height_2dec_div_MTRANS', 'MTRANS_div_Height_2dec', 'Mean_Height_2dec_MTRANS', 'Std_Height_2dec_MTRANS', 'percent_sum_Height_2dec_MTRANS', 'percent_sum_MTRANS_Height_2dec', 'Height_2dec_gt_MTRANS', 'Height_2dec_x_family_history_with_overweight', 'Height_2dec_plus_family_history_with_overweight', 'Height_2dec_minus_family_history_with_overweight', 'family_history_with_overweight_minus_Height_2dec', 'Height_2dec_div_family_history_with_overweight', 'family_history_with_overweight_div_Height_2dec', 'Mean_Height_2dec_family_history_with_overweight', 'Std_Height_2dec_family_history_with_overweight', 'percent_sum_Height_2dec_family_history_with_overweight', 'percent_sum_family_history_with_overweight_Height_2dec', 'Height_2dec_gt_family_history_with_overweight', 'Height_2dec_x_FAVC', 'Height_2dec_plus_FAVC', 'Height_2dec_minus_FAVC', 'FAVC_minus_Height_2dec', 'Height_2dec_div_FAVC', 'FAVC_div_Height_2dec', 'Mean_Height_2dec_FAVC', 'Std_Height_2dec_FAVC', 'percent_sum_Height_2dec_FAVC', 'percent_sum_FAVC_Height_2dec', 'Height_2dec_x_SMOKE', 'Height_2dec_plus_SMOKE', 'Height_2dec_minus_SMOKE', 'SMOKE_minus_Height_2dec', 'Height_2dec_div_SMOKE', 'SMOKE_div_Height_2dec', 'Mean_Height_2dec_SMOKE', 'Std_Height_2dec_SMOKE', 'percent_sum_Height_2dec_SMOKE', 'percent_sum_SMOKE_Height_2dec', 'Height_2dec_x_SCC', 'Height_2dec_plus_SCC', 'Height_2dec_minus_SCC', 'SCC_minus_Height_2dec', 'Height_2dec_div_SCC', 'SCC_div_Height_2dec', 'Mean_Height_2dec_SCC', 'Std_Height_2dec_SCC', 'percent_sum_Height_2dec_SCC', 'percent_sum_SCC_Height_2dec', 'Height_2dec_gt_SCC', 'Weight_2dec_x_Gender', 'Weight_2dec_plus_Gender', 'Weight_2dec_minus_Gender', 'Gender_minus_Weight_2dec', 'Weight_2dec_div_Gender', 'Gender_div_Weight_2dec', 'Mean_Weight_2dec_Gender', 'Std_Weight_2dec_Gender', 'percent_sum_Weight_2dec_Gender', 'percent_sum_Gender_Weight_2dec', 'Weight_2dec_x_CALC', 'Weight_2dec_plus_CALC', 'Weight_2dec_minus_CALC', 'CALC_minus_Weight_2dec', 'CALC_div_Weight_2dec', 'Mean_Weight_2dec_CALC', 'Std_Weight_2dec_CALC', 'percent_sum_Weight_2dec_CALC', 'percent_sum_CALC_Weight_2dec', 'Weight_2dec_x_CAEC', 'Weight_2dec_plus_CAEC', 'Weight_2dec_minus_CAEC', 'CAEC_minus_Weight_2dec', 'Mean_Weight_2dec_CAEC', 'Std_Weight_2dec_CAEC', 'percent_sum_Weight_2dec_CAEC', 'percent_sum_CAEC_Weight_2dec', 'Weight_2dec_x_MTRANS', 'Weight_2dec_plus_MTRANS', 'Weight_2dec_minus_MTRANS', 'MTRANS_minus_Weight_2dec', 'Weight_2dec_div_MTRANS', 'MTRANS_div_Weight_2dec', 'Mean_Weight_2dec_MTRANS', 'Std_Weight_2dec_MTRANS', 'percent_sum_Weight_2dec_MTRANS', 'percent_sum_MTRANS_Weight_2dec', 'Weight_2dec_x_family_history_with_overweight', 'Weight_2dec_plus_family_history_with_overweight', 'Weight_2dec_minus_family_history_with_overweight', 'family_history_with_overweight_minus_Weight_2dec', 'Mean_Weight_2dec_family_history_with_overweight', 'Std_Weight_2dec_family_history_with_overweight', 'percent_sum_Weight_2dec_family_history_with_overweight', 'percent_sum_family_history_with_overweight_Weight_2dec', 'Weight_2dec_x_FAVC', 'Weight_2dec_plus_FAVC', 'Weight_2dec_minus_FAVC', 'FAVC_minus_Weight_2dec', 'FAVC_div_Weight_2dec', 'Mean_Weight_2dec_FAVC', 'Std_Weight_2dec_FAVC', 'percent_sum_Weight_2dec_FAVC', 'percent_sum_FAVC_Weight_2dec', 'Weight_2dec_x_SMOKE', 'Weight_2dec_plus_SMOKE', 'Weight_2dec_minus_SMOKE', 'SMOKE_minus_Weight_2dec', 'Weight_2dec_div_SMOKE', 'SMOKE_div_Weight_2dec', 'Mean_Weight_2dec_SMOKE', 'Std_Weight_2dec_SMOKE', 'percent_sum_Weight_2dec_SMOKE', 'percent_sum_SMOKE_Weight_2dec', 'Weight_2dec_x_SCC', 'Weight_2dec_plus_SCC', 'Weight_2dec_minus_SCC', 'SCC_minus_Weight_2dec', 'SCC_div_Weight_2dec', 'Mean_Weight_2dec_SCC', 'Std_Weight_2dec_SCC', 'percent_sum_Weight_2dec_SCC', 'percent_sum_SCC_Weight_2dec', 'Gender_x_CALC', 'Gender_plus_CALC', 'Gender_minus_CALC', 'CALC_minus_Gender', 'Gender_div_CALC', 'CALC_div_Gender', 'Mean_Gender_CALC', 'percent_sum_Gender_CALC', 'percent_sum_CALC_Gender', 'Gender_gt_CALC', 'Gender_x_CAEC', 'Gender_plus_CAEC', 'Gender_minus_CAEC', 'CAEC_minus_Gender', 'Gender_div_CAEC', 'CAEC_div_Gender', 'Mean_Gender_CAEC', 'Std_Gender_CAEC', 'percent_sum_Gender_CAEC', 'percent_sum_CAEC_Gender', 'Gender_gt_CAEC', 'Gender_x_MTRANS', 'Gender_plus_MTRANS', 'Gender_minus_MTRANS', 'MTRANS_minus_Gender', 'Gender_div_MTRANS', 'MTRANS_div_Gender', 'Mean_Gender_MTRANS', 'Std_Gender_MTRANS', 'percent_sum_Gender_MTRANS', 'percent_sum_MTRANS_Gender', 'Gender_gt_MTRANS', 'Gender_x_family_history_with_overweight', 'Gender_plus_family_history_with_overweight', 'Gender_minus_family_history_with_overweight', 'family_history_with_overweight_minus_Gender', 'Gender_div_family_history_with_overweight', 'family_history_with_overweight_div_Gender', 'Mean_Gender_family_history_with_overweight', 'Std_Gender_family_history_with_overweight', 'percent_sum_Gender_family_history_with_overweight', 'percent_sum_family_history_with_overweight_Gender', 'Gender_gt_family_history_with_overweight', 'Gender_x_FAVC', 'Gender_plus_FAVC', 'Gender_minus_FAVC', 'FAVC_minus_Gender', 'Gender_div_FAVC', 'FAVC_div_Gender', 'Mean_Gender_FAVC', 'Std_Gender_FAVC', 'percent_sum_Gender_FAVC', 'percent_sum_FAVC_Gender', 'Gender_gt_FAVC', 'Gender_x_SMOKE', 'Gender_plus_SMOKE', 'Gender_minus_SMOKE', 'SMOKE_minus_Gender', 'Gender_div_SMOKE', 'SMOKE_div_Gender', 'Mean_Gender_SMOKE', 'percent_sum_Gender_SMOKE', 'percent_sum_SMOKE_Gender', 'Gender_gt_SMOKE', 'Gender_x_SCC', 'Gender_plus_SCC', 'Gender_minus_SCC', 'SCC_minus_Gender', 'Gender_div_SCC', 'SCC_div_Gender', 'Mean_Gender_SCC', 'Std_Gender_SCC', 'percent_sum_Gender_SCC', 'percent_sum_SCC_Gender', 'Gender_gt_SCC', 'CALC_plus_CAEC', 'CAEC_minus_CALC', 'CALC_div_CAEC', 'CAEC_div_CALC', 'Mean_CALC_CAEC', 'Std_CALC_CAEC', 'percent_sum_CALC_CAEC', 'percent_sum_CAEC_CALC', 'CALC_gt_CAEC', 'CALC_plus_MTRANS', 'MTRANS_minus_CALC', 'CALC_div_MTRANS', 'MTRANS_div_CALC', 'Mean_CALC_MTRANS', 'Std_CALC_MTRANS', 'percent_sum_CALC_MTRANS', 'percent_sum_MTRANS_CALC', 'CALC_gt_MTRANS', 'CALC_x_family_history_with_overweight', 'CALC_plus_family_history_with_overweight', 'CALC_minus_family_history_with_overweight', 'family_history_with_overweight_minus_CALC', 'CALC_div_family_history_with_overweight', 'family_history_with_overweight_div_CALC', 'Mean_CALC_family_history_with_overweight', 'Std_CALC_family_history_with_overweight', 'percent_sum_CALC_family_history_with_overweight', 'percent_sum_family_history_with_overweight_CALC', 'CALC_gt_family_history_with_overweight', 'CALC_plus_FAVC', 'FAVC_minus_CALC', 'CALC_div_FAVC', 'FAVC_div_CALC', 'Mean_CALC_FAVC', 'percent_sum_CALC_FAVC', 'percent_sum_FAVC_CALC', 'CALC_gt_FAVC', 'CALC_x_SMOKE', 'CALC_plus_SMOKE', 'CALC_minus_SMOKE', 'SMOKE_minus_CALC', 'CALC_div_SMOKE', 'SMOKE_div_CALC', 'Mean_CALC_SMOKE', 'Std_CALC_SMOKE', 'percent_sum_CALC_SMOKE', 'percent_sum_SMOKE_CALC', 'CALC_gt_SMOKE', 'CALC_plus_SCC', 'SCC_minus_CALC', 'SCC_div_CALC', 'Mean_CALC_SCC', 'Std_CALC_SCC', 'percent_sum_CALC_SCC', 'percent_sum_SCC_CALC', 'CALC_gt_SCC', 'CAEC_x_MTRANS', 'CAEC_plus_MTRANS', 'CAEC_minus_MTRANS', 'MTRANS_minus_CAEC', 'CAEC_div_MTRANS', 'MTRANS_div_CAEC', 'Mean_CAEC_MTRANS', 'Std_CAEC_MTRANS', 'percent_sum_CAEC_MTRANS', 'percent_sum_MTRANS_CAEC', 'CAEC_gt_MTRANS', 'CAEC_plus_family_history_with_overweight', 'family_history_with_overweight_minus_CAEC', 'CAEC_div_family_history_with_overweight', 'Mean_CAEC_family_history_with_overweight', 'percent_sum_CAEC_family_history_with_overweight', 'percent_sum_family_history_with_overweight_CAEC', 'CAEC_gt_family_history_with_overweight', 'CAEC_x_FAVC', 'CAEC_plus_FAVC', 'FAVC_minus_CAEC', 'CAEC_div_FAVC', 'FAVC_div_CAEC', 'Mean_CAEC_FAVC', 'Std_CAEC_FAVC', 'percent_sum_CAEC_FAVC', 'percent_sum_FAVC_CAEC', 'CAEC_gt_FAVC', 'CAEC_x_SMOKE', 'CAEC_plus_SMOKE', 'CAEC_minus_SMOKE', 'SMOKE_minus_CAEC', 'CAEC_div_SMOKE', 'SMOKE_div_CAEC', 'Mean_CAEC_SMOKE', 'Std_CAEC_SMOKE', 'percent_sum_CAEC_SMOKE', 'percent_sum_SMOKE_CAEC', 'CAEC_gt_SMOKE', 'CAEC_x_SCC', 'CAEC_plus_SCC', 'CAEC_minus_SCC', 'SCC_minus_CAEC', 'CAEC_div_SCC', 'SCC_div_CAEC', 'Mean_CAEC_SCC', 'Std_CAEC_SCC', 'percent_sum_CAEC_SCC', 'percent_sum_SCC_CAEC', 'CAEC_gt_SCC', 'MTRANS_x_family_history_with_overweight', 'MTRANS_plus_family_history_with_overweight', 'MTRANS_minus_family_history_with_overweight', 'family_history_with_overweight_minus_MTRANS', 'MTRANS_div_family_history_with_overweight', 'family_history_with_overweight_div_MTRANS', 'Mean_MTRANS_family_history_with_overweight', 'Std_MTRANS_family_history_with_overweight', 'percent_sum_MTRANS_family_history_with_overweight', 'percent_sum_family_history_with_overweight_MTRANS', 'MTRANS_gt_family_history_with_overweight', 'MTRANS_plus_FAVC', 'FAVC_minus_MTRANS', 'MTRANS_div_FAVC', 'FAVC_div_MTRANS', 'Mean_MTRANS_FAVC', 'Std_MTRANS_FAVC', 'percent_sum_MTRANS_FAVC', 'percent_sum_FAVC_MTRANS', 'MTRANS_gt_FAVC', 'MTRANS_x_SMOKE', 'MTRANS_plus_SMOKE', 'MTRANS_minus_SMOKE', 'SMOKE_minus_MTRANS', 'MTRANS_div_SMOKE', 'SMOKE_div_MTRANS', 'Mean_MTRANS_SMOKE', 'Std_MTRANS_SMOKE', 'percent_sum_MTRANS_SMOKE', 'percent_sum_SMOKE_MTRANS', 'MTRANS_gt_SMOKE', 'MTRANS_plus_SCC', 'SCC_minus_MTRANS', 'MTRANS_div_SCC', 'SCC_div_MTRANS', 'Mean_MTRANS_SCC', 'Std_MTRANS_SCC', 'percent_sum_MTRANS_SCC', 'percent_sum_SCC_MTRANS', 'MTRANS_gt_SCC', 'family_history_with_overweight_x_FAVC', 'family_history_with_overweight_plus_FAVC', 'family_history_with_overweight_minus_FAVC', 'FAVC_minus_family_history_with_overweight', 'family_history_with_overweight_div_FAVC', 'FAVC_div_family_history_with_overweight', 'Mean_family_history_with_overweight_FAVC', 'Std_family_history_with_overweight_FAVC', 'percent_sum_family_history_with_overweight_FAVC', 'percent_sum_FAVC_family_history_with_overweight', 'family_history_with_overweight_gt_FAVC', 'family_history_with_overweight_x_SMOKE', 'family_history_with_overweight_plus_SMOKE', 'family_history_with_overweight_minus_SMOKE', 'SMOKE_minus_family_history_with_overweight', 'family_history_with_overweight_div_SMOKE', 'SMOKE_div_family_history_with_overweight', 'Mean_family_history_with_overweight_SMOKE', 'Std_family_history_with_overweight_SMOKE', 'percent_sum_family_history_with_overweight_SMOKE', 'percent_sum_SMOKE_family_history_with_overweight', 'family_history_with_overweight_gt_SMOKE', 'family_history_with_overweight_x_SCC', 'family_history_with_overweight_plus_SCC', 'family_history_with_overweight_minus_SCC', 'SCC_minus_family_history_with_overweight', 'family_history_with_overweight_div_SCC', 'SCC_div_family_history_with_overweight', 'Mean_family_history_with_overweight_SCC', 'Std_family_history_with_overweight_SCC', 'percent_sum_family_history_with_overweight_SCC', 'percent_sum_SCC_family_history_with_overweight', 'family_history_with_overweight_gt_SCC', 'FAVC_x_SMOKE', 'FAVC_plus_SMOKE', 'FAVC_minus_SMOKE', 'SMOKE_minus_FAVC', 'FAVC_div_SMOKE', 'SMOKE_div_FAVC', 'Mean_FAVC_SMOKE', 'Std_FAVC_SMOKE', 'percent_sum_FAVC_SMOKE', 'percent_sum_SMOKE_FAVC', 'FAVC_gt_SMOKE', 'FAVC_plus_SCC', 'SCC_minus_FAVC', 'SCC_div_FAVC', 'Mean_FAVC_SCC', 'Std_FAVC_SCC', 'percent_sum_FAVC_SCC', 'percent_sum_SCC_FAVC', 'FAVC_gt_SCC', 'SMOKE_x_SCC', 'SMOKE_plus_SCC', 'SMOKE_minus_SCC', 'SCC_minus_SMOKE', 'SMOKE_div_SCC', 'SCC_div_SMOKE', 'Mean_SMOKE_SCC', 'Std_SMOKE_SCC', 'percent_sum_SMOKE_SCC', 'percent_sum_SCC_SMOKE'
,'NCP_round', 'Age_round', 'Age_ceil_round', 'Public_Transportation', 'Gender', 'MTRANS', 'family_history_with_overweight', 'FAVC', 'SMOKE', 'SCC', 'Weight_2dec_plus_Height_2dec_minus_Gender', 'Weight_2dec_plus_Gender_minus_Height_2dec', 'Height_2dec_plus_Gender_minus_Weight_2dec', 'Weight_2dec_plus_Height_2dec_plus_Gender', 'Weight_2dec_plus_Height_2dec_minus_CALC', 'Weight_2dec_plus_CALC_minus_Height_2dec', 'Height_2dec_plus_CALC_minus_Weight_2dec', 'Weight_2dec_plus_Height_2dec_plus_CALC', 'Weight_2dec_plus_Height_2dec_minus_CAEC', 'Weight_2dec_plus_CAEC_minus_Height_2dec', 'Height_2dec_plus_CAEC_minus_Weight_2dec', 'Weight_2dec_plus_Height_2dec_plus_CAEC', 'Weight_2dec_plus_Height_2dec_minus_MTRANS', 'Weight_2dec_plus_MTRANS_minus_Height_2dec', 'Height_2dec_plus_MTRANS_minus_Weight_2dec', 'Weight_2dec_plus_Height_2dec_plus_MTRANS', 'Weight_2dec_plus_Height_2dec_minus_family_history_with_overweight', 'Weight_2dec_plus_family_history_with_overweight_minus_Height_2dec', 'Height_2dec_plus_family_history_with_overweight_minus_Weight_2dec', 'Weight_2dec_plus_Height_2dec_plus_family_history_with_overweight', 'Weight_2dec_plus_Height_2dec_minus_FAVC', 'Weight_2dec_plus_FAVC_minus_Height_2dec', 'Height_2dec_plus_FAVC_minus_Weight_2dec', 'Weight_2dec_plus_Height_2dec_plus_FAVC', 'Weight_2dec_plus_Height_2dec_minus_SMOKE', 'Weight_2dec_plus_SMOKE_minus_Height_2dec', 'Height_2dec_plus_SMOKE_minus_Weight_2dec', 'Weight_2dec_plus_Height_2dec_plus_SMOKE', 'Weight_2dec_plus_Height_2dec_minus_SCC', 'Weight_2dec_plus_SCC_minus_Height_2dec', 'Height_2dec_plus_SCC_minus_Weight_2dec', 'Weight_2dec_plus_Height_2dec_plus_SCC', 'Weight_2dec_plus_Gender_minus_CALC', 'Weight_2dec_plus_CALC_minus_Gender', 'Gender_plus_CALC_minus_Weight_2dec', 'Weight_2dec_plus_Gender_plus_CALC', 'Weight_2dec_plus_Gender_minus_CAEC', 'Weight_2dec_plus_CAEC_minus_Gender', 'Gender_plus_CAEC_minus_Weight_2dec', 'Weight_2dec_plus_Gender_plus_CAEC', 'Weight_2dec_plus_Gender_minus_MTRANS', 'Weight_2dec_plus_MTRANS_minus_Gender', 'Gender_plus_MTRANS_minus_Weight_2dec', 'Weight_2dec_plus_Gender_plus_MTRANS', 'Weight_2dec_plus_Gender_minus_family_history_with_overweight', 'Weight_2dec_plus_family_history_with_overweight_minus_Gender', 'Gender_plus_family_history_with_overweight_minus_Weight_2dec', 'Weight_2dec_plus_Gender_plus_family_history_with_overweight', 'Weight_2dec_plus_Gender_minus_FAVC', 'Weight_2dec_plus_FAVC_minus_Gender', 'Gender_plus_FAVC_minus_Weight_2dec', 'Weight_2dec_plus_Gender_plus_FAVC', 'Weight_2dec_plus_Gender_minus_SMOKE', 'Weight_2dec_plus_SMOKE_minus_Gender', 'Gender_plus_SMOKE_minus_Weight_2dec', 'Weight_2dec_plus_Gender_plus_SMOKE', 'Weight_2dec_plus_Gender_minus_SCC', 'Weight_2dec_plus_SCC_minus_Gender', 'Gender_plus_SCC_minus_Weight_2dec', 'Weight_2dec_plus_Gender_plus_SCC', 'Weight_2dec_plus_CALC_minus_CAEC', 'Weight_2dec_plus_CAEC_minus_CALC', 'CALC_plus_CAEC_minus_Weight_2dec', 'Weight_2dec_plus_CALC_plus_CAEC', 'Weight_2dec_plus_CALC_minus_MTRANS', 'Weight_2dec_plus_MTRANS_minus_CALC', 'CALC_plus_MTRANS_minus_Weight_2dec', 'Weight_2dec_plus_CALC_plus_MTRANS', 'Weight_2dec_plus_CALC_minus_family_history_with_overweight', 'Weight_2dec_plus_family_history_with_overweight_minus_CALC', 'CALC_plus_family_history_with_overweight_minus_Weight_2dec', 'Weight_2dec_plus_CALC_plus_family_history_with_overweight', 'Weight_2dec_plus_CALC_minus_FAVC', 'Weight_2dec_plus_FAVC_minus_CALC', 'CALC_plus_FAVC_minus_Weight_2dec', 'Weight_2dec_plus_CALC_plus_FAVC', 'Weight_2dec_plus_CALC_minus_SMOKE', 'Weight_2dec_plus_SMOKE_minus_CALC', 'CALC_plus_SMOKE_minus_Weight_2dec', 'Weight_2dec_plus_CALC_plus_SMOKE', 'Weight_2dec_plus_CALC_minus_SCC', 'Weight_2dec_plus_SCC_minus_CALC', 'CALC_plus_SCC_minus_Weight_2dec', 'Weight_2dec_plus_CALC_plus_SCC', 'Weight_2dec_plus_CAEC_minus_MTRANS', 'Weight_2dec_plus_MTRANS_minus_CAEC', 'CAEC_plus_MTRANS_minus_Weight_2dec', 'Weight_2dec_plus_CAEC_plus_MTRANS', 'Weight_2dec_plus_CAEC_minus_family_history_with_overweight', 'Weight_2dec_plus_family_history_with_overweight_minus_CAEC', 'CAEC_plus_family_history_with_overweight_minus_Weight_2dec', 'Weight_2dec_plus_CAEC_plus_family_history_with_overweight', 'Weight_2dec_plus_CAEC_minus_FAVC', 'Weight_2dec_plus_FAVC_minus_CAEC', 'CAEC_plus_FAVC_minus_Weight_2dec', 'Weight_2dec_plus_CAEC_plus_FAVC', 'Weight_2dec_plus_CAEC_minus_SMOKE', 'Weight_2dec_plus_SMOKE_minus_CAEC', 'CAEC_plus_SMOKE_minus_Weight_2dec', 'Weight_2dec_plus_CAEC_plus_SMOKE', 'Weight_2dec_plus_CAEC_minus_SCC', 'Weight_2dec_plus_SCC_minus_CAEC', 'CAEC_plus_SCC_minus_Weight_2dec', 'Weight_2dec_plus_CAEC_plus_SCC', 'Weight_2dec_plus_MTRANS_minus_family_history_with_overweight', 'Weight_2dec_plus_family_history_with_overweight_minus_MTRANS', 'MTRANS_plus_family_history_with_overweight_minus_Weight_2dec', 'Weight_2dec_plus_MTRANS_plus_family_history_with_overweight', 'Weight_2dec_plus_MTRANS_minus_FAVC', 'Weight_2dec_plus_FAVC_minus_MTRANS', 'MTRANS_plus_FAVC_minus_Weight_2dec', 'Weight_2dec_plus_MTRANS_plus_FAVC', 'Weight_2dec_plus_MTRANS_minus_SMOKE', 'Weight_2dec_plus_SMOKE_minus_MTRANS', 'MTRANS_plus_SMOKE_minus_Weight_2dec', 'Weight_2dec_plus_MTRANS_plus_SMOKE', 'Weight_2dec_plus_MTRANS_minus_SCC', 'Weight_2dec_plus_SCC_minus_MTRANS', 'MTRANS_plus_SCC_minus_Weight_2dec', 'Weight_2dec_plus_MTRANS_plus_SCC', 'Weight_2dec_plus_family_history_with_overweight_minus_FAVC', 'Weight_2dec_plus_FAVC_minus_family_history_with_overweight', 'family_history_with_overweight_plus_FAVC_minus_Weight_2dec', 'Weight_2dec_plus_family_history_with_overweight_plus_FAVC', 'Weight_2dec_plus_family_history_with_overweight_minus_SMOKE', 'Weight_2dec_plus_SMOKE_minus_family_history_with_overweight', 'family_history_with_overweight_plus_SMOKE_minus_Weight_2dec', 'Weight_2dec_plus_family_history_with_overweight_plus_SMOKE', 'Weight_2dec_plus_family_history_with_overweight_minus_SCC', 'Weight_2dec_plus_SCC_minus_family_history_with_overweight', 'family_history_with_overweight_plus_SCC_minus_Weight_2dec', 'Weight_2dec_plus_family_history_with_overweight_plus_SCC', 'Weight_2dec_plus_FAVC_minus_SMOKE', 'Weight_2dec_plus_SMOKE_minus_FAVC', 'FAVC_plus_SMOKE_minus_Weight_2dec', 'Weight_2dec_plus_FAVC_plus_SMOKE', 'Weight_2dec_plus_FAVC_minus_SCC', 'Weight_2dec_plus_SCC_minus_FAVC', 'FAVC_plus_SCC_minus_Weight_2dec', 'Weight_2dec_plus_FAVC_plus_SCC', 'Weight_2dec_plus_SMOKE_minus_SCC', 'Weight_2dec_plus_SCC_minus_SMOKE', 'SMOKE_plus_SCC_minus_Weight_2dec', 'Weight_2dec_plus_SMOKE_plus_SCC', 'Height_2dec_plus_Gender_minus_CALC', 'Height_2dec_plus_CALC_minus_Gender', 'Gender_plus_CALC_minus_Height_2dec', 'Height_2dec_plus_Gender_plus_CALC', 'Height_2dec_plus_Gender_minus_CAEC', 'Height_2dec_plus_CAEC_minus_Gender', 'Gender_plus_CAEC_minus_Height_2dec', 'Height_2dec_plus_Gender_plus_CAEC', 'Height_2dec_plus_Gender_minus_MTRANS', 'Height_2dec_plus_Gender_plus_MTRANS', 'Height_2dec_plus_Gender_minus_family_history_with_overweight', 'Height_2dec_plus_family_history_with_overweight_minus_Gender', 'Gender_plus_family_history_with_overweight_minus_Height_2dec', 'Height_2dec_plus_Gender_plus_family_history_with_overweight', 'Height_2dec_plus_Gender_minus_FAVC', 'Height_2dec_plus_FAVC_minus_Gender', 'Gender_plus_FAVC_minus_Height_2dec', 'Height_2dec_plus_Gender_plus_FAVC', 'Height_2dec_plus_SMOKE_minus_Gender', 'Gender_plus_SMOKE_minus_Height_2dec', 'Height_2dec_plus_Gender_minus_SCC', 'Height_2dec_plus_SCC_minus_Gender', 'Height_2dec_plus_Gender_plus_SCC', 'Height_2dec_plus_CAEC_minus_CALC', 'Height_2dec_plus_CALC_plus_CAEC', 'Height_2dec_plus_MTRANS_minus_CALC', 'Height_2dec_plus_CALC_plus_MTRANS', 'Height_2dec_plus_CALC_minus_family_history_with_overweight', 'Height_2dec_plus_family_history_with_overweight_minus_CALC', 'CALC_plus_family_history_with_overweight_minus_Height_2dec', 'Height_2dec_plus_CALC_plus_family_history_with_overweight', 'Height_2dec_plus_FAVC_minus_CALC', 'Height_2dec_plus_CALC_plus_FAVC', 'Height_2dec_plus_CALC_minus_SMOKE', 'Height_2dec_plus_SMOKE_minus_CALC', 'CALC_plus_SMOKE_minus_Height_2dec', 'Height_2dec_plus_CALC_plus_SMOKE', 'Height_2dec_plus_SCC_minus_CALC', 'Height_2dec_plus_CALC_plus_SCC', 'Height_2dec_plus_CAEC_minus_MTRANS', 'Height_2dec_plus_MTRANS_minus_CAEC', 'CAEC_plus_MTRANS_minus_Height_2dec', 'Height_2dec_plus_CAEC_plus_MTRANS', 'Height_2dec_plus_family_history_with_overweight_minus_CAEC', 'Height_2dec_plus_CAEC_plus_family_history_with_overweight', 'Height_2dec_plus_FAVC_minus_CAEC', 'CAEC_plus_FAVC_minus_Height_2dec', 'Height_2dec_plus_CAEC_plus_FAVC', 'Height_2dec_plus_CAEC_minus_SMOKE', 'Height_2dec_plus_SMOKE_minus_CAEC', 'CAEC_plus_SMOKE_minus_Height_2dec', 'Height_2dec_plus_CAEC_plus_SMOKE', 'Height_2dec_plus_CAEC_minus_SCC', 'Height_2dec_plus_SCC_minus_CAEC', 'CAEC_plus_SCC_minus_Height_2dec', 'Height_2dec_plus_CAEC_plus_SCC', 'Height_2dec_plus_MTRANS_minus_family_history_with_overweight', 'Height_2dec_plus_family_history_with_overweight_minus_MTRANS', 'MTRANS_plus_family_history_with_overweight_minus_Height_2dec', 'Height_2dec_plus_MTRANS_plus_family_history_with_overweight', 'Height_2dec_plus_FAVC_minus_MTRANS', 'Height_2dec_plus_MTRANS_plus_FAVC', 'Height_2dec_plus_MTRANS_minus_SMOKE', 'Height_2dec_plus_SMOKE_minus_MTRANS', 'MTRANS_plus_SMOKE_minus_Height_2dec', 'Height_2dec_plus_MTRANS_plus_SMOKE', 'Height_2dec_plus_SCC_minus_MTRANS', 'Height_2dec_plus_MTRANS_plus_SCC', 'Height_2dec_plus_family_history_with_overweight_minus_FAVC', 'Height_2dec_plus_FAVC_minus_family_history_with_overweight', 'family_history_with_overweight_plus_FAVC_minus_Height_2dec', 'Height_2dec_plus_family_history_with_overweight_plus_FAVC', 'Height_2dec_plus_family_history_with_overweight_minus_SMOKE', 'Height_2dec_plus_SMOKE_minus_family_history_with_overweight', 'family_history_with_overweight_plus_SMOKE_minus_Height_2dec', 'Height_2dec_plus_family_history_with_overweight_plus_SMOKE', 'Height_2dec_plus_family_history_with_overweight_minus_SCC', 'Height_2dec_plus_SCC_minus_family_history_with_overweight', 'family_history_with_overweight_plus_SCC_minus_Height_2dec', 'Height_2dec_plus_family_history_with_overweight_plus_SCC', 'Height_2dec_plus_FAVC_minus_SMOKE', 'Height_2dec_plus_SMOKE_minus_FAVC', 'FAVC_plus_SMOKE_minus_Height_2dec', 'Height_2dec_plus_FAVC_plus_SMOKE', 'Height_2dec_plus_SCC_minus_FAVC', 'Height_2dec_plus_FAVC_plus_SCC', 'Height_2dec_plus_SMOKE_minus_SCC', 'Height_2dec_plus_SCC_minus_SMOKE', 'SMOKE_plus_SCC_minus_Height_2dec', 'Height_2dec_plus_SMOKE_plus_SCC', 'Gender_plus_CALC_minus_CAEC', 'Gender_plus_CAEC_minus_CALC', 'CALC_plus_CAEC_minus_Gender', 'Gender_plus_CALC_plus_CAEC', 'Gender_plus_CALC_minus_MTRANS', 'Gender_plus_MTRANS_minus_CALC', 'CALC_plus_MTRANS_minus_Gender', 'Gender_plus_CALC_plus_MTRANS', 'Gender_plus_CALC_minus_family_history_with_overweight', 'Gender_plus_family_history_with_overweight_minus_CALC', 'CALC_plus_family_history_with_overweight_minus_Gender', 'Gender_plus_CALC_plus_family_history_with_overweight', 'Gender_plus_CALC_minus_FAVC', 'Gender_plus_FAVC_minus_CALC', 'CALC_plus_FAVC_minus_Gender', 'Gender_plus_CALC_plus_FAVC', 'Gender_plus_CALC_minus_SMOKE', 'Gender_plus_SMOKE_minus_CALC', 'CALC_plus_SMOKE_minus_Gender', 'Gender_plus_CALC_plus_SMOKE', 'Gender_plus_CALC_minus_SCC', 'Gender_plus_SCC_minus_CALC', 'CALC_plus_SCC_minus_Gender', 'Gender_plus_CALC_plus_SCC', 'Gender_plus_CAEC_minus_MTRANS', 'Gender_plus_MTRANS_minus_CAEC', 'CAEC_plus_MTRANS_minus_Gender', 'Gender_plus_CAEC_plus_MTRANS', 'Gender_plus_CAEC_minus_family_history_with_overweight', 'Gender_plus_family_history_with_overweight_minus_CAEC', 'CAEC_plus_family_history_with_overweight_minus_Gender', 'Gender_plus_CAEC_plus_family_history_with_overweight', 'Gender_plus_CAEC_minus_FAVC', 'Gender_plus_FAVC_minus_CAEC', 'CAEC_plus_FAVC_minus_Gender', 'Gender_plus_CAEC_plus_FAVC', 'Gender_plus_CAEC_minus_SMOKE', 'Gender_plus_SMOKE_minus_CAEC', 'CAEC_plus_SMOKE_minus_Gender', 'Gender_plus_CAEC_plus_SMOKE', 'Gender_plus_CAEC_minus_SCC', 'Gender_plus_SCC_minus_CAEC', 'CAEC_plus_SCC_minus_Gender', 'Gender_plus_CAEC_plus_SCC', 'Gender_plus_MTRANS_minus_family_history_with_overweight', 'Gender_plus_family_history_with_overweight_minus_MTRANS', 'MTRANS_plus_family_history_with_overweight_minus_Gender', 'Gender_plus_MTRANS_plus_family_history_with_overweight', 'Gender_plus_MTRANS_minus_FAVC', 'Gender_plus_FAVC_minus_MTRANS', 'MTRANS_plus_FAVC_minus_Gender', 'Gender_plus_MTRANS_plus_FAVC', 'Gender_plus_MTRANS_minus_SMOKE', 'Gender_plus_SMOKE_minus_MTRANS', 'MTRANS_plus_SMOKE_minus_Gender', 'Gender_plus_MTRANS_plus_SMOKE', 'Gender_plus_MTRANS_minus_SCC', 'Gender_plus_SCC_minus_MTRANS', 'MTRANS_plus_SCC_minus_Gender', 'Gender_plus_MTRANS_plus_SCC', 'Gender_plus_family_history_with_overweight_minus_FAVC', 'Gender_plus_FAVC_minus_family_history_with_overweight', 'family_history_with_overweight_plus_FAVC_minus_Gender', 'Gender_plus_family_history_with_overweight_plus_FAVC', 'Gender_plus_family_history_with_overweight_minus_SMOKE', 'Gender_plus_SMOKE_minus_family_history_with_overweight', 'family_history_with_overweight_plus_SMOKE_minus_Gender', 'Gender_plus_family_history_with_overweight_plus_SMOKE', 'Gender_plus_family_history_with_overweight_minus_SCC', 'Gender_plus_SCC_minus_family_history_with_overweight', 'family_history_with_overweight_plus_SCC_minus_Gender', 'Gender_plus_family_history_with_overweight_plus_SCC', 'Gender_plus_FAVC_minus_SMOKE', 'Gender_plus_SMOKE_minus_FAVC', 'FAVC_plus_SMOKE_minus_Gender', 'Gender_plus_FAVC_plus_SMOKE', 'Gender_plus_FAVC_minus_SCC', 'Gender_plus_SCC_minus_FAVC', 'FAVC_plus_SCC_minus_Gender', 'Gender_plus_FAVC_plus_SCC', 'Gender_plus_SMOKE_minus_SCC', 'Gender_plus_SCC_minus_SMOKE', 'SMOKE_plus_SCC_minus_Gender', 'Gender_plus_SMOKE_plus_SCC', 'CALC_plus_CAEC_minus_MTRANS', 'CALC_plus_MTRANS_minus_CAEC', 'CAEC_plus_MTRANS_minus_CALC', 'CALC_plus_CAEC_plus_MTRANS', 'CALC_plus_CAEC_minus_family_history_with_overweight', 'CALC_plus_family_history_with_overweight_minus_CAEC', 'CAEC_plus_family_history_with_overweight_minus_CALC', 'CALC_plus_CAEC_plus_family_history_with_overweight', 'CALC_plus_CAEC_minus_FAVC', 'CALC_plus_FAVC_minus_CAEC', 'CAEC_plus_FAVC_minus_CALC', 'CALC_plus_CAEC_plus_FAVC', 'CALC_plus_CAEC_minus_SMOKE', 'CALC_plus_SMOKE_minus_CAEC', 'CAEC_plus_SMOKE_minus_CALC', 'CALC_plus_CAEC_plus_SMOKE', 'CALC_plus_CAEC_minus_SCC', 'CALC_plus_SCC_minus_CAEC', 'CAEC_plus_SCC_minus_CALC', 'CALC_plus_CAEC_plus_SCC', 'CALC_plus_MTRANS_minus_family_history_with_overweight', 'CALC_plus_family_history_with_overweight_minus_MTRANS', 'MTRANS_plus_family_history_with_overweight_minus_CALC', 'CALC_plus_MTRANS_plus_family_history_with_overweight', 'CALC_plus_MTRANS_minus_FAVC', 'CALC_plus_FAVC_minus_MTRANS', 'MTRANS_plus_FAVC_minus_CALC', 'CALC_plus_MTRANS_plus_FAVC', 'CALC_plus_MTRANS_minus_SMOKE', 'CALC_plus_SMOKE_minus_MTRANS', 'MTRANS_plus_SMOKE_minus_CALC', 'CALC_plus_MTRANS_plus_SMOKE', 'CALC_plus_MTRANS_minus_SCC', 'CALC_plus_SCC_minus_MTRANS', 'MTRANS_plus_SCC_minus_CALC', 'CALC_plus_MTRANS_plus_SCC', 'CALC_plus_family_history_with_overweight_minus_FAVC', 'CALC_plus_FAVC_minus_family_history_with_overweight', 'family_history_with_overweight_plus_FAVC_minus_CALC', 'CALC_plus_family_history_with_overweight_plus_FAVC', 'CALC_plus_family_history_with_overweight_minus_SMOKE', 'CALC_plus_SMOKE_minus_family_history_with_overweight', 'family_history_with_overweight_plus_SMOKE_minus_CALC', 'CALC_plus_family_history_with_overweight_plus_SMOKE', 'CALC_plus_family_history_with_overweight_minus_SCC', 'CALC_plus_SCC_minus_family_history_with_overweight', 'family_history_with_overweight_plus_SCC_minus_CALC', 'CALC_plus_family_history_with_overweight_plus_SCC', 'CALC_plus_FAVC_minus_SMOKE', 'CALC_plus_SMOKE_minus_FAVC', 'FAVC_plus_SMOKE_minus_CALC', 'CALC_plus_FAVC_plus_SMOKE', 'CALC_plus_SMOKE_minus_SCC', 'CALC_plus_SCC_minus_SMOKE', 'SMOKE_plus_SCC_minus_CALC', 'CALC_plus_SMOKE_plus_SCC', 'CAEC_plus_MTRANS_minus_family_history_with_overweight', 'CAEC_plus_family_history_with_overweight_minus_MTRANS', 'MTRANS_plus_family_history_with_overweight_minus_CAEC', 'CAEC_plus_MTRANS_plus_family_history_with_overweight', 'CAEC_plus_MTRANS_minus_FAVC', 'CAEC_plus_FAVC_minus_MTRANS', 'MTRANS_plus_FAVC_minus_CAEC', 'CAEC_plus_MTRANS_plus_FAVC', 'CAEC_plus_MTRANS_minus_SMOKE', 'CAEC_plus_SMOKE_minus_MTRANS', 'MTRANS_plus_SMOKE_minus_CAEC', 'CAEC_plus_MTRANS_plus_SMOKE', 'CAEC_plus_MTRANS_minus_SCC', 'CAEC_plus_SCC_minus_MTRANS', 'MTRANS_plus_SCC_minus_CAEC', 'CAEC_plus_MTRANS_plus_SCC', 'CAEC_plus_family_history_with_overweight_minus_FAVC', 'CAEC_plus_FAVC_minus_family_history_with_overweight', 'family_history_with_overweight_plus_FAVC_minus_CAEC', 'CAEC_plus_family_history_with_overweight_plus_FAVC', 'CAEC_plus_family_history_with_overweight_minus_SMOKE', 'CAEC_plus_SMOKE_minus_family_history_with_overweight', 'family_history_with_overweight_plus_SMOKE_minus_CAEC', 'CAEC_plus_family_history_with_overweight_plus_SMOKE', 'CAEC_plus_family_history_with_overweight_minus_SCC', 'CAEC_plus_SCC_minus_family_history_with_overweight', 'family_history_with_overweight_plus_SCC_minus_CAEC', 'CAEC_plus_family_history_with_overweight_plus_SCC', 'CAEC_plus_FAVC_minus_SMOKE', 'CAEC_plus_SMOKE_minus_FAVC', 'FAVC_plus_SMOKE_minus_CAEC', 'CAEC_plus_FAVC_plus_SMOKE', 'CAEC_plus_FAVC_minus_SCC', 'CAEC_plus_SCC_minus_FAVC', 'FAVC_plus_SCC_minus_CAEC', 'CAEC_plus_FAVC_plus_SCC', 'CAEC_plus_SMOKE_minus_SCC', 'CAEC_plus_SCC_minus_SMOKE', 'SMOKE_plus_SCC_minus_CAEC', 'CAEC_plus_SMOKE_plus_SCC', 'MTRANS_plus_family_history_with_overweight_minus_FAVC', 'MTRANS_plus_FAVC_minus_family_history_with_overweight', 'family_history_with_overweight_plus_FAVC_minus_MTRANS', 'MTRANS_plus_family_history_with_overweight_plus_FAVC', 'MTRANS_plus_family_history_with_overweight_minus_SMOKE', 'MTRANS_plus_SMOKE_minus_family_history_with_overweight', 'family_history_with_overweight_plus_SMOKE_minus_MTRANS', 'MTRANS_plus_family_history_with_overweight_plus_SMOKE', 'MTRANS_plus_family_history_with_overweight_minus_SCC', 'MTRANS_plus_SCC_minus_family_history_with_overweight', 'family_history_with_overweight_plus_SCC_minus_MTRANS', 'MTRANS_plus_family_history_with_overweight_plus_SCC', 'MTRANS_plus_FAVC_minus_SMOKE', 'MTRANS_plus_SMOKE_minus_FAVC', 'FAVC_plus_SMOKE_minus_MTRANS', 'MTRANS_plus_FAVC_plus_SMOKE', 'MTRANS_plus_FAVC_minus_SCC', 'FAVC_plus_SCC_minus_MTRANS', 'MTRANS_plus_FAVC_plus_SCC', 'MTRANS_plus_SMOKE_minus_SCC', 'MTRANS_plus_SCC_minus_SMOKE', 'SMOKE_plus_SCC_minus_MTRANS', 'MTRANS_plus_SMOKE_plus_SCC', 'family_history_with_overweight_plus_FAVC_minus_SMOKE', 'family_history_with_overweight_plus_SMOKE_minus_FAVC', 'FAVC_plus_SMOKE_minus_family_history_with_overweight', 'family_history_with_overweight_plus_FAVC_plus_SMOKE', 'family_history_with_overweight_plus_FAVC_minus_SCC', 'family_history_with_overweight_plus_SCC_minus_FAVC', 'FAVC_plus_SCC_minus_family_history_with_overweight', 'family_history_with_overweight_plus_FAVC_plus_SCC', 'family_history_with_overweight_plus_SMOKE_minus_SCC', 'family_history_with_overweight_plus_SCC_minus_SMOKE', 'SMOKE_plus_SCC_minus_family_history_with_overweight', 'family_history_with_overweight_plus_SMOKE_plus_SCC', 'FAVC_plus_SMOKE_minus_SCC', 'FAVC_plus_SCC_minus_SMOKE', 'SMOKE_plus_SCC_minus_FAVC', 'FAVC_plus_SMOKE_plus_SCC'
,'NCP_round', 'Age_round', 'Age_ceil_round', 'Public_Transportation', 'Gender', 'MTRANS', 'family_history_with_overweight', 'FAVC', 'SMOKE', 'SCC', 'Weight_2dec_plus_Height_2dec_sub_by_Gender', 'Weight_2dec_plus_Gender_sub_by_Height_2dec', 'Height_2dec_plus_Gender_sub_by_Weight_2dec', 'Weight_2dec_plus_Height_2dec_sub_by_CALC', 'Weight_2dec_plus_CALC_sub_by_Height_2dec', 'Height_2dec_plus_CALC_sub_by_Weight_2dec', 'Weight_2dec_plus_Height_2dec_sub_by_CAEC', 'Weight_2dec_plus_CAEC_sub_by_Height_2dec', 'Height_2dec_plus_CAEC_sub_by_Weight_2dec', 'Weight_2dec_plus_Height_2dec_sub_by_MTRANS', 'Weight_2dec_plus_MTRANS_sub_by_Height_2dec', 'Height_2dec_plus_MTRANS_sub_by_Weight_2dec', 'Weight_2dec_plus_Height_2dec_sub_by_family_history_with_overweight', 'Weight_2dec_plus_family_history_with_overweight_sub_by_Height_2dec', 'Height_2dec_plus_family_history_with_overweight_sub_by_Weight_2dec', 'Weight_2dec_plus_Height_2dec_sub_by_FAVC', 'Weight_2dec_plus_FAVC_sub_by_Height_2dec', 'Height_2dec_plus_FAVC_sub_by_Weight_2dec', 'Weight_2dec_plus_Height_2dec_sub_by_SMOKE', 'Weight_2dec_plus_SMOKE_sub_by_Height_2dec', 'Height_2dec_plus_SMOKE_sub_by_Weight_2dec', 'Weight_2dec_plus_Height_2dec_sub_by_SCC', 'Weight_2dec_plus_SCC_sub_by_Height_2dec', 'Height_2dec_plus_SCC_sub_by_Weight_2dec', 'Weight_2dec_plus_Gender_sub_by_CALC', 'Weight_2dec_plus_CALC_sub_by_Gender', 'Gender_plus_CALC_sub_by_Weight_2dec', 'Weight_2dec_plus_Gender_sub_by_CAEC', 'Weight_2dec_plus_CAEC_sub_by_Gender', 'Gender_plus_CAEC_sub_by_Weight_2dec', 'Weight_2dec_plus_Gender_sub_by_MTRANS', 'Weight_2dec_plus_MTRANS_sub_by_Gender', 'Gender_plus_MTRANS_sub_by_Weight_2dec', 'Weight_2dec_plus_Gender_sub_by_family_history_with_overweight', 'Weight_2dec_plus_family_history_with_overweight_sub_by_Gender', 'Gender_plus_family_history_with_overweight_sub_by_Weight_2dec', 'Weight_2dec_plus_Gender_sub_by_FAVC', 'Weight_2dec_plus_FAVC_sub_by_Gender', 'Gender_plus_FAVC_sub_by_Weight_2dec', 'Weight_2dec_plus_Gender_sub_by_SMOKE', 'Weight_2dec_plus_SMOKE_sub_by_Gender', 'Gender_plus_SMOKE_sub_by_Weight_2dec', 'Weight_2dec_plus_Gender_sub_by_SCC', 'Weight_2dec_plus_SCC_sub_by_Gender', 'Gender_plus_SCC_sub_by_Weight_2dec', 'Weight_2dec_plus_CALC_sub_by_CAEC', 'Weight_2dec_plus_CAEC_sub_by_CALC', 'CALC_plus_CAEC_sub_by_Weight_2dec', 'Weight_2dec_plus_CALC_sub_by_MTRANS', 'Weight_2dec_plus_MTRANS_sub_by_CALC', 'CALC_plus_MTRANS_sub_by_Weight_2dec', 'Weight_2dec_plus_CALC_sub_by_family_history_with_overweight', 'Weight_2dec_plus_family_history_with_overweight_sub_by_CALC', 'CALC_plus_family_history_with_overweight_sub_by_Weight_2dec', 'Weight_2dec_plus_CALC_sub_by_FAVC', 'Weight_2dec_plus_FAVC_sub_by_CALC', 'CALC_plus_FAVC_sub_by_Weight_2dec', 'Weight_2dec_plus_CALC_sub_by_SMOKE', 'Weight_2dec_plus_SMOKE_sub_by_CALC', 'CALC_plus_SMOKE_sub_by_Weight_2dec', 'Weight_2dec_plus_CALC_sub_by_SCC', 'Weight_2dec_plus_SCC_sub_by_CALC', 'CALC_plus_SCC_sub_by_Weight_2dec', 'Weight_2dec_plus_CAEC_sub_by_MTRANS', 'Weight_2dec_plus_MTRANS_sub_by_CAEC', 'CAEC_plus_MTRANS_sub_by_Weight_2dec', 'Weight_2dec_plus_CAEC_sub_by_family_history_with_overweight', 'Weight_2dec_plus_family_history_with_overweight_sub_by_CAEC', 'CAEC_plus_family_history_with_overweight_sub_by_Weight_2dec', 'Weight_2dec_plus_CAEC_sub_by_FAVC', 'Weight_2dec_plus_FAVC_sub_by_CAEC', 'CAEC_plus_FAVC_sub_by_Weight_2dec', 'Weight_2dec_plus_CAEC_sub_by_SMOKE', 'Weight_2dec_plus_SMOKE_sub_by_CAEC', 'CAEC_plus_SMOKE_sub_by_Weight_2dec', 'Weight_2dec_plus_CAEC_sub_by_SCC', 'Weight_2dec_plus_SCC_sub_by_CAEC', 'CAEC_plus_SCC_sub_by_Weight_2dec', 'Weight_2dec_plus_MTRANS_sub_by_family_history_with_overweight', 'Weight_2dec_plus_family_history_with_overweight_sub_by_MTRANS', 'MTRANS_plus_family_history_with_overweight_sub_by_Weight_2dec', 'Weight_2dec_plus_MTRANS_sub_by_FAVC', 'Weight_2dec_plus_FAVC_sub_by_MTRANS', 'MTRANS_plus_FAVC_sub_by_Weight_2dec', 'Weight_2dec_plus_MTRANS_sub_by_SMOKE', 'Weight_2dec_plus_SMOKE_sub_by_MTRANS', 'MTRANS_plus_SMOKE_sub_by_Weight_2dec', 'Weight_2dec_plus_MTRANS_sub_by_SCC', 'Weight_2dec_plus_SCC_sub_by_MTRANS', 'MTRANS_plus_SCC_sub_by_Weight_2dec', 'Weight_2dec_plus_family_history_with_overweight_sub_by_FAVC', 'Weight_2dec_plus_FAVC_sub_by_family_history_with_overweight', 'family_history_with_overweight_plus_FAVC_sub_by_Weight_2dec', 'Weight_2dec_plus_family_history_with_overweight_sub_by_SMOKE', 'Weight_2dec_plus_SMOKE_sub_by_family_history_with_overweight', 'family_history_with_overweight_plus_SMOKE_sub_by_Weight_2dec', 'Weight_2dec_plus_family_history_with_overweight_sub_by_SCC', 'Weight_2dec_plus_SCC_sub_by_family_history_with_overweight', 'family_history_with_overweight_plus_SCC_sub_by_Weight_2dec', 'Weight_2dec_plus_FAVC_sub_by_SMOKE', 'Weight_2dec_plus_SMOKE_sub_by_FAVC', 'FAVC_plus_SMOKE_sub_by_Weight_2dec', 'Weight_2dec_plus_FAVC_sub_by_SCC', 'Weight_2dec_plus_SCC_sub_by_FAVC', 'FAVC_plus_SCC_sub_by_Weight_2dec', 'Weight_2dec_plus_SMOKE_sub_by_SCC', 'Weight_2dec_plus_SCC_sub_by_SMOKE', 'SMOKE_plus_SCC_sub_by_Weight_2dec', 'Height_2dec_plus_Gender_sub_by_CALC', 'Height_2dec_plus_CALC_sub_by_Gender', 'Gender_plus_CALC_sub_by_Height_2dec', 'Height_2dec_plus_Gender_sub_by_CAEC', 'Height_2dec_plus_CAEC_sub_by_Gender', 'Gender_plus_CAEC_sub_by_Height_2dec', 'Height_2dec_plus_Gender_sub_by_MTRANS', 'Height_2dec_plus_Gender_sub_by_family_history_with_overweight', 'Height_2dec_plus_family_history_with_overweight_sub_by_Gender', 'Gender_plus_family_history_with_overweight_sub_by_Height_2dec', 'Height_2dec_plus_Gender_sub_by_FAVC', 'Height_2dec_plus_FAVC_sub_by_Gender', 'Gender_plus_FAVC_sub_by_Height_2dec', 'Height_2dec_plus_SMOKE_sub_by_Gender', 'Gender_plus_SMOKE_sub_by_Height_2dec', 'Height_2dec_plus_Gender_sub_by_SCC', 'Height_2dec_plus_SCC_sub_by_Gender', 'Height_2dec_plus_CAEC_sub_by_CALC', 'Height_2dec_plus_MTRANS_sub_by_CALC', 'Height_2dec_plus_CALC_sub_by_family_history_with_overweight', 'Height_2dec_plus_family_history_with_overweight_sub_by_CALC', 'CALC_plus_family_history_with_overweight_sub_by_Height_2dec', 'Height_2dec_plus_FAVC_sub_by_CALC', 'Height_2dec_plus_CALC_sub_by_SMOKE', 'Height_2dec_plus_SMOKE_sub_by_CALC', 'CALC_plus_SMOKE_sub_by_Height_2dec', 'Height_2dec_plus_SCC_sub_by_CALC', 'Height_2dec_plus_CAEC_sub_by_MTRANS', 'Height_2dec_plus_MTRANS_sub_by_CAEC', 'CAEC_plus_MTRANS_sub_by_Height_2dec', 'Height_2dec_plus_family_history_with_overweight_sub_by_CAEC', 'Height_2dec_plus_FAVC_sub_by_CAEC', 'CAEC_plus_FAVC_sub_by_Height_2dec', 'Height_2dec_plus_CAEC_sub_by_SMOKE', 'Height_2dec_plus_SMOKE_sub_by_CAEC', 'CAEC_plus_SMOKE_sub_by_Height_2dec', 'Height_2dec_plus_CAEC_sub_by_SCC', 'Height_2dec_plus_SCC_sub_by_CAEC', 'CAEC_plus_SCC_sub_by_Height_2dec', 'Height_2dec_plus_MTRANS_sub_by_family_history_with_overweight', 'Height_2dec_plus_family_history_with_overweight_sub_by_MTRANS', 'MTRANS_plus_family_history_with_overweight_sub_by_Height_2dec', 'Height_2dec_plus_FAVC_sub_by_MTRANS', 'Height_2dec_plus_MTRANS_sub_by_SMOKE', 'Height_2dec_plus_SMOKE_sub_by_MTRANS', 'MTRANS_plus_SMOKE_sub_by_Height_2dec', 'Height_2dec_plus_SCC_sub_by_MTRANS', 'Height_2dec_plus_family_history_with_overweight_sub_by_FAVC', 'Height_2dec_plus_FAVC_sub_by_family_history_with_overweight', 'family_history_with_overweight_plus_FAVC_sub_by_Height_2dec', 'Height_2dec_plus_family_history_with_overweight_sub_by_SMOKE', 'Height_2dec_plus_SMOKE_sub_by_family_history_with_overweight', 'family_history_with_overweight_plus_SMOKE_sub_by_Height_2dec', 'Height_2dec_plus_family_history_with_overweight_sub_by_SCC', 'Height_2dec_plus_SCC_sub_by_family_history_with_overweight', 'family_history_with_overweight_plus_SCC_sub_by_Height_2dec', 'Height_2dec_plus_FAVC_sub_by_SMOKE', 'Height_2dec_plus_SMOKE_sub_by_FAVC', 'FAVC_plus_SMOKE_sub_by_Height_2dec', 'Height_2dec_plus_SCC_sub_by_FAVC', 'Height_2dec_plus_SMOKE_sub_by_SCC', 'Height_2dec_plus_SCC_sub_by_SMOKE', 'SMOKE_plus_SCC_sub_by_Height_2dec', 'Gender_plus_CALC_sub_by_CAEC', 'Gender_plus_CAEC_sub_by_CALC', 'CALC_plus_CAEC_sub_by_Gender', 'Gender_plus_CALC_sub_by_MTRANS', 'Gender_plus_MTRANS_sub_by_CALC', 'CALC_plus_MTRANS_sub_by_Gender', 'Gender_plus_CALC_sub_by_family_history_with_overweight', 'Gender_plus_family_history_with_overweight_sub_by_CALC', 'CALC_plus_family_history_with_overweight_sub_by_Gender', 'Gender_plus_CALC_sub_by_FAVC', 'Gender_plus_FAVC_sub_by_CALC', 'CALC_plus_FAVC_sub_by_Gender', 'Gender_plus_CALC_sub_by_SMOKE', 'Gender_plus_SMOKE_sub_by_CALC', 'CALC_plus_SMOKE_sub_by_Gender', 'Gender_plus_CALC_sub_by_SCC', 'Gender_plus_SCC_sub_by_CALC', 'CALC_plus_SCC_sub_by_Gender', 'Gender_plus_CAEC_sub_by_MTRANS', 'Gender_plus_MTRANS_sub_by_CAEC', 'CAEC_plus_MTRANS_sub_by_Gender', 'Gender_plus_CAEC_sub_by_family_history_with_overweight', 'Gender_plus_family_history_with_overweight_sub_by_CAEC', 'CAEC_plus_family_history_with_overweight_sub_by_Gender', 'Gender_plus_CAEC_sub_by_FAVC', 'Gender_plus_FAVC_sub_by_CAEC', 'CAEC_plus_FAVC_sub_by_Gender', 'Gender_plus_CAEC_sub_by_SMOKE', 'Gender_plus_SMOKE_sub_by_CAEC', 'CAEC_plus_SMOKE_sub_by_Gender', 'Gender_plus_CAEC_sub_by_SCC', 'Gender_plus_SCC_sub_by_CAEC', 'CAEC_plus_SCC_sub_by_Gender', 'Gender_plus_MTRANS_sub_by_family_history_with_overweight', 'Gender_plus_family_history_with_overweight_sub_by_MTRANS', 'MTRANS_plus_family_history_with_overweight_sub_by_Gender', 'Gender_plus_MTRANS_sub_by_FAVC', 'Gender_plus_FAVC_sub_by_MTRANS', 'MTRANS_plus_FAVC_sub_by_Gender', 'Gender_plus_MTRANS_sub_by_SMOKE', 'Gender_plus_SMOKE_sub_by_MTRANS', 'MTRANS_plus_SMOKE_sub_by_Gender', 'Gender_plus_MTRANS_sub_by_SCC', 'Gender_plus_SCC_sub_by_MTRANS', 'MTRANS_plus_SCC_sub_by_Gender', 'Gender_plus_family_history_with_overweight_sub_by_FAVC', 'Gender_plus_FAVC_sub_by_family_history_with_overweight', 'family_history_with_overweight_plus_FAVC_sub_by_Gender', 'Gender_plus_family_history_with_overweight_sub_by_SMOKE', 'Gender_plus_SMOKE_sub_by_family_history_with_overweight', 'family_history_with_overweight_plus_SMOKE_sub_by_Gender', 'Gender_plus_family_history_with_overweight_sub_by_SCC', 'Gender_plus_SCC_sub_by_family_history_with_overweight', 'family_history_with_overweight_plus_SCC_sub_by_Gender', 'Gender_plus_FAVC_sub_by_SMOKE', 'Gender_plus_SMOKE_sub_by_FAVC', 'FAVC_plus_SMOKE_sub_by_Gender', 'Gender_plus_FAVC_sub_by_SCC', 'Gender_plus_SCC_sub_by_FAVC', 'FAVC_plus_SCC_sub_by_Gender', 'Gender_plus_SMOKE_sub_by_SCC', 'Gender_plus_SCC_sub_by_SMOKE', 'SMOKE_plus_SCC_sub_by_Gender', 'CALC_plus_CAEC_sub_by_MTRANS', 'CALC_plus_MTRANS_sub_by_CAEC', 'CAEC_plus_MTRANS_sub_by_CALC', 'CALC_plus_CAEC_sub_by_family_history_with_overweight', 'CALC_plus_family_history_with_overweight_sub_by_CAEC', 'CAEC_plus_family_history_with_overweight_sub_by_CALC', 'CALC_plus_CAEC_sub_by_FAVC', 'CALC_plus_FAVC_sub_by_CAEC', 'CAEC_plus_FAVC_sub_by_CALC', 'CALC_plus_CAEC_sub_by_SMOKE', 'CALC_plus_SMOKE_sub_by_CAEC', 'CAEC_plus_SMOKE_sub_by_CALC', 'CALC_plus_CAEC_sub_by_SCC', 'CALC_plus_SCC_sub_by_CAEC', 'CAEC_plus_SCC_sub_by_CALC', 'CALC_plus_MTRANS_sub_by_family_history_with_overweight', 'CALC_plus_family_history_with_overweight_sub_by_MTRANS', 'MTRANS_plus_family_history_with_overweight_sub_by_CALC', 'CALC_plus_MTRANS_sub_by_FAVC', 'CALC_plus_FAVC_sub_by_MTRANS', 'MTRANS_plus_FAVC_sub_by_CALC', 'CALC_plus_MTRANS_sub_by_SMOKE', 'CALC_plus_SMOKE_sub_by_MTRANS', 'MTRANS_plus_SMOKE_sub_by_CALC', 'CALC_plus_MTRANS_sub_by_SCC', 'CALC_plus_SCC_sub_by_MTRANS', 'MTRANS_plus_SCC_sub_by_CALC', 'CALC_plus_family_history_with_overweight_sub_by_FAVC', 'CALC_plus_FAVC_sub_by_family_history_with_overweight', 'family_history_with_overweight_plus_FAVC_sub_by_CALC', 'CALC_plus_family_history_with_overweight_sub_by_SMOKE', 'CALC_plus_SMOKE_sub_by_family_history_with_overweight', 'family_history_with_overweight_plus_SMOKE_sub_by_CALC', 'CALC_plus_family_history_with_overweight_sub_by_SCC', 'CALC_plus_SCC_sub_by_family_history_with_overweight', 'family_history_with_overweight_plus_SCC_sub_by_CALC', 'CALC_plus_FAVC_sub_by_SMOKE', 'CALC_plus_SMOKE_sub_by_FAVC', 'FAVC_plus_SMOKE_sub_by_CALC', 'CALC_plus_SMOKE_sub_by_SCC', 'CALC_plus_SCC_sub_by_SMOKE', 'SMOKE_plus_SCC_sub_by_CALC', 'CAEC_plus_MTRANS_sub_by_family_history_with_overweight', 'CAEC_plus_family_history_with_overweight_sub_by_MTRANS', 'MTRANS_plus_family_history_with_overweight_sub_by_CAEC', 'CAEC_plus_MTRANS_sub_by_FAVC', 'CAEC_plus_FAVC_sub_by_MTRANS', 'MTRANS_plus_FAVC_sub_by_CAEC', 'CAEC_plus_MTRANS_sub_by_SMOKE', 'CAEC_plus_SMOKE_sub_by_MTRANS', 'MTRANS_plus_SMOKE_sub_by_CAEC', 'CAEC_plus_MTRANS_sub_by_SCC', 'CAEC_plus_SCC_sub_by_MTRANS', 'MTRANS_plus_SCC_sub_by_CAEC', 'CAEC_plus_family_history_with_overweight_sub_by_FAVC', 'CAEC_plus_FAVC_sub_by_family_history_with_overweight', 'family_history_with_overweight_plus_FAVC_sub_by_CAEC', 'CAEC_plus_family_history_with_overweight_sub_by_SMOKE', 'CAEC_plus_SMOKE_sub_by_family_history_with_overweight', 'family_history_with_overweight_plus_SMOKE_sub_by_CAEC', 'CAEC_plus_family_history_with_overweight_sub_by_SCC', 'CAEC_plus_SCC_sub_by_family_history_with_overweight', 'family_history_with_overweight_plus_SCC_sub_by_CAEC', 'CAEC_plus_FAVC_sub_by_SMOKE', 'CAEC_plus_SMOKE_sub_by_FAVC', 'FAVC_plus_SMOKE_sub_by_CAEC', 'CAEC_plus_FAVC_sub_by_SCC', 'CAEC_plus_SCC_sub_by_FAVC', 'FAVC_plus_SCC_sub_by_CAEC', 'CAEC_plus_SMOKE_sub_by_SCC', 'CAEC_plus_SCC_sub_by_SMOKE', 'SMOKE_plus_SCC_sub_by_CAEC', 'MTRANS_plus_family_history_with_overweight_sub_by_FAVC', 'MTRANS_plus_FAVC_sub_by_family_history_with_overweight', 'family_history_with_overweight_plus_FAVC_sub_by_MTRANS', 'MTRANS_plus_family_history_with_overweight_sub_by_SMOKE', 'MTRANS_plus_SMOKE_sub_by_family_history_with_overweight', 'family_history_with_overweight_plus_SMOKE_sub_by_MTRANS', 'MTRANS_plus_family_history_with_overweight_sub_by_SCC', 'MTRANS_plus_SCC_sub_by_family_history_with_overweight', 'family_history_with_overweight_plus_SCC_sub_by_MTRANS', 'MTRANS_plus_FAVC_sub_by_SMOKE', 'MTRANS_plus_SMOKE_sub_by_FAVC', 'FAVC_plus_SMOKE_sub_by_MTRANS', 'MTRANS_plus_FAVC_sub_by_SCC', 'FAVC_plus_SCC_sub_by_MTRANS', 'MTRANS_plus_SMOKE_sub_by_SCC', 'MTRANS_plus_SCC_sub_by_SMOKE', 'SMOKE_plus_SCC_sub_by_MTRANS', 'family_history_with_overweight_plus_FAVC_sub_by_SMOKE', 'family_history_with_overweight_plus_SMOKE_sub_by_FAVC', 'FAVC_plus_SMOKE_sub_by_family_history_with_overweight', 'family_history_with_overweight_plus_FAVC_sub_by_SCC', 'family_history_with_overweight_plus_SCC_sub_by_FAVC', 'FAVC_plus_SCC_sub_by_family_history_with_overweight', 'family_history_with_overweight_plus_SMOKE_sub_by_SCC', 'family_history_with_overweight_plus_SCC_sub_by_SMOKE', 'SMOKE_plus_SCC_sub_by_family_history_with_overweight', 'FAVC_plus_SMOKE_sub_by_SCC', 'FAVC_plus_SCC_sub_by_SMOKE', 'SMOKE_plus_SCC_sub_by_FAVC'
,'NCP_round', 'Age_round', 'Age_ceil_round', 'Public_Transportation', 'Gender', 'MTRANS', 'family_history_with_overweight', 'FAVC', 'SMOKE', 'SCC', 'Weight_2dec_x_Height_2dec_x_Gender', 'Weight_2dec_x_Height_2dec_x_CALC', 'Weight_2dec_x_Height_2dec_x_CAEC', 'Weight_2dec_x_Height_2dec_x_MTRANS', 'Weight_2dec_x_Height_2dec_x_family_history_with_overweight', 'Weight_2dec_x_Height_2dec_x_FAVC', 'Weight_2dec_x_Height_2dec_x_SMOKE', 'Weight_2dec_x_Height_2dec_x_SCC', 'Weight_2dec_x_Gender_x_CALC', 'Weight_2dec_x_Gender_x_CAEC', 'Weight_2dec_x_Gender_x_MTRANS', 'Weight_2dec_x_Gender_x_family_history_with_overweight', 'Weight_2dec_x_Gender_x_FAVC', 'Weight_2dec_x_Gender_x_SMOKE', 'Weight_2dec_x_Gender_x_SCC', 'Weight_2dec_x_CALC_x_CAEC', 'Weight_2dec_x_CALC_x_MTRANS', 'Weight_2dec_x_CALC_x_family_history_with_overweight', 'Weight_2dec_x_CALC_x_FAVC', 'Weight_2dec_x_CALC_x_SMOKE', 'Weight_2dec_x_CALC_x_SCC', 'Weight_2dec_x_CAEC_x_MTRANS', 'Weight_2dec_x_CAEC_x_family_history_with_overweight', 'Weight_2dec_x_CAEC_x_FAVC', 'Weight_2dec_x_CAEC_x_SMOKE', 'Weight_2dec_x_CAEC_x_SCC', 'Weight_2dec_x_MTRANS_x_family_history_with_overweight', 'Weight_2dec_x_MTRANS_x_FAVC', 'Weight_2dec_x_MTRANS_x_SMOKE', 'Weight_2dec_x_MTRANS_x_SCC', 'Weight_2dec_x_family_history_with_overweight_x_FAVC', 'Weight_2dec_x_family_history_with_overweight_x_SMOKE', 'Weight_2dec_x_family_history_with_overweight_x_SCC', 'Weight_2dec_x_FAVC_x_SMOKE', 'Weight_2dec_x_FAVC_x_SCC', 'Weight_2dec_x_SMOKE_x_SCC', 'Height_2dec_x_Gender_x_CALC', 'Height_2dec_x_Gender_x_CAEC', 'Height_2dec_x_Gender_x_family_history_with_overweight', 'Height_2dec_x_Gender_x_FAVC', 'Height_2dec_x_Gender_x_SCC', 'Height_2dec_x_CALC_x_family_history_with_overweight', 'Height_2dec_x_CALC_x_SMOKE', 'Height_2dec_x_CAEC_x_MTRANS', 'Height_2dec_x_CAEC_x_FAVC', 'Height_2dec_x_CAEC_x_SMOKE', 'Height_2dec_x_CAEC_x_SCC', 'Height_2dec_x_MTRANS_x_family_history_with_overweight', 'Height_2dec_x_family_history_with_overweight_x_FAVC', 'Height_2dec_x_family_history_with_overweight_x_SMOKE', 'Height_2dec_x_family_history_with_overweight_x_SCC', 'Height_2dec_x_FAVC_x_SMOKE', 'Height_2dec_x_SMOKE_x_SCC', 'Gender_x_CALC_x_CAEC', 'Gender_x_CALC_x_family_history_with_overweight', 'Gender_x_CALC_x_FAVC', 'Gender_x_CALC_x_SMOKE', 'Gender_x_CALC_x_SCC', 'Gender_x_CAEC_x_MTRANS', 'Gender_x_CAEC_x_family_history_with_overweight', 'Gender_x_CAEC_x_FAVC', 'Gender_x_CAEC_x_SMOKE', 'Gender_x_CAEC_x_SCC', 'Gender_x_MTRANS_x_family_history_with_overweight', 'Gender_x_family_history_with_overweight_x_FAVC', 'Gender_x_family_history_with_overweight_x_SMOKE', 'Gender_x_family_history_with_overweight_x_SCC', 'Gender_x_FAVC_x_SMOKE', 'Gender_x_SMOKE_x_SCC', 'CALC_x_CAEC_x_MTRANS', 'CALC_x_CAEC_x_family_history_with_overweight', 'CALC_x_CAEC_x_FAVC', 'CALC_x_CAEC_x_SMOKE', 'CALC_x_CAEC_x_SCC', 'CALC_x_MTRANS_x_family_history_with_overweight', 'CALC_x_MTRANS_x_FAVC', 'CALC_x_MTRANS_x_SMOKE', 'CALC_x_MTRANS_x_SCC', 'CALC_x_family_history_with_overweight_x_FAVC', 'CALC_x_family_history_with_overweight_x_SMOKE', 'CALC_x_family_history_with_overweight_x_SCC', 'CALC_x_FAVC_x_SMOKE', 'CALC_x_SMOKE_x_SCC', 'CAEC_x_MTRANS_x_family_history_with_overweight', 'CAEC_x_MTRANS_x_FAVC', 'CAEC_x_MTRANS_x_SMOKE', 'CAEC_x_MTRANS_x_SCC', 'CAEC_x_family_history_with_overweight_x_FAVC', 'CAEC_x_family_history_with_overweight_x_SMOKE', 'CAEC_x_family_history_with_overweight_x_SCC', 'CAEC_x_FAVC_x_SMOKE', 'CAEC_x_FAVC_x_SCC', 'CAEC_x_SMOKE_x_SCC', 'MTRANS_x_family_history_with_overweight_x_FAVC', 'MTRANS_x_family_history_with_overweight_x_SMOKE', 'MTRANS_x_family_history_with_overweight_x_SCC', 'MTRANS_x_FAVC_x_SMOKE', 'MTRANS_x_FAVC_x_SCC', 'MTRANS_x_SMOKE_x_SCC', 'family_history_with_overweight_x_FAVC_x_SMOKE', 'family_history_with_overweight_x_FAVC_x_SCC', 'family_history_with_overweight_x_SMOKE_x_SCC', 'FAVC_x_SMOKE_x_SCC'
,'NCP_round', 'Age_round', 'Age_ceil_round', 'Public_Transportation', 'Gender', 'MTRANS', 'family_history_with_overweight', 'FAVC', 'SMOKE', 'SCC', 'Weight_2dec_div_sum_of_Height_2dec_and_Gender', 'Height_2dec_div_sum_of_Weight_2dec_and_Gender', 'Gender_div_sum_of_Weight_2dec_and_Height_2dec', 'Weight_2dec_div_sum_of_Height_2dec_and_CALC', 'Height_2dec_div_sum_of_Weight_2dec_and_CALC', 'CALC_div_sum_of_Weight_2dec_and_Height_2dec', 'Height_2dec_div_sum_of_Weight_2dec_and_CAEC', 'Weight_2dec_div_sum_of_Height_2dec_and_MTRANS', 'Height_2dec_div_sum_of_Weight_2dec_and_MTRANS', 'MTRANS_div_sum_of_Weight_2dec_and_Height_2dec', 'Height_2dec_div_sum_of_Weight_2dec_and_family_history_with_overweight', 'Weight_2dec_div_sum_of_Height_2dec_and_FAVC', 'Height_2dec_div_sum_of_Weight_2dec_and_FAVC', 'FAVC_div_sum_of_Weight_2dec_and_Height_2dec', 'Weight_2dec_div_sum_of_Height_2dec_and_SMOKE', 'Height_2dec_div_sum_of_Weight_2dec_and_SMOKE', 'SMOKE_div_sum_of_Weight_2dec_and_Height_2dec', 'Weight_2dec_div_sum_of_Height_2dec_and_SCC', 'Height_2dec_div_sum_of_Weight_2dec_and_SCC', 'SCC_div_sum_of_Weight_2dec_and_Height_2dec', 'Weight_2dec_div_sum_of_Gender_and_CALC', 'Gender_div_sum_of_Weight_2dec_and_CALC', 'CALC_div_sum_of_Weight_2dec_and_Gender', 'Weight_2dec_div_sum_of_Gender_and_CAEC', 'Gender_div_sum_of_Weight_2dec_and_CAEC', 'CAEC_div_sum_of_Weight_2dec_and_Gender', 'Weight_2dec_div_sum_of_Gender_and_MTRANS', 'Gender_div_sum_of_Weight_2dec_and_MTRANS', 'MTRANS_div_sum_of_Weight_2dec_and_Gender', 'Weight_2dec_div_sum_of_Gender_and_family_history_with_overweight', 'Gender_div_sum_of_Weight_2dec_and_family_history_with_overweight', 'family_history_with_overweight_div_sum_of_Weight_2dec_and_Gender', 'Weight_2dec_div_sum_of_Gender_and_FAVC', 'Gender_div_sum_of_Weight_2dec_and_FAVC', 'FAVC_div_sum_of_Weight_2dec_and_Gender', 'Weight_2dec_div_sum_of_Gender_and_SMOKE', 'Gender_div_sum_of_Weight_2dec_and_SMOKE', 'SMOKE_div_sum_of_Weight_2dec_and_Gender', 'Weight_2dec_div_sum_of_Gender_and_SCC', 'Gender_div_sum_of_Weight_2dec_and_SCC', 'SCC_div_sum_of_Weight_2dec_and_Gender', 'Weight_2dec_div_sum_of_CALC_and_CAEC', 'CALC_div_sum_of_Weight_2dec_and_CAEC', 'CAEC_div_sum_of_Weight_2dec_and_CALC', 'Weight_2dec_div_sum_of_CALC_and_MTRANS', 'CALC_div_sum_of_Weight_2dec_and_MTRANS', 'MTRANS_div_sum_of_Weight_2dec_and_CALC', 'Weight_2dec_div_sum_of_CALC_and_family_history_with_overweight', 'CALC_div_sum_of_Weight_2dec_and_family_history_with_overweight', 'family_history_with_overweight_div_sum_of_Weight_2dec_and_CALC', 'Weight_2dec_div_sum_of_CALC_and_FAVC', 'CALC_div_sum_of_Weight_2dec_and_FAVC', 'FAVC_div_sum_of_Weight_2dec_and_CALC', 'Weight_2dec_div_sum_of_CALC_and_SMOKE', 'CALC_div_sum_of_Weight_2dec_and_SMOKE', 'SMOKE_div_sum_of_Weight_2dec_and_CALC', 'Weight_2dec_div_sum_of_CALC_and_SCC', 'CALC_div_sum_of_Weight_2dec_and_SCC', 'SCC_div_sum_of_Weight_2dec_and_CALC', 'Weight_2dec_div_sum_of_CAEC_and_MTRANS', 'CAEC_div_sum_of_Weight_2dec_and_MTRANS', 'MTRANS_div_sum_of_Weight_2dec_and_CAEC', 'CAEC_div_sum_of_Weight_2dec_and_family_history_with_overweight', 'family_history_with_overweight_div_sum_of_Weight_2dec_and_CAEC', 'Weight_2dec_div_sum_of_CAEC_and_FAVC', 'CAEC_div_sum_of_Weight_2dec_and_FAVC', 'FAVC_div_sum_of_Weight_2dec_and_CAEC', 'Weight_2dec_div_sum_of_CAEC_and_SMOKE', 'CAEC_div_sum_of_Weight_2dec_and_SMOKE', 'SMOKE_div_sum_of_Weight_2dec_and_CAEC', 'Weight_2dec_div_sum_of_CAEC_and_SCC', 'CAEC_div_sum_of_Weight_2dec_and_SCC', 'SCC_div_sum_of_Weight_2dec_and_CAEC', 'Weight_2dec_div_sum_of_MTRANS_and_family_history_with_overweight', 'MTRANS_div_sum_of_Weight_2dec_and_family_history_with_overweight', 'family_history_with_overweight_div_sum_of_Weight_2dec_and_MTRANS', 'Weight_2dec_div_sum_of_MTRANS_and_FAVC', 'MTRANS_div_sum_of_Weight_2dec_and_FAVC', 'FAVC_div_sum_of_Weight_2dec_and_MTRANS', 'Weight_2dec_div_sum_of_MTRANS_and_SMOKE', 'MTRANS_div_sum_of_Weight_2dec_and_SMOKE', 'SMOKE_div_sum_of_Weight_2dec_and_MTRANS', 'Weight_2dec_div_sum_of_MTRANS_and_SCC', 'MTRANS_div_sum_of_Weight_2dec_and_SCC', 'SCC_div_sum_of_Weight_2dec_and_MTRANS', 'Weight_2dec_div_sum_of_family_history_with_overweight_and_FAVC', 'family_history_with_overweight_div_sum_of_Weight_2dec_and_FAVC', 'FAVC_div_sum_of_Weight_2dec_and_family_history_with_overweight', 'Weight_2dec_div_sum_of_family_history_with_overweight_and_SMOKE', 'family_history_with_overweight_div_sum_of_Weight_2dec_and_SMOKE', 'SMOKE_div_sum_of_Weight_2dec_and_family_history_with_overweight', 'family_history_with_overweight_div_sum_of_Weight_2dec_and_SCC', 'SCC_div_sum_of_Weight_2dec_and_family_history_with_overweight', 'Weight_2dec_div_sum_of_FAVC_and_SMOKE', 'FAVC_div_sum_of_Weight_2dec_and_SMOKE', 'SMOKE_div_sum_of_Weight_2dec_and_FAVC', 'Weight_2dec_div_sum_of_FAVC_and_SCC', 'FAVC_div_sum_of_Weight_2dec_and_SCC', 'SCC_div_sum_of_Weight_2dec_and_FAVC', 'Weight_2dec_div_sum_of_SMOKE_and_SCC', 'SMOKE_div_sum_of_Weight_2dec_and_SCC', 'SCC_div_sum_of_Weight_2dec_and_SMOKE', 'CALC_div_sum_of_Height_2dec_and_Gender', 'Height_2dec_div_sum_of_Gender_and_CAEC', 'Gender_div_sum_of_Height_2dec_and_CAEC', 'CAEC_div_sum_of_Height_2dec_and_Gender', 'MTRANS_div_sum_of_Height_2dec_and_Gender', 'Height_2dec_div_sum_of_Gender_and_family_history_with_overweight', 'Gender_div_sum_of_Height_2dec_and_family_history_with_overweight', 'family_history_with_overweight_div_sum_of_Height_2dec_and_Gender', 'Gender_div_sum_of_Height_2dec_and_FAVC', 'FAVC_div_sum_of_Height_2dec_and_Gender', 'Height_2dec_div_sum_of_Gender_and_SMOKE', 'Gender_div_sum_of_Height_2dec_and_SMOKE', 'Gender_div_sum_of_Height_2dec_and_SCC', 'SCC_div_sum_of_Height_2dec_and_Gender', 'Height_2dec_div_sum_of_CALC_and_CAEC', 'CAEC_div_sum_of_Height_2dec_and_CALC', 'MTRANS_div_sum_of_Height_2dec_and_CALC', 'Height_2dec_div_sum_of_CALC_and_family_history_with_overweight', 'CALC_div_sum_of_Height_2dec_and_family_history_with_overweight', 'family_history_with_overweight_div_sum_of_Height_2dec_and_CALC', 'FAVC_div_sum_of_Height_2dec_and_CALC', 'Height_2dec_div_sum_of_CALC_and_SMOKE', 'CALC_div_sum_of_Height_2dec_and_SMOKE', 'SMOKE_div_sum_of_Height_2dec_and_CALC', 'SCC_div_sum_of_Height_2dec_and_CALC', 'Height_2dec_div_sum_of_CAEC_and_MTRANS', 'CAEC_div_sum_of_Height_2dec_and_MTRANS', 'MTRANS_div_sum_of_Height_2dec_and_CAEC', 'CAEC_div_sum_of_Height_2dec_and_FAVC', 'FAVC_div_sum_of_Height_2dec_and_CAEC', 'Height_2dec_div_sum_of_CAEC_and_SMOKE', 'CAEC_div_sum_of_Height_2dec_and_SMOKE', 'SMOKE_div_sum_of_Height_2dec_and_CAEC', 'CAEC_div_sum_of_Height_2dec_and_SCC', 'SCC_div_sum_of_Height_2dec_and_CAEC', 'Height_2dec_div_sum_of_MTRANS_and_family_history_with_overweight', 'MTRANS_div_sum_of_Height_2dec_and_family_history_with_overweight', 'family_history_with_overweight_div_sum_of_Height_2dec_and_MTRANS', 'FAVC_div_sum_of_Height_2dec_and_MTRANS', 'Height_2dec_div_sum_of_MTRANS_and_SMOKE', 'MTRANS_div_sum_of_Height_2dec_and_SMOKE', 'SMOKE_div_sum_of_Height_2dec_and_MTRANS', 'SCC_div_sum_of_Height_2dec_and_MTRANS', 'family_history_with_overweight_div_sum_of_Height_2dec_and_FAVC', 'FAVC_div_sum_of_Height_2dec_and_family_history_with_overweight', 'Height_2dec_div_sum_of_family_history_with_overweight_and_SMOKE', 'family_history_with_overweight_div_sum_of_Height_2dec_and_SMOKE', 'SMOKE_div_sum_of_Height_2dec_and_family_history_with_overweight', 'family_history_with_overweight_div_sum_of_Height_2dec_and_SCC', 'SCC_div_sum_of_Height_2dec_and_family_history_with_overweight', 'Height_2dec_div_sum_of_FAVC_and_SMOKE', 'FAVC_div_sum_of_Height_2dec_and_SMOKE', 'SMOKE_div_sum_of_Height_2dec_and_FAVC', 'SCC_div_sum_of_Height_2dec_and_FAVC', 'Height_2dec_div_sum_of_SMOKE_and_SCC', 'SMOKE_div_sum_of_Height_2dec_and_SCC', 'SCC_div_sum_of_Height_2dec_and_SMOKE', 'Gender_div_sum_of_CALC_and_CAEC', 'CALC_div_sum_of_Gender_and_CAEC', 'CAEC_div_sum_of_Gender_and_CALC', 'CALC_div_sum_of_Gender_and_MTRANS', 'MTRANS_div_sum_of_Gender_and_CALC', 'Gender_div_sum_of_CALC_and_family_history_with_overweight', 'CALC_div_sum_of_Gender_and_family_history_with_overweight', 'family_history_with_overweight_div_sum_of_Gender_and_CALC', 'CALC_div_sum_of_Gender_and_FAVC', 'FAVC_div_sum_of_Gender_and_CALC', 'Gender_div_sum_of_CALC_and_SMOKE', 'CALC_div_sum_of_Gender_and_SMOKE', 'SMOKE_div_sum_of_Gender_and_CALC', 'CALC_div_sum_of_Gender_and_SCC', 'SCC_div_sum_of_Gender_and_CALC', 'Gender_div_sum_of_CAEC_and_MTRANS', 'CAEC_div_sum_of_Gender_and_MTRANS', 'MTRANS_div_sum_of_Gender_and_CAEC', 'Gender_div_sum_of_CAEC_and_family_history_with_overweight', 'CAEC_div_sum_of_Gender_and_family_history_with_overweight', 'family_history_with_overweight_div_sum_of_Gender_and_CAEC', 'Gender_div_sum_of_CAEC_and_FAVC', 'CAEC_div_sum_of_Gender_and_FAVC', 'FAVC_div_sum_of_Gender_and_CAEC', 'Gender_div_sum_of_CAEC_and_SMOKE', 'CAEC_div_sum_of_Gender_and_SMOKE', 'SMOKE_div_sum_of_Gender_and_CAEC', 'Gender_div_sum_of_CAEC_and_SCC', 'CAEC_div_sum_of_Gender_and_SCC', 'SCC_div_sum_of_Gender_and_CAEC', 'Gender_div_sum_of_MTRANS_and_family_history_with_overweight', 'MTRANS_div_sum_of_Gender_and_family_history_with_overweight', 'family_history_with_overweight_div_sum_of_Gender_and_MTRANS', 'MTRANS_div_sum_of_Gender_and_FAVC', 'FAVC_div_sum_of_Gender_and_MTRANS', 'Gender_div_sum_of_MTRANS_and_SMOKE', 'MTRANS_div_sum_of_Gender_and_SMOKE', 'SMOKE_div_sum_of_Gender_and_MTRANS', 'MTRANS_div_sum_of_Gender_and_SCC', 'SCC_div_sum_of_Gender_and_MTRANS', 'Gender_div_sum_of_family_history_with_overweight_and_FAVC', 'family_history_with_overweight_div_sum_of_Gender_and_FAVC', 'FAVC_div_sum_of_Gender_and_family_history_with_overweight', 'Gender_div_sum_of_family_history_with_overweight_and_SMOKE', 'family_history_with_overweight_div_sum_of_Gender_and_SMOKE', 'SMOKE_div_sum_of_Gender_and_family_history_with_overweight', 'Gender_div_sum_of_family_history_with_overweight_and_SCC', 'family_history_with_overweight_div_sum_of_Gender_and_SCC', 'SCC_div_sum_of_Gender_and_family_history_with_overweight', 'Gender_div_sum_of_FAVC_and_SMOKE', 'FAVC_div_sum_of_Gender_and_SMOKE', 'SMOKE_div_sum_of_Gender_and_FAVC', 'FAVC_div_sum_of_Gender_and_SCC', 'SCC_div_sum_of_Gender_and_FAVC', 'Gender_div_sum_of_SMOKE_and_SCC', 'SMOKE_div_sum_of_Gender_and_SCC', 'SCC_div_sum_of_Gender_and_SMOKE', 'CALC_div_sum_of_CAEC_and_MTRANS', 'CAEC_div_sum_of_CALC_and_MTRANS', 'MTRANS_div_sum_of_CALC_and_CAEC', 'CALC_div_sum_of_CAEC_and_family_history_with_overweight', 'CAEC_div_sum_of_CALC_and_family_history_with_overweight', 'family_history_with_overweight_div_sum_of_CALC_and_CAEC', 'CAEC_div_sum_of_CALC_and_FAVC', 'FAVC_div_sum_of_CALC_and_CAEC', 'CALC_div_sum_of_CAEC_and_SMOKE', 'CAEC_div_sum_of_CALC_and_SMOKE', 'SMOKE_div_sum_of_CALC_and_CAEC', 'CAEC_div_sum_of_CALC_and_SCC', 'SCC_div_sum_of_CALC_and_CAEC', 'CALC_div_sum_of_MTRANS_and_family_history_with_overweight', 'MTRANS_div_sum_of_CALC_and_family_history_with_overweight', 'family_history_with_overweight_div_sum_of_CALC_and_MTRANS', 'CALC_div_sum_of_MTRANS_and_FAVC', 'FAVC_div_sum_of_CALC_and_MTRANS', 'CALC_div_sum_of_MTRANS_and_SMOKE', 'MTRANS_div_sum_of_CALC_and_SMOKE', 'SMOKE_div_sum_of_CALC_and_MTRANS', 'CALC_div_sum_of_MTRANS_and_SCC', 'SCC_div_sum_of_CALC_and_MTRANS', 'family_history_with_overweight_div_sum_of_CALC_and_FAVC', 'FAVC_div_sum_of_CALC_and_family_history_with_overweight', 'CALC_div_sum_of_family_history_with_overweight_and_SMOKE', 'family_history_with_overweight_div_sum_of_CALC_and_SMOKE', 'SMOKE_div_sum_of_CALC_and_family_history_with_overweight', 'family_history_with_overweight_div_sum_of_CALC_and_SCC', 'SCC_div_sum_of_CALC_and_family_history_with_overweight', 'CALC_div_sum_of_FAVC_and_SMOKE', 'FAVC_div_sum_of_CALC_and_SMOKE', 'SMOKE_div_sum_of_CALC_and_FAVC', 'CALC_div_sum_of_SMOKE_and_SCC', 'SMOKE_div_sum_of_CALC_and_SCC', 'SCC_div_sum_of_CALC_and_SMOKE', 'CAEC_div_sum_of_MTRANS_and_family_history_with_overweight', 'MTRANS_div_sum_of_CAEC_and_family_history_with_overweight', 'family_history_with_overweight_div_sum_of_CAEC_and_MTRANS', 'CAEC_div_sum_of_MTRANS_and_FAVC', 'MTRANS_div_sum_of_CAEC_and_FAVC', 'FAVC_div_sum_of_CAEC_and_MTRANS', 'CAEC_div_sum_of_MTRANS_and_SMOKE', 'MTRANS_div_sum_of_CAEC_and_SMOKE', 'SMOKE_div_sum_of_CAEC_and_MTRANS', 'CAEC_div_sum_of_MTRANS_and_SCC', 'MTRANS_div_sum_of_CAEC_and_SCC', 'SCC_div_sum_of_CAEC_and_MTRANS', 'CAEC_div_sum_of_family_history_with_overweight_and_FAVC', 'family_history_with_overweight_div_sum_of_CAEC_and_FAVC', 'FAVC_div_sum_of_CAEC_and_family_history_with_overweight', 'CAEC_div_sum_of_family_history_with_overweight_and_SMOKE', 'family_history_with_overweight_div_sum_of_CAEC_and_SMOKE', 'SMOKE_div_sum_of_CAEC_and_family_history_with_overweight', 'CAEC_div_sum_of_family_history_with_overweight_and_SCC', 'family_history_with_overweight_div_sum_of_CAEC_and_SCC', 'SCC_div_sum_of_CAEC_and_family_history_with_overweight', 'CAEC_div_sum_of_FAVC_and_SMOKE', 'FAVC_div_sum_of_CAEC_and_SMOKE', 'SMOKE_div_sum_of_CAEC_and_FAVC', 'CAEC_div_sum_of_FAVC_and_SCC', 'CAEC_div_sum_of_SMOKE_and_SCC', 'SMOKE_div_sum_of_CAEC_and_SCC', 'SCC_div_sum_of_CAEC_and_SMOKE', 'MTRANS_div_sum_of_family_history_with_overweight_and_FAVC', 'family_history_with_overweight_div_sum_of_MTRANS_and_FAVC', 'FAVC_div_sum_of_MTRANS_and_family_history_with_overweight', 'MTRANS_div_sum_of_family_history_with_overweight_and_SMOKE', 'family_history_with_overweight_div_sum_of_MTRANS_and_SMOKE', 'SMOKE_div_sum_of_MTRANS_and_family_history_with_overweight', 'MTRANS_div_sum_of_family_history_with_overweight_and_SCC', 'family_history_with_overweight_div_sum_of_MTRANS_and_SCC', 'SCC_div_sum_of_MTRANS_and_family_history_with_overweight', 'MTRANS_div_sum_of_FAVC_and_SMOKE', 'FAVC_div_sum_of_MTRANS_and_SMOKE', 'SMOKE_div_sum_of_MTRANS_and_FAVC', 'FAVC_div_sum_of_MTRANS_and_SCC', 'SCC_div_sum_of_MTRANS_and_FAVC', 'MTRANS_div_sum_of_SMOKE_and_SCC', 'SMOKE_div_sum_of_MTRANS_and_SCC', 'SCC_div_sum_of_MTRANS_and_SMOKE', 'family_history_with_overweight_div_sum_of_FAVC_and_SMOKE', 'FAVC_div_sum_of_family_history_with_overweight_and_SMOKE', 'SMOKE_div_sum_of_family_history_with_overweight_and_FAVC', 'family_history_with_overweight_div_sum_of_FAVC_and_SCC', 'family_history_with_overweight_div_sum_of_SMOKE_and_SCC', 'SMOKE_div_sum_of_family_history_with_overweight_and_SCC', 'SCC_div_sum_of_family_history_with_overweight_and_SMOKE', 'FAVC_div_sum_of_SMOKE_and_SCC', 'SMOKE_div_sum_of_FAVC_and_SCC', 'SCC_div_sum_of_FAVC_and_SMOKE'
,'NCP_round', 'Age_round', 'Age_ceil_round', 'Public_Transportation', 'Gender', 'MTRANS', 'family_history_with_overweight', 'FAVC', 'SMOKE', 'SCC', 'Weight_2dec_div_prod_of_Height_2dec_and_Gender', 'Height_2dec_div_prod_of_Weight_2dec_and_Gender', 'Gender_div_prod_of_Weight_2dec_and_Height_2dec', 'Height_2dec_div_prod_of_Weight_2dec_and_CAEC', 'Weight_2dec_div_prod_of_Height_2dec_and_MTRANS', 'MTRANS_div_prod_of_Weight_2dec_and_Height_2dec', 'Height_2dec_div_prod_of_Weight_2dec_and_family_history_with_overweight', 'Weight_2dec_div_prod_of_Height_2dec_and_SMOKE', 'Height_2dec_div_prod_of_Weight_2dec_and_SMOKE', 'SMOKE_div_prod_of_Weight_2dec_and_Height_2dec', 'Weight_2dec_div_prod_of_Gender_and_CALC', 'Gender_div_prod_of_Weight_2dec_and_CALC', 'CALC_div_prod_of_Weight_2dec_and_Gender', 'Weight_2dec_div_prod_of_Gender_and_CAEC', 'Gender_div_prod_of_Weight_2dec_and_CAEC', 'CAEC_div_prod_of_Weight_2dec_and_Gender', 'Weight_2dec_div_prod_of_Gender_and_MTRANS', 'Gender_div_prod_of_Weight_2dec_and_MTRANS', 'MTRANS_div_prod_of_Weight_2dec_and_Gender', 'Weight_2dec_div_prod_of_Gender_and_family_history_with_overweight', 'Gender_div_prod_of_Weight_2dec_and_family_history_with_overweight', 'family_history_with_overweight_div_prod_of_Weight_2dec_and_Gender', 'Weight_2dec_div_prod_of_Gender_and_FAVC', 'Gender_div_prod_of_Weight_2dec_and_FAVC', 'FAVC_div_prod_of_Weight_2dec_and_Gender', 'Weight_2dec_div_prod_of_Gender_and_SMOKE', 'Gender_div_prod_of_Weight_2dec_and_SMOKE', 'SMOKE_div_prod_of_Weight_2dec_and_Gender', 'Weight_2dec_div_prod_of_Gender_and_SCC', 'Gender_div_prod_of_Weight_2dec_and_SCC', 'SCC_div_prod_of_Weight_2dec_and_Gender', 'Weight_2dec_div_prod_of_CALC_and_CAEC', 'CALC_div_prod_of_Weight_2dec_and_CAEC', 'CAEC_div_prod_of_Weight_2dec_and_CALC', 'CALC_div_prod_of_Weight_2dec_and_MTRANS', 'MTRANS_div_prod_of_Weight_2dec_and_CALC', 'Weight_2dec_div_prod_of_CALC_and_family_history_with_overweight', 'CALC_div_prod_of_Weight_2dec_and_family_history_with_overweight', 'family_history_with_overweight_div_prod_of_Weight_2dec_and_CALC', 'CALC_div_prod_of_Weight_2dec_and_FAVC', 'Weight_2dec_div_prod_of_CALC_and_SMOKE', 'CALC_div_prod_of_Weight_2dec_and_SMOKE', 'SMOKE_div_prod_of_Weight_2dec_and_CALC', 'CALC_div_prod_of_Weight_2dec_and_SCC', 'SCC_div_prod_of_Weight_2dec_and_CALC', 'Weight_2dec_div_prod_of_CAEC_and_MTRANS', 'CAEC_div_prod_of_Weight_2dec_and_MTRANS', 'MTRANS_div_prod_of_Weight_2dec_and_CAEC', 'FAVC_div_prod_of_Weight_2dec_and_CAEC', 'Weight_2dec_div_prod_of_CAEC_and_SMOKE', 'CAEC_div_prod_of_Weight_2dec_and_SMOKE', 'SMOKE_div_prod_of_Weight_2dec_and_CAEC', 'SCC_div_prod_of_Weight_2dec_and_CAEC', 'Weight_2dec_div_prod_of_MTRANS_and_family_history_with_overweight', 'MTRANS_div_prod_of_Weight_2dec_and_family_history_with_overweight', 'family_history_with_overweight_div_prod_of_Weight_2dec_and_MTRANS', 'MTRANS_div_prod_of_Weight_2dec_and_FAVC', 'Weight_2dec_div_prod_of_MTRANS_and_SMOKE', 'MTRANS_div_prod_of_Weight_2dec_and_SMOKE', 'SMOKE_div_prod_of_Weight_2dec_and_MTRANS', 'Weight_2dec_div_prod_of_MTRANS_and_SCC', 'MTRANS_div_prod_of_Weight_2dec_and_SCC', 'SCC_div_prod_of_Weight_2dec_and_MTRANS', 'FAVC_div_prod_of_Weight_2dec_and_family_history_with_overweight', 'Weight_2dec_div_prod_of_family_history_with_overweight_and_SMOKE', 'family_history_with_overweight_div_prod_of_Weight_2dec_and_SMOKE', 'SMOKE_div_prod_of_Weight_2dec_and_family_history_with_overweight', 'SCC_div_prod_of_Weight_2dec_and_family_history_with_overweight', 'Weight_2dec_div_prod_of_FAVC_and_SMOKE', 'FAVC_div_prod_of_Weight_2dec_and_SMOKE', 'SMOKE_div_prod_of_Weight_2dec_and_FAVC', 'FAVC_div_prod_of_Weight_2dec_and_SCC', 'SCC_div_prod_of_Weight_2dec_and_FAVC', 'Weight_2dec_div_prod_of_SMOKE_and_SCC', 'SMOKE_div_prod_of_Weight_2dec_and_SCC', 'SCC_div_prod_of_Weight_2dec_and_SMOKE', 'Height_2dec_div_prod_of_Gender_and_CALC', 'CALC_div_prod_of_Height_2dec_and_Gender', 'Height_2dec_div_prod_of_Gender_and_CAEC', 'Gender_div_prod_of_Height_2dec_and_CAEC', 'CAEC_div_prod_of_Height_2dec_and_Gender', 'Height_2dec_div_prod_of_Gender_and_family_history_with_overweight', 'Gender_div_prod_of_Height_2dec_and_family_history_with_overweight', 'family_history_with_overweight_div_prod_of_Height_2dec_and_Gender', 'Height_2dec_div_prod_of_Gender_and_FAVC', 'Gender_div_prod_of_Height_2dec_and_FAVC', 'FAVC_div_prod_of_Height_2dec_and_Gender', 'Height_2dec_div_prod_of_Gender_and_SMOKE', 'Gender_div_prod_of_Height_2dec_and_SMOKE', 'Height_2dec_div_prod_of_Gender_and_SCC', 'Gender_div_prod_of_Height_2dec_and_SCC', 'SCC_div_prod_of_Height_2dec_and_Gender', 'Height_2dec_div_prod_of_CALC_and_CAEC', 'CALC_div_prod_of_Height_2dec_and_CAEC', 'Height_2dec_div_prod_of_CALC_and_family_history_with_overweight', 'CALC_div_prod_of_Height_2dec_and_family_history_with_overweight', 'Height_2dec_div_prod_of_CALC_and_SMOKE', 'CALC_div_prod_of_Height_2dec_and_SMOKE', 'SMOKE_div_prod_of_Height_2dec_and_CALC', 'Height_2dec_div_prod_of_CAEC_and_MTRANS', 'MTRANS_div_prod_of_Height_2dec_and_CAEC', 'Height_2dec_div_prod_of_CAEC_and_family_history_with_overweight', 'CAEC_div_prod_of_Height_2dec_and_family_history_with_overweight', 'family_history_with_overweight_div_prod_of_Height_2dec_and_CAEC', 'Height_2dec_div_prod_of_CAEC_and_FAVC', 'FAVC_div_prod_of_Height_2dec_and_CAEC', 'Height_2dec_div_prod_of_CAEC_and_SMOKE', 'CAEC_div_prod_of_Height_2dec_and_SMOKE', 'SMOKE_div_prod_of_Height_2dec_and_CAEC', 'Height_2dec_div_prod_of_CAEC_and_SCC', 'SCC_div_prod_of_Height_2dec_and_CAEC', 'Height_2dec_div_prod_of_MTRANS_and_family_history_with_overweight', 'MTRANS_div_prod_of_Height_2dec_and_family_history_with_overweight', 'family_history_with_overweight_div_prod_of_Height_2dec_and_MTRANS', 'MTRANS_div_prod_of_Height_2dec_and_FAVC', 'Height_2dec_div_prod_of_MTRANS_and_SMOKE', 'MTRANS_div_prod_of_Height_2dec_and_SMOKE', 'SMOKE_div_prod_of_Height_2dec_and_MTRANS', 'MTRANS_div_prod_of_Height_2dec_and_SCC', 'Height_2dec_div_prod_of_family_history_with_overweight_and_FAVC', 'FAVC_div_prod_of_Height_2dec_and_family_history_with_overweight', 'Height_2dec_div_prod_of_family_history_with_overweight_and_SMOKE', 'family_history_with_overweight_div_prod_of_Height_2dec_and_SMOKE', 'SMOKE_div_prod_of_Height_2dec_and_family_history_with_overweight', 'Height_2dec_div_prod_of_family_history_with_overweight_and_SCC', 'SCC_div_prod_of_Height_2dec_and_family_history_with_overweight', 'Height_2dec_div_prod_of_FAVC_and_SMOKE', 'FAVC_div_prod_of_Height_2dec_and_SMOKE', 'SMOKE_div_prod_of_Height_2dec_and_FAVC', 'Height_2dec_div_prod_of_SMOKE_and_SCC', 'SMOKE_div_prod_of_Height_2dec_and_SCC', 'SCC_div_prod_of_Height_2dec_and_SMOKE', 'Gender_div_prod_of_CALC_and_CAEC', 'CALC_div_prod_of_Gender_and_CAEC', 'CAEC_div_prod_of_Gender_and_CALC', 'Gender_div_prod_of_CALC_and_MTRANS', 'CALC_div_prod_of_Gender_and_MTRANS', 'MTRANS_div_prod_of_Gender_and_CALC', 'Gender_div_prod_of_CALC_and_family_history_with_overweight', 'CALC_div_prod_of_Gender_and_family_history_with_overweight', 'family_history_with_overweight_div_prod_of_Gender_and_CALC', 'Gender_div_prod_of_CALC_and_FAVC', 'CALC_div_prod_of_Gender_and_FAVC', 'FAVC_div_prod_of_Gender_and_CALC', 'Gender_div_prod_of_CALC_and_SMOKE', 'CALC_div_prod_of_Gender_and_SMOKE', 'SMOKE_div_prod_of_Gender_and_CALC', 'Gender_div_prod_of_CALC_and_SCC', 'CALC_div_prod_of_Gender_and_SCC', 'SCC_div_prod_of_Gender_and_CALC', 'Gender_div_prod_of_CAEC_and_MTRANS', 'CAEC_div_prod_of_Gender_and_MTRANS', 'MTRANS_div_prod_of_Gender_and_CAEC', 'Gender_div_prod_of_CAEC_and_family_history_with_overweight', 'CAEC_div_prod_of_Gender_and_family_history_with_overweight', 'family_history_with_overweight_div_prod_of_Gender_and_CAEC', 'Gender_div_prod_of_CAEC_and_FAVC', 'CAEC_div_prod_of_Gender_and_FAVC', 'FAVC_div_prod_of_Gender_and_CAEC', 'Gender_div_prod_of_CAEC_and_SMOKE', 'CAEC_div_prod_of_Gender_and_SMOKE', 'SMOKE_div_prod_of_Gender_and_CAEC', 'Gender_div_prod_of_CAEC_and_SCC', 'CAEC_div_prod_of_Gender_and_SCC', 'SCC_div_prod_of_Gender_and_CAEC', 'Gender_div_prod_of_MTRANS_and_family_history_with_overweight', 'MTRANS_div_prod_of_Gender_and_family_history_with_overweight', 'family_history_with_overweight_div_prod_of_Gender_and_MTRANS', 'Gender_div_prod_of_MTRANS_and_FAVC', 'MTRANS_div_prod_of_Gender_and_FAVC', 'FAVC_div_prod_of_Gender_and_MTRANS', 'Gender_div_prod_of_MTRANS_and_SMOKE', 'MTRANS_div_prod_of_Gender_and_SMOKE', 'SMOKE_div_prod_of_Gender_and_MTRANS', 'Gender_div_prod_of_MTRANS_and_SCC', 'MTRANS_div_prod_of_Gender_and_SCC', 'SCC_div_prod_of_Gender_and_MTRANS', 'Gender_div_prod_of_family_history_with_overweight_and_FAVC', 'family_history_with_overweight_div_prod_of_Gender_and_FAVC', 'FAVC_div_prod_of_Gender_and_family_history_with_overweight', 'Gender_div_prod_of_family_history_with_overweight_and_SMOKE', 'family_history_with_overweight_div_prod_of_Gender_and_SMOKE', 'SMOKE_div_prod_of_Gender_and_family_history_with_overweight', 'Gender_div_prod_of_family_history_with_overweight_and_SCC', 'family_history_with_overweight_div_prod_of_Gender_and_SCC', 'SCC_div_prod_of_Gender_and_family_history_with_overweight', 'Gender_div_prod_of_FAVC_and_SMOKE', 'FAVC_div_prod_of_Gender_and_SMOKE', 'SMOKE_div_prod_of_Gender_and_FAVC', 'Gender_div_prod_of_FAVC_and_SCC', 'FAVC_div_prod_of_Gender_and_SCC', 'SCC_div_prod_of_Gender_and_FAVC', 'Gender_div_prod_of_SMOKE_and_SCC', 'SMOKE_div_prod_of_Gender_and_SCC', 'SCC_div_prod_of_Gender_and_SMOKE', 'CALC_div_prod_of_CAEC_and_MTRANS', 'CAEC_div_prod_of_CALC_and_MTRANS', 'MTRANS_div_prod_of_CALC_and_CAEC', 'CALC_div_prod_of_CAEC_and_family_history_with_overweight', 'CAEC_div_prod_of_CALC_and_family_history_with_overweight', 'family_history_with_overweight_div_prod_of_CALC_and_CAEC', 'CALC_div_prod_of_CAEC_and_FAVC', 'FAVC_div_prod_of_CALC_and_CAEC', 'CALC_div_prod_of_CAEC_and_SMOKE', 'CAEC_div_prod_of_CALC_and_SMOKE', 'SMOKE_div_prod_of_CALC_and_CAEC', 'CALC_div_prod_of_CAEC_and_SCC', 'SCC_div_prod_of_CALC_and_CAEC', 'CALC_div_prod_of_MTRANS_and_family_history_with_overweight', 'MTRANS_div_prod_of_CALC_and_family_history_with_overweight', 'family_history_with_overweight_div_prod_of_CALC_and_MTRANS', 'MTRANS_div_prod_of_CALC_and_FAVC', 'CALC_div_prod_of_MTRANS_and_SMOKE', 'MTRANS_div_prod_of_CALC_and_SMOKE', 'SMOKE_div_prod_of_CALC_and_MTRANS', 'CALC_div_prod_of_MTRANS_and_SCC', 'MTRANS_div_prod_of_CALC_and_SCC', 'CALC_div_prod_of_family_history_with_overweight_and_FAVC', 'family_history_with_overweight_div_prod_of_CALC_and_FAVC', 'FAVC_div_prod_of_CALC_and_family_history_with_overweight', 'CALC_div_prod_of_family_history_with_overweight_and_SMOKE', 'family_history_with_overweight_div_prod_of_CALC_and_SMOKE', 'SMOKE_div_prod_of_CALC_and_family_history_with_overweight', 'CALC_div_prod_of_family_history_with_overweight_and_SCC', 'family_history_with_overweight_div_prod_of_CALC_and_SCC', 'SCC_div_prod_of_CALC_and_family_history_with_overweight', 'CALC_div_prod_of_FAVC_and_SMOKE', 'FAVC_div_prod_of_CALC_and_SMOKE', 'SMOKE_div_prod_of_CALC_and_FAVC', 'CALC_div_prod_of_FAVC_and_SCC', 'SCC_div_prod_of_CALC_and_FAVC', 'CALC_div_prod_of_SMOKE_and_SCC', 'SMOKE_div_prod_of_CALC_and_SCC', 'SCC_div_prod_of_CALC_and_SMOKE', 'CAEC_div_prod_of_MTRANS_and_family_history_with_overweight', 'MTRANS_div_prod_of_CAEC_and_family_history_with_overweight', 'family_history_with_overweight_div_prod_of_CAEC_and_MTRANS', 'MTRANS_div_prod_of_CAEC_and_FAVC', 'FAVC_div_prod_of_CAEC_and_MTRANS', 'CAEC_div_prod_of_MTRANS_and_SMOKE', 'MTRANS_div_prod_of_CAEC_and_SMOKE', 'SMOKE_div_prod_of_CAEC_and_MTRANS', 'CAEC_div_prod_of_MTRANS_and_SCC', 'MTRANS_div_prod_of_CAEC_and_SCC', 'SCC_div_prod_of_CAEC_and_MTRANS', 'CAEC_div_prod_of_family_history_with_overweight_and_FAVC', 'family_history_with_overweight_div_prod_of_CAEC_and_FAVC', 'FAVC_div_prod_of_CAEC_and_family_history_with_overweight', 'CAEC_div_prod_of_family_history_with_overweight_and_SMOKE', 'family_history_with_overweight_div_prod_of_CAEC_and_SMOKE', 'SMOKE_div_prod_of_CAEC_and_family_history_with_overweight', 'SCC_div_prod_of_CAEC_and_family_history_with_overweight', 'CAEC_div_prod_of_FAVC_and_SMOKE', 'FAVC_div_prod_of_CAEC_and_SMOKE', 'SMOKE_div_prod_of_CAEC_and_FAVC', 'FAVC_div_prod_of_CAEC_and_SCC', 'SCC_div_prod_of_CAEC_and_FAVC', 'CAEC_div_prod_of_SMOKE_and_SCC', 'SMOKE_div_prod_of_CAEC_and_SCC', 'SCC_div_prod_of_CAEC_and_SMOKE', 'MTRANS_div_prod_of_family_history_with_overweight_and_FAVC', 'family_history_with_overweight_div_prod_of_MTRANS_and_FAVC', 'FAVC_div_prod_of_MTRANS_and_family_history_with_overweight', 'MTRANS_div_prod_of_family_history_with_overweight_and_SMOKE', 'family_history_with_overweight_div_prod_of_MTRANS_and_SMOKE', 'SMOKE_div_prod_of_MTRANS_and_family_history_with_overweight', 'MTRANS_div_prod_of_family_history_with_overweight_and_SCC', 'family_history_with_overweight_div_prod_of_MTRANS_and_SCC', 'SCC_div_prod_of_MTRANS_and_family_history_with_overweight', 'MTRANS_div_prod_of_FAVC_and_SMOKE', 'FAVC_div_prod_of_MTRANS_and_SMOKE', 'SMOKE_div_prod_of_MTRANS_and_FAVC', 'MTRANS_div_prod_of_FAVC_and_SCC', 'MTRANS_div_prod_of_SMOKE_and_SCC', 'SMOKE_div_prod_of_MTRANS_and_SCC', 'SCC_div_prod_of_MTRANS_and_SMOKE', 'family_history_with_overweight_div_prod_of_FAVC_and_SMOKE', 'FAVC_div_prod_of_family_history_with_overweight_and_SMOKE', 'SMOKE_div_prod_of_family_history_with_overweight_and_FAVC', 'FAVC_div_prod_of_family_history_with_overweight_and_SCC', 'SCC_div_prod_of_family_history_with_overweight_and_FAVC', 'family_history_with_overweight_div_prod_of_SMOKE_and_SCC', 'SMOKE_div_prod_of_family_history_with_overweight_and_SCC', 'SCC_div_prod_of_family_history_with_overweight_and_SMOKE', 'FAVC_div_prod_of_SMOKE_and_SCC', 'SMOKE_div_prod_of_FAVC_and_SCC', 'SCC_div_prod_of_FAVC_and_SMOKE'
,'NCP_round', 'Age_round', 'Age_ceil_round', 'Public_Transportation', 'Gender', 'MTRANS', 'family_history_with_overweight', 'FAVC', 'SMOKE', 'SCC', 'Mean_of_Weight_2dec_Height_2dec_Gender', 'Std_of_Weight_2dec_Height_2dec_Gender', 'Mean_of_Weight_2dec_Height_2dec_CALC', 'Std_of_Weight_2dec_Height_2dec_CALC', 'Mean_of_Weight_2dec_Height_2dec_CAEC', 'Std_of_Weight_2dec_Height_2dec_CAEC', 'Mean_of_Weight_2dec_Height_2dec_MTRANS', 'Std_of_Weight_2dec_Height_2dec_MTRANS', 'Mean_of_Weight_2dec_Height_2dec_family_history_with_overweight', 'Std_of_Weight_2dec_Height_2dec_family_history_with_overweight', 'Mean_of_Weight_2dec_Height_2dec_FAVC', 'Std_of_Weight_2dec_Height_2dec_FAVC', 'Mean_of_Weight_2dec_Height_2dec_SMOKE', 'Std_of_Weight_2dec_Height_2dec_SMOKE', 'Mean_of_Weight_2dec_Height_2dec_SCC', 'Std_of_Weight_2dec_Height_2dec_SCC', 'Mean_of_Weight_2dec_Gender_CALC', 'Std_of_Weight_2dec_Gender_CALC', 'Mean_of_Weight_2dec_Gender_CAEC', 'Std_of_Weight_2dec_Gender_CAEC', 'Mean_of_Weight_2dec_Gender_MTRANS', 'Std_of_Weight_2dec_Gender_MTRANS', 'Mean_of_Weight_2dec_Gender_family_history_with_overweight', 'Std_of_Weight_2dec_Gender_family_history_with_overweight', 'Mean_of_Weight_2dec_Gender_FAVC', 'Std_of_Weight_2dec_Gender_FAVC', 'Mean_of_Weight_2dec_Gender_SMOKE', 'Std_of_Weight_2dec_Gender_SMOKE', 'Mean_of_Weight_2dec_Gender_SCC', 'Std_of_Weight_2dec_Gender_SCC', 'Mean_of_Weight_2dec_CALC_CAEC', 'Std_of_Weight_2dec_CALC_CAEC', 'Mean_of_Weight_2dec_CALC_MTRANS', 'Std_of_Weight_2dec_CALC_MTRANS', 'Mean_of_Weight_2dec_CALC_family_history_with_overweight', 'Std_of_Weight_2dec_CALC_family_history_with_overweight', 'Mean_of_Weight_2dec_CALC_FAVC', 'Std_of_Weight_2dec_CALC_FAVC', 'Mean_of_Weight_2dec_CALC_SMOKE', 'Std_of_Weight_2dec_CALC_SMOKE', 'Mean_of_Weight_2dec_CALC_SCC', 'Std_of_Weight_2dec_CALC_SCC', 'Mean_of_Weight_2dec_CAEC_MTRANS', 'Std_of_Weight_2dec_CAEC_MTRANS', 'Mean_of_Weight_2dec_CAEC_family_history_with_overweight', 'Std_of_Weight_2dec_CAEC_family_history_with_overweight', 'Mean_of_Weight_2dec_CAEC_FAVC', 'Std_of_Weight_2dec_CAEC_FAVC', 'Mean_of_Weight_2dec_CAEC_SMOKE', 'Std_of_Weight_2dec_CAEC_SMOKE', 'Mean_of_Weight_2dec_CAEC_SCC', 'Std_of_Weight_2dec_CAEC_SCC', 'Mean_of_Weight_2dec_MTRANS_family_history_with_overweight', 'Std_of_Weight_2dec_MTRANS_family_history_with_overweight', 'Mean_of_Weight_2dec_MTRANS_FAVC', 'Std_of_Weight_2dec_MTRANS_FAVC', 'Mean_of_Weight_2dec_MTRANS_SMOKE', 'Std_of_Weight_2dec_MTRANS_SMOKE', 'Mean_of_Weight_2dec_MTRANS_SCC', 'Std_of_Weight_2dec_MTRANS_SCC', 'Mean_of_Weight_2dec_family_history_with_overweight_FAVC', 'Std_of_Weight_2dec_family_history_with_overweight_FAVC', 'Mean_of_Weight_2dec_family_history_with_overweight_SMOKE', 'Std_of_Weight_2dec_family_history_with_overweight_SMOKE', 'Mean_of_Weight_2dec_family_history_with_overweight_SCC', 'Std_of_Weight_2dec_family_history_with_overweight_SCC', 'Mean_of_Weight_2dec_FAVC_SMOKE', 'Std_of_Weight_2dec_FAVC_SMOKE', 'Mean_of_Weight_2dec_FAVC_SCC', 'Std_of_Weight_2dec_FAVC_SCC', 'Mean_of_Weight_2dec_SMOKE_SCC', 'Std_of_Weight_2dec_SMOKE_SCC', 'Mean_of_Height_2dec_Gender_CALC', 'Mean_of_Height_2dec_Gender_CAEC', 'Mean_of_Height_2dec_Gender_MTRANS', 'Mean_of_Height_2dec_Gender_family_history_with_overweight', 'Mean_of_Height_2dec_Gender_FAVC', 'Std_of_Height_2dec_Gender_FAVC', 'Std_of_Height_2dec_Gender_SMOKE', 'Mean_of_Height_2dec_Gender_SCC', 'Std_of_Height_2dec_Gender_SCC', 'Mean_of_Height_2dec_CALC_family_history_with_overweight', 'Mean_of_Height_2dec_CALC_SMOKE', 'Std_of_Height_2dec_CALC_SMOKE', 'Std_of_Height_2dec_CALC_SCC', 'Mean_of_Height_2dec_CAEC_MTRANS', 'Mean_of_Height_2dec_CAEC_FAVC', 'Mean_of_Height_2dec_CAEC_SMOKE', 'Std_of_Height_2dec_CAEC_SMOKE', 'Mean_of_Height_2dec_CAEC_SCC', 'Mean_of_Height_2dec_MTRANS_family_history_with_overweight', 'Mean_of_Height_2dec_MTRANS_SMOKE', 'Std_of_Height_2dec_MTRANS_SMOKE', 'Mean_of_Height_2dec_family_history_with_overweight_FAVC', 'Mean_of_Height_2dec_family_history_with_overweight_SMOKE', 'Mean_of_Height_2dec_family_history_with_overweight_SCC', 'Mean_of_Height_2dec_FAVC_SMOKE', 'Mean_of_Height_2dec_SMOKE_SCC', 'Std_of_Height_2dec_SMOKE_SCC', 'Mean_of_Gender_CALC_CAEC', 'Std_of_Gender_CALC_CAEC', 'Mean_of_Gender_CALC_MTRANS', 'Std_of_Gender_CALC_MTRANS', 'Mean_of_Gender_CALC_family_history_with_overweight', 'Std_of_Gender_CALC_family_history_with_overweight', 'Mean_of_Gender_CALC_FAVC', 'Std_of_Gender_CALC_FAVC', 'Mean_of_Gender_CALC_SMOKE', 'Std_of_Gender_CALC_SMOKE', 'Mean_of_Gender_CALC_SCC', 'Std_of_Gender_CALC_SCC', 'Mean_of_Gender_CAEC_MTRANS', 'Std_of_Gender_CAEC_MTRANS', 'Mean_of_Gender_CAEC_family_history_with_overweight', 'Std_of_Gender_CAEC_family_history_with_overweight', 'Mean_of_Gender_CAEC_FAVC', 'Std_of_Gender_CAEC_FAVC', 'Mean_of_Gender_CAEC_SMOKE', 'Std_of_Gender_CAEC_SMOKE', 'Mean_of_Gender_CAEC_SCC', 'Std_of_Gender_CAEC_SCC', 'Mean_of_Gender_MTRANS_family_history_with_overweight', 'Std_of_Gender_MTRANS_family_history_with_overweight', 'Mean_of_Gender_MTRANS_FAVC', 'Std_of_Gender_MTRANS_FAVC', 'Mean_of_Gender_MTRANS_SMOKE', 'Std_of_Gender_MTRANS_SMOKE', 'Mean_of_Gender_MTRANS_SCC', 'Std_of_Gender_MTRANS_SCC', 'Mean_of_Gender_family_history_with_overweight_FAVC', 'Std_of_Gender_family_history_with_overweight_FAVC', 'Mean_of_Gender_family_history_with_overweight_SMOKE', 'Std_of_Gender_family_history_with_overweight_SMOKE', 'Mean_of_Gender_family_history_with_overweight_SCC', 'Std_of_Gender_family_history_with_overweight_SCC', 'Mean_of_Gender_FAVC_SMOKE', 'Std_of_Gender_FAVC_SMOKE', 'Mean_of_Gender_FAVC_SCC', 'Std_of_Gender_FAVC_SCC', 'Mean_of_Gender_SMOKE_SCC', 'Std_of_Gender_SMOKE_SCC', 'Mean_of_CALC_CAEC_MTRANS', 'Std_of_CALC_CAEC_MTRANS', 'Mean_of_CALC_CAEC_family_history_with_overweight', 'Mean_of_CALC_CAEC_FAVC', 'Std_of_CALC_CAEC_FAVC', 'Mean_of_CALC_CAEC_SMOKE', 'Std_of_CALC_CAEC_SMOKE', 'Mean_of_CALC_CAEC_SCC', 'Std_of_CALC_CAEC_SCC', 'Mean_of_CALC_MTRANS_family_history_with_overweight', 'Std_of_CALC_MTRANS_family_history_with_overweight', 'Mean_of_CALC_MTRANS_FAVC', 'Std_of_CALC_MTRANS_FAVC', 'Mean_of_CALC_MTRANS_SMOKE', 'Std_of_CALC_MTRANS_SMOKE', 'Mean_of_CALC_MTRANS_SCC', 'Std_of_CALC_MTRANS_SCC', 'Mean_of_CALC_family_history_with_overweight_FAVC', 'Std_of_CALC_family_history_with_overweight_FAVC', 'Mean_of_CALC_family_history_with_overweight_SMOKE', 'Std_of_CALC_family_history_with_overweight_SMOKE', 'Mean_of_CALC_family_history_with_overweight_SCC', 'Std_of_CALC_family_history_with_overweight_SCC', 'Mean_of_CALC_FAVC_SMOKE', 'Std_of_CALC_FAVC_SMOKE', 'Std_of_CALC_FAVC_SCC', 'Mean_of_CALC_SMOKE_SCC', 'Std_of_CALC_SMOKE_SCC', 'Mean_of_CAEC_MTRANS_family_history_with_overweight', 'Std_of_CAEC_MTRANS_family_history_with_overweight', 'Mean_of_CAEC_MTRANS_FAVC', 'Std_of_CAEC_MTRANS_FAVC', 'Mean_of_CAEC_MTRANS_SMOKE', 'Std_of_CAEC_MTRANS_SMOKE', 'Mean_of_CAEC_MTRANS_SCC', 'Std_of_CAEC_MTRANS_SCC', 'Mean_of_CAEC_family_history_with_overweight_FAVC', 'Std_of_CAEC_family_history_with_overweight_FAVC', 'Mean_of_CAEC_family_history_with_overweight_SMOKE', 'Std_of_CAEC_family_history_with_overweight_SMOKE', 'Mean_of_CAEC_family_history_with_overweight_SCC', 'Std_of_CAEC_family_history_with_overweight_SCC', 'Mean_of_CAEC_FAVC_SMOKE', 'Std_of_CAEC_FAVC_SMOKE', 'Mean_of_CAEC_FAVC_SCC', 'Std_of_CAEC_FAVC_SCC', 'Mean_of_CAEC_SMOKE_SCC', 'Std_of_CAEC_SMOKE_SCC', 'Mean_of_MTRANS_family_history_with_overweight_FAVC', 'Std_of_MTRANS_family_history_with_overweight_FAVC', 'Mean_of_MTRANS_family_history_with_overweight_SMOKE', 'Std_of_MTRANS_family_history_with_overweight_SMOKE', 'Mean_of_MTRANS_family_history_with_overweight_SCC', 'Std_of_MTRANS_family_history_with_overweight_SCC', 'Mean_of_MTRANS_FAVC_SMOKE', 'Std_of_MTRANS_FAVC_SMOKE', 'Mean_of_MTRANS_FAVC_SCC', 'Std_of_MTRANS_FAVC_SCC', 'Mean_of_MTRANS_SMOKE_SCC', 'Std_of_MTRANS_SMOKE_SCC', 'Mean_of_family_history_with_overweight_FAVC_SMOKE', 'Std_of_family_history_with_overweight_FAVC_SMOKE', 'Mean_of_family_history_with_overweight_FAVC_SCC', 'Std_of_family_history_with_overweight_FAVC_SCC', 'Mean_of_family_history_with_overweight_SMOKE_SCC', 'Std_of_family_history_with_overweight_SMOKE_SCC', 'Mean_of_FAVC_SMOKE_SCC', 'Std_of_FAVC_SMOKE_SCC'
,'NCP_round', 'Age_round', 'Age_ceil_round', 'Public_Transportation', 'Gender', 'MTRANS', 'family_history_with_overweight', 'FAVC', 'SMOKE', 'SCC', 'FCVC_round+FCVC_ceil_round', 'FCVC_round+CH2O_ceil_round', 'FCVC_round+NCP_floor_round', 'FCVC_round+NCP_round', 'FCVC_round+FAF_round', 'FCVC_round+FCVC_floor_round', 'FCVC_round+Age_floor_round', 'FCVC_round+Age_round', 'FCVC_round+Age_ceil_round', 'FCVC_round+Bike', 'FCVC_round+Motorbike', 'FCVC_round+Walking', 'FCVC_round+SMOKE_map', 'FCVC_round+SCC_map', 'FCVC_ceil_round+CH2O_round', 'FCVC_ceil_round+TUE_round', 'FCVC_ceil_round+NCP_ceil_round', 'FCVC_ceil_round+FAF_ceil_round', 'FCVC_ceil_round+CH2O_ceil_round', 'FCVC_ceil_round+NCP_floor_round', 'FCVC_ceil_round+NCP_round', 'FCVC_ceil_round+FAF_floor_round', 'FCVC_ceil_round+TUE_ceil_round', 'FCVC_ceil_round+TUE_floor_round', 'FCVC_ceil_round+FAF_round', 'FCVC_ceil_round+CH2O_floor_round', 'FCVC_ceil_round+FCVC_floor_round', 'FCVC_ceil_round+Age_floor_round', 'FCVC_ceil_round+Age_round', 'FCVC_ceil_round+Age_ceil_round', 'FCVC_ceil_round+Bike', 'FCVC_ceil_round+Motorbike', 'FCVC_ceil_round+Public_Transportation', 'FCVC_ceil_round+Walking', 'FCVC_ceil_round+SMOKE_map', 'FCVC_ceil_round+SCC_map', 'FCVC_ceil_round+CALC_map', 'FCVC_ceil_round+family_history_with_overweight_map', 'FCVC_ceil_round+CAEC_map', 'CH2O_round+CH2O_ceil_round', 'CH2O_round+NCP_floor_round', 'CH2O_round+NCP_round', 'CH2O_round+FAF_floor_round', 'CH2O_round+TUE_ceil_round', 'CH2O_round+FAF_round', 'CH2O_round+CH2O_floor_round', 'CH2O_round+FCVC_floor_round', 'CH2O_round+Age_floor_round', 'CH2O_round+Age_round', 'CH2O_round+Age_ceil_round', 'CH2O_round+Bike', 'CH2O_round+Motorbike', 'CH2O_round+Walking', 'CH2O_round+SMOKE_map', 'CH2O_round+SCC_map', 'CH2O_round+FAVC_map', 'TUE_round+CH2O_ceil_round', 'TUE_round+NCP_floor_round', 'TUE_round+NCP_round', 'TUE_round+TUE_ceil_round', 'TUE_round+TUE_floor_round', 'TUE_round+FAF_round', 'TUE_round+FCVC_floor_round', 'TUE_round+Age_floor_round', 'TUE_round+Age_round', 'TUE_round+Age_ceil_round', 'TUE_round+Bike', 'TUE_round+Motorbike', 'TUE_round+Walking', 'TUE_round+SMOKE_map', 'TUE_round+SCC_map', 'TUE_round+FAVC_map', 'NCP_ceil_round+CH2O_ceil_round', 'NCP_ceil_round+NCP_floor_round', 'NCP_ceil_round+NCP_round', 'NCP_ceil_round+FAF_floor_round', 'NCP_ceil_round+TUE_ceil_round', 'NCP_ceil_round+TUE_floor_round', 'NCP_ceil_round+FAF_round', 'NCP_ceil_round+FCVC_floor_round', 'NCP_ceil_round+Age_floor_round', 'NCP_ceil_round+Age_round', 'NCP_ceil_round+Age_ceil_round', 'NCP_ceil_round+Bike', 'NCP_ceil_round+Motorbike', 'NCP_ceil_round+Walking', 'NCP_ceil_round+SMOKE_map', 'NCP_ceil_round+SCC_map', 'NCP_ceil_round+FAVC_map', 'FAF_ceil_round+CH2O_ceil_round', 'FAF_ceil_round+NCP_floor_round', 'FAF_ceil_round+NCP_round', 'FAF_ceil_round+FAF_floor_round', 'FAF_ceil_round+TUE_ceil_round', 'FAF_ceil_round+TUE_floor_round', 'FAF_ceil_round+FAF_round', 'FAF_ceil_round+CH2O_floor_round', 'FAF_ceil_round+FCVC_floor_round', 'FAF_ceil_round+Age_floor_round', 'FAF_ceil_round+Age_round', 'FAF_ceil_round+Age_ceil_round', 'FAF_ceil_round+Automobile', 'FAF_ceil_round+Bike', 'FAF_ceil_round+Motorbike', 'FAF_ceil_round+Public_Transportation', 'FAF_ceil_round+Walking', 'FAF_ceil_round+SMOKE_map', 'FAF_ceil_round+SCC_map', 'FAF_ceil_round+family_history_with_overweight_map', 'FAF_ceil_round+FAVC_map', 'CH2O_ceil_round+NCP_floor_round', 'CH2O_ceil_round+NCP_round', 'CH2O_ceil_round+FAF_floor_round', 'CH2O_ceil_round+TUE_ceil_round', 'CH2O_ceil_round+FAF_round', 'CH2O_ceil_round+CH2O_floor_round', 'CH2O_ceil_round+FCVC_floor_round', 'CH2O_ceil_round+Age_floor_round', 'CH2O_ceil_round+Age_round', 'CH2O_ceil_round+Age_ceil_round', 'CH2O_ceil_round+Bike', 'CH2O_ceil_round+Motorbike', 'CH2O_ceil_round+Walking', 'CH2O_ceil_round+SMOKE_map', 'CH2O_ceil_round+SCC_map', 'CH2O_ceil_round+family_history_with_overweight_map', 'CH2O_ceil_round+FAVC_map', 'NCP_floor_round+NCP_round', 'NCP_floor_round+FAF_floor_round', 'NCP_floor_round+TUE_ceil_round', 'NCP_floor_round+TUE_floor_round', 'NCP_floor_round+FAF_round', 'NCP_floor_round+CH2O_floor_round', 'NCP_floor_round+FCVC_floor_round', 'NCP_floor_round+Age_floor_round', 'NCP_floor_round+Age_round', 'NCP_floor_round+Age_ceil_round', 'NCP_floor_round+Bike', 'NCP_floor_round+Motorbike', 'NCP_floor_round+Public_Transportation', 'NCP_floor_round+Walking', 'NCP_floor_round+SMOKE_map', 'NCP_floor_round+SCC_map', 'NCP_floor_round+Gender_map', 'NCP_floor_round+CALC_map', 'NCP_floor_round+family_history_with_overweight_map', 'NCP_floor_round+FAVC_map', 'NCP_floor_round+CAEC_map', 'NCP_round+FAF_floor_round', 'NCP_round+TUE_ceil_round', 'NCP_round+TUE_floor_round', 'NCP_round+FAF_round', 'NCP_round+CH2O_floor_round', 'NCP_round+FCVC_floor_round', 'NCP_round+Age_floor_round', 'NCP_round+Age_round', 'NCP_round+Age_ceil_round', 'NCP_round+Automobile', 'NCP_round+Bike', 'NCP_round+Motorbike', 'NCP_round+Public_Transportation', 'NCP_round+Walking', 'NCP_round+SMOKE_map', 'NCP_round+SCC_map', 'NCP_round+Gender_map', 'NCP_round+CALC_map', 'NCP_round+family_history_with_overweight_map', 'NCP_round+FAVC_map', 'NCP_round+CAEC_map', 'FAF_floor_round+TUE_ceil_round', 'FAF_floor_round+TUE_floor_round', 'FAF_floor_round+FAF_round', 'FAF_floor_round+CH2O_floor_round', 'FAF_floor_round+FCVC_floor_round', 'FAF_floor_round+Age_floor_round', 'FAF_floor_round+Age_round', 'FAF_floor_round+Age_ceil_round', 'FAF_floor_round+Automobile', 'FAF_floor_round+Bike', 'FAF_floor_round+Motorbike', 'FAF_floor_round+Walking', 'FAF_floor_round+SMOKE_map', 'FAF_floor_round+SCC_map', 'FAF_floor_round+FAVC_map', 'TUE_ceil_round+TUE_floor_round', 'TUE_ceil_round+FAF_round', 'TUE_ceil_round+FCVC_floor_round', 'TUE_ceil_round+Age_floor_round', 'TUE_ceil_round+Age_round', 'TUE_ceil_round+Age_ceil_round', 'TUE_ceil_round+Bike', 'TUE_ceil_round+Motorbike', 'TUE_ceil_round+Walking', 'TUE_ceil_round+SMOKE_map', 'TUE_ceil_round+SCC_map', 'TUE_ceil_round+FAVC_map', 'TUE_floor_round+FAF_round', 'TUE_floor_round+Age_floor_round', 'TUE_floor_round+Age_round', 'TUE_floor_round+Age_ceil_round', 'TUE_floor_round+Bike', 'TUE_floor_round+Motorbike', 'TUE_floor_round+Walking', 'TUE_floor_round+SMOKE_map', 'TUE_floor_round+SCC_map', 'TUE_floor_round+FAVC_map', 'FAF_round+CH2O_floor_round', 'FAF_round+FCVC_floor_round', 'FAF_round+Age_floor_round', 'FAF_round+Age_round', 'FAF_round+Age_ceil_round', 'FAF_round+Automobile', 'FAF_round+Bike', 'FAF_round+Motorbike', 'FAF_round+Public_Transportation', 'FAF_round+Walking', 'FAF_round+SMOKE_map', 'FAF_round+SCC_map', 'FAF_round+Gender_map', 'FAF_round+CALC_map', 'FAF_round+family_history_with_overweight_map', 'FAF_round+FAVC_map', 'FAF_round+CAEC_map', 'CH2O_floor_round+FCVC_floor_round', 'CH2O_floor_round+Age_floor_round', 'CH2O_floor_round+Age_round', 'CH2O_floor_round+Age_ceil_round', 'CH2O_floor_round+Bike', 'CH2O_floor_round+Motorbike', 'CH2O_floor_round+Walking', 'CH2O_floor_round+SMOKE_map', 'CH2O_floor_round+SCC_map', 'FCVC_floor_round+Age_floor_round', 'FCVC_floor_round+Age_round', 'FCVC_floor_round+Age_ceil_round', 'FCVC_floor_round+Bike', 'FCVC_floor_round+Motorbike', 'FCVC_floor_round+Walking', 'FCVC_floor_round+SMOKE_map', 'FCVC_floor_round+SCC_map', 'Age_floor_round+Age_round', 'Age_floor_round+Age_ceil_round', 'Age_floor_round+Automobile', 'Age_floor_round+Bike', 'Age_floor_round+Motorbike', 'Age_floor_round+Public_Transportation', 'Age_floor_round+Walking', 'Age_floor_round+SMOKE_map', 'Age_floor_round+SCC_map', 'Age_floor_round+Gender_map', 'Age_floor_round+CALC_map', 'Age_floor_round+family_history_with_overweight_map', 'Age_floor_round+FAVC_map', 'Age_floor_round+CAEC_map', 'Age_round+Age_ceil_round', 'Age_round+Automobile', 'Age_round+Bike', 'Age_round+Motorbike', 'Age_round+Public_Transportation', 'Age_round+Walking', 'Age_round+SMOKE_map', 'Age_round+SCC_map', 'Age_round+Gender_map', 'Age_round+CALC_map', 'Age_round+family_history_with_overweight_map', 'Age_round+FAVC_map', 'Age_round+CAEC_map', 'Age_ceil_round+Automobile', 'Age_ceil_round+Bike', 'Age_ceil_round+Motorbike', 'Age_ceil_round+Public_Transportation', 'Age_ceil_round+Walking', 'Age_ceil_round+SMOKE_map', 'Age_ceil_round+SCC_map', 'Age_ceil_round+Gender_map', 'Age_ceil_round+CALC_map', 'Age_ceil_round+family_history_with_overweight_map', 'Age_ceil_round+FAVC_map', 'Age_ceil_round+CAEC_map', 'Automobile+Bike', 'Automobile+Motorbike', 'Automobile+Public_Transportation', 'Automobile+Walking', 'Automobile+SMOKE_map', 'Automobile+SCC_map', 'Bike+Public_Transportation', 'Bike+Walking', 'Bike+SMOKE_map', 'Bike+SCC_map', 'Bike+Gender_map', 'Bike+CALC_map', 'Bike+family_history_with_overweight_map', 'Bike+FAVC_map', 'Bike+CAEC_map', 'Motorbike+Public_Transportation', 'Motorbike+Walking', 'Motorbike+SMOKE_map', 'Motorbike+SCC_map', 'Motorbike+Gender_map', 'Motorbike+CALC_map', 'Motorbike+family_history_with_overweight_map', 'Motorbike+FAVC_map', 'Motorbike+CAEC_map', 'Public_Transportation+Walking', 'Public_Transportation+SMOKE_map', 'Public_Transportation+SCC_map', 'Walking+Gender_map', 'Walking+CALC_map', 'Walking+family_history_with_overweight_map', 'Walking+CAEC_map', 'SMOKE_map+Gender_map', 'SMOKE_map+CALC_map', 'SMOKE_map+family_history_with_overweight_map', 'SMOKE_map+FAVC_map', 'SMOKE_map+CAEC_map', 'SCC_map+Gender_map', 'SCC_map+CALC_map', 'SCC_map+CAEC_map'
,'NCP_round', 'Age_round', 'Age_ceil_round', 'Public_Transportation', 'Gender', 'MTRANS', 'family_history_with_overweight', 'FAVC', 'SMOKE', 'SCC', 'BMI_2dec', 'BMI_category']))
len(pearson_high_corr_feats)

2513

In [35]:
new_train_spearman = new_train.drop(spearman_high_corr_feats, axis=1, errors='ignore')
new_train_spearman.shape

(20758, 938)

In [36]:
new_train_pearson = new_train.drop(pearson_high_corr_feats, axis=1, errors='ignore')
new_train_pearson.shape

(20758, 461)

### Remove highly correlated features

- Using Spearman correlation

In [43]:
# Remove correlated features (leaving just 1 of each pair)
# Leave features highly correlated with the target
df_no_corr = new_train_spearman.copy()
correlation_matrix_spear = df_no_corr.corr(method='spearman').abs()

# Select upper triangle of correlation matrix
upper_spear = correlation_matrix_spear.where(np.triu(np.ones(correlation_matrix_spear.shape), k=1).astype(bool))

# Find index of feature columns with correlation greater than a threshold (e.g., 0.9 in this case)
to_drop_spear = [column for column in upper_spear.columns if any(upper_spear[column] >= 0.9)]

# Drop features
df_reduced_spear = df_no_corr.drop(to_drop_spear, axis=1)

# Get list of low correlation features excluding TARGET
low_corr_feats_spear = list(df_reduced_spear.columns)

with open('new_low_corr_spear.txt', 'w') as f:
    f.write(str(low_corr_feats_spear))
    f.write('\n')

# Print the high correlation features effect
# Both pre and post drop dfs contain the TARGET
print(f"Dropped {len(to_drop_spear)} highly correlated features.\nOld Shape of the dataset was {df_no_corr.shape}\nNew shape of the dataset is {df_reduced_spear.shape}")

Dropped 603 highly correlated features.
Old Shape of the dataset was (20758, 938)
New shape of the dataset is (20758, 335)


- Using Pearson correlation

In [42]:
# Remove correlated features (leaving just 1 of each pair)
# Leave features highly correlated with the target
df_no_corr = new_train_pearson.copy()
correlation_matrix_pearson = df_no_corr.corr(method='pearson').abs()

# Select upper triangle of correlation matrix
upper_pearson = correlation_matrix_pearson.where(np.triu(np.ones(correlation_matrix_pearson.shape), k=1).astype(bool))

# Find index of feature columns with correlation greater than a threshold (e.g., 0.9 in this case)
to_drop_pearson = [column for column in upper_pearson.columns if any(upper_pearson[column] >= 0.9)]

# Drop features
df_reduced_pearson = df_no_corr.drop(to_drop_pearson, axis=1)

# Get list of low correlation features excluding TARGET
low_corr_feats_pearson = list(df_reduced_pearson.columns)

with open('new_low_corr_pearson.txt', 'w') as f:
    f.write(str(low_corr_feats_pearson))
    f.write('\n')

# Print the high correlation features effect
# Both pre and post drop dfs contain the TARGET
print(f"Dropped {len(to_drop_pearson)} highly correlated features.\nOld Shape of the dataset was {df_no_corr.shape}\nNew shape of the dataset is {df_reduced_pearson.shape}")

Dropped 187 highly correlated features.
Old Shape of the dataset was (20758, 461)
New shape of the dataset is (20758, 274)
